In [1]:
!pip install requests pandas googlemaps

Defaulting to user installation because normal site-packages is not writeable


## Portugal

### Tourist Attractions

In [24]:
import requests
import pandas as pd
import time

# 🔑 Insert your Google API Key here
API_KEY = input("Enter your Google API key: ")
PLACES_URL = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# 📍 Top 10 Most Visited Cities in Portugal with Coordinates & Search Radius (in meters)
CITIES_PORTUGAL = [
    {"name": "Lisbon", "lat": 38.7169, "lng": -9.1399, "radius": 20000},  
    {"name": "Porto", "lat": 41.1579, "lng": -8.6291, "radius": 18000},  
    {"name": "Sintra", "lat": 38.8029, "lng": -9.3817, "radius": 15000},  
    {"name": "Funchal", "lat": 32.6669, "lng": -16.9241, "radius": 15000},  
    {"name": "Coimbra", "lat": 40.2033, "lng": -8.4103, "radius": 12000},  
    {"name": "Braga", "lat": 41.5454, "lng": -8.4265, "radius": 12000},  
    {"name": "Aveiro", "lat": 40.6405, "lng": -8.6538, "radius": 10000},  
    {"name": "Nazaré", "lat": 39.6013, "lng": -9.0707, "radius": 12000},  
    {"name": "Setúbal", "lat": 38.5243, "lng": -8.8926, "radius": 12000},  
    {"name": "Faro", "lat": 37.0194, "lng": -7.9304, "radius": 15000}  
]

# Function to fetch tourist attractions for a given city
def get_places(city_name, lat, lng, radius, type_place="tourist_attraction"):
    params = {
        "location": f"{lat},{lng}",
        "radius": radius,
        "type": type_place,
        "key": API_KEY
    }
    
    places = []
    next_page_token = None

    print(f"📍 Fetching attractions in {city_name}...")
    
    while True:
        if next_page_token:
            params["pagetoken"] = next_page_token
        
        response = requests.get(PLACES_URL, params=params)
        data = response.json()
        
        if response.status_code != 200 or "results" not in data:
            print(f"⚠️ Error fetching data for {city_name}: {data.get('status', 'Unknown error')}")
            break
        
        places.extend(data["results"])
        
        next_page_token = data.get("next_page_token")
        if not next_page_token:
            break
        
        time.sleep(2)  # Google requires a small delay before using next_page_token

    return places

# Collect data for all cities
all_places = []
seen_places = set()

for city in CITIES_PORTUGAL:
    places = get_places(city["name"], city["lat"], city["lng"], city["radius"])
    
    for p in places:
        place_id = p["place_id"]
        
        if place_id not in seen_places:
            seen_places.add(place_id)
            all_places.append({
                "city": city["name"],
                "place_id": place_id,
                "name": p.get("name", "Unknown"),
                "rating": p.get("rating", "N/A"),
                "address": p.get("vicinity", "Unknown"),
                "latitude": p["geometry"]["location"]["lat"],
                "longitude": p["geometry"]["location"]["lng"],
                "types": ", ".join(p.get("types", []))
            })

    time.sleep(5)  # Avoid excessive API requests

# Save data to CSV
df_places = pd.DataFrame(all_places)
df_places.to_csv("portugal_tourist_attractions.csv", index=False)

print(f"✅ Collected {len(df_places)} tourist attractions across 10 cities in Portugal!")



📍 Fetching attractions in Lisbon...
📍 Fetching attractions in Porto...
📍 Fetching attractions in Sintra...
📍 Fetching attractions in Funchal...
📍 Fetching attractions in Coimbra...
📍 Fetching attractions in Braga...
📍 Fetching attractions in Aveiro...
📍 Fetching attractions in Nazaré...
📍 Fetching attractions in Setúbal...
📍 Fetching attractions in Faro...
✅ Collected 597 tourist attractions across 10 cities in Portugal!


### Hotels

In [41]:
import requests 
import pandas as pd
import time

OVERPASS_URL = "https://overpass-api.de/api/interpreter"

CITIES_PORTUGAL = [
    {"name": "Lisbon", "lat": 38.7169, "lng": -9.1399, "radius": 20000},  
    {"name": "Porto", "lat": 41.1579, "lng": -8.6291, "radius": 18000},  
    {"name": "Sintra", "lat": 38.8029, "lng": -9.3817, "radius": 15000},  
    {"name": "Funchal", "lat": 32.6669, "lng": -16.9241, "radius": 15000},  
    {"name": "Coimbra", "lat": 40.2033, "lng": -8.4103, "radius": 12000},  
    {"name": "Braga", "lat": 41.5454, "lng": -8.4265, "radius": 12000},  
    {"name": "Aveiro", "lat": 40.6405, "lng": -8.6538, "radius": 10000},  
    {"name": "Nazaré", "lat": 39.6013, "lng": -9.0707, "radius": 12000},  
    {"name": "Setúbal", "lat": 38.5243, "lng": -8.8926, "radius": 12000},  
    {"name": "Faro", "lat": 37.0194, "lng": -7.9304, "radius": 15000}  
]

def get_osm_hotels(lat, lon):
    query = f"""
    [out:json];
    (
      node["tourism"="hotel"](around:10000,{lat},{lon});
      node["tourism"="hostel"](around:10000,{lat},{lon});
      node["tourism"="guest_house"](around:10000,{lat},{lon});
    );
    out center;
    """
    
    response = requests.get(OVERPASS_URL, params={"data": query})
    return response.json().get("elements", []) if response.status_code == 200 else []

all_hotels = []
for city in CITIES_PORTUGAL:
    print(f"📍 Fetching hotels and hostels in {city['name']}...")
    hotels = get_osm_hotels(city["lat"], city["lng"])

    for hotel in hotels:
        tags = hotel.get("tags", {})

        all_hotels.append({
            "city": city["name"],
            "name": tags.get("name", "Unknown"),
            "type": tags.get("tourism", "Unknown"),
            "latitude": hotel.get("lat"),
            "longitude": hotel.get("lon"),
            "stars": tags.get("stars", "Unknown"),
            "operator": tags.get("operator", "Unknown")
        })
    time.sleep(2)

df_hotels = pd.DataFrame(all_hotels)
df_hotels.to_csv("portugal_hotels_hostels.csv", index=False)

print(f"✅ Collected {len(df_hotels)} hotels and hostels across 10 cities in Portugal!")


📍 Fetching hotels and hostels in Lisbon...
📍 Fetching hotels and hostels in Porto...
📍 Fetching hotels and hostels in Sintra...
📍 Fetching hotels and hostels in Funchal...
📍 Fetching hotels and hostels in Coimbra...
📍 Fetching hotels and hostels in Braga...
📍 Fetching hotels and hostels in Aveiro...
📍 Fetching hotels and hostels in Nazaré...
📍 Fetching hotels and hostels in Setúbal...
📍 Fetching hotels and hostels in Faro...
✅ Collected 1154 hotels and hostels across 10 cities in Portugal!


### Restaurants

In [13]:
import requests
import pandas as pd
import time

OVERPASS_URL = "https://overpass-api.de/api/interpreter"

# 📍 Top 10 Most Visited Cities in Portugal with Coordinates & Search Radius (in meters)
CITIES_PORTUGAL = [
    {"name": "Lisbon", "lat": 38.7169, "lon": -9.1399, "radius": 10000},
    {"name": "Porto", "lat": 41.1579, "lon": -8.6291, "radius": 10000},
    {"name": "Sintra", "lat": 38.8029, "lon": -9.3817, "radius": 8000},
    {"name": "Funchal", "lat": 32.6669, "lon": -16.9241, "radius": 8000},
    {"name": "Coimbra", "lat": 40.2033, "lon": -8.4103, "radius": 8000},
    {"name": "Braga", "lat": 41.5454, "lon": -8.4265, "radius": 8000},
    {"name": "Aveiro", "lat": 40.6405, "lon": -8.6538, "radius": 8000},
    {"name": "Nazaré", "lat": 39.6013, "lon": -9.0707, "radius": 6000},
    {"name": "Setúbal", "lat": 38.5243, "lon": -8.8926, "radius": 8000},
    {"name": "Faro", "lat": 37.0194, "lon": -7.9304, "radius": 8000}
]

def get_osm_restaurants(city):
    """Busca restaurantes, cafés e bares em uma cidade específica de Portugal."""
    query = f"""
    [out:json];
    (
      node["amenity"="restaurant"](around:{city["radius"]},{city["lat"]},{city["lon"]});
      node["amenity"="cafe"](around:{city["radius"]},{city["lat"]},{city["lon"]});
      node["amenity"="bar"](around:{city["radius"]},{city["lat"]},{city["lon"]});
    );
    out center;
    """
    
    response = requests.get(OVERPASS_URL, params={"data": query})
    
    if response.status_code == 200:
        return response.json().get("elements", [])
    else:
        print(f"❌ Error fetching restaurants in {city['name']}")
        return []

# 🔍 Collect data for all cities
all_places = []
for city in CITIES_PORTUGAL:
    print(f"📍 Fetching gastronomic locations in {city['name']}...")
    places = get_osm_restaurants(city)

    for place in places:
        tags = place.get("tags", {})

        all_places.append({
            "city": city["name"],
            "name": tags.get("name", "Unknown"),
            "type": tags.get("amenity", "Unknown"),
            "latitude": place.get("lat"),
            "longitude": place.get("lon"),
            "street": tags.get("addr:street", "Unknown"),
            "postcode": tags.get("addr:postcode", "Unknown"),
            "cuisine": tags.get("cuisine", "Not specified")  # Type of food
        })

    time.sleep(2)  # Avoid overloading the API

# 📝 Save data to CSV
df = pd.DataFrame(all_places)
df.to_csv("portugal_all_restaurants_bars_cafes.csv", index=False)

print(f"✅ Collected {len(df)} gastronomic locations across 10 cities in Portugal! Saved as 'portugal_all_restaurants_bars_cafes.csv'")


📍 Fetching gastronomic locations in Lisbon...
📍 Fetching gastronomic locations in Porto...
📍 Fetching gastronomic locations in Sintra...
📍 Fetching gastronomic locations in Funchal...
📍 Fetching gastronomic locations in Coimbra...
📍 Fetching gastronomic locations in Braga...
📍 Fetching gastronomic locations in Aveiro...
📍 Fetching gastronomic locations in Nazaré...
📍 Fetching gastronomic locations in Setúbal...
📍 Fetching gastronomic locations in Faro...
✅ Collected 11201 gastronomic locations across 10 cities in Portugal! Saved as 'portugal_all_restaurants_bars_cafes.csv'


### Events

In [7]:
import requests
import pandas as pd
import time

# 🔑 Solicita a chave da API do usuário
API_KEY = input("Enter your Google API key: ")
PLACES_URL = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# 📍 Top 10 Most Visited Cities in Portugal with Coordinates & Search Radius (in meters)
CITIES_PORTUGAL = [
    {"name": "Lisbon", "lat": 38.7169, "lng": -9.1399, "radius": 15000},
    {"name": "Porto", "lat": 41.1579, "lng": -8.6291, "radius": 15000},
    {"name": "Sintra", "lat": 38.8029, "lng": -9.3817, "radius": 10000},
    {"name": "Funchal", "lat": 32.6669, "lng": -16.9241, "radius": 10000},
    {"name": "Coimbra", "lat": 40.2033, "lng": -8.4103, "radius": 10000},
    {"name": "Braga", "lat": 41.5454, "lng": -8.4265, "radius": 10000},
    {"name": "Aveiro", "lat": 40.6405, "lng": -8.6538, "radius": 10000},
    {"name": "Nazaré", "lat": 39.6013, "lng": -9.0707, "radius": 8000},
    {"name": "Setúbal", "lat": 38.5243, "lng": -8.8926, "radius": 10000},
    {"name": "Faro", "lat": 37.0194, "lng": -7.9304, "radius": 10000}
]

# 🎭 Definir categorias de locais onde eventos podem ocorrer
EVENT_TYPES = [
    "stadium", "casino", "night_club", "art_gallery", "museum",
    "tourist_attraction", "amusement_park", "university", "movie_theater"
]

# Função para buscar locais de eventos para uma cidade
def get_places(city_name, lat, lng, radius):
    places = []
    seen_places = set()

    for place_type in EVENT_TYPES:
        params = {
            "location": f"{lat},{lng}",
            "radius": radius,
            "type": place_type,
            "key": API_KEY
        }
        
        print(f"📍 Fetching event venues in {city_name} ({place_type})...")
        
        response = requests.get(PLACES_URL, params=params)
        data = response.json()

        if response.status_code == 200 and "results" in data:
            for p in data["results"]:
                place_id = p["place_id"]

                if place_id not in seen_places:
                    seen_places.add(place_id)
                    places.append({
                        "city": city_name,
                        "place_id": place_id,
                        "name": p.get("name", "Unknown"),
                        "rating": p.get("rating", "N/A"),
                        "address": p.get("vicinity", "Unknown"),
                        "latitude": p["geometry"]["location"]["lat"],
                        "longitude": p["geometry"]["location"]["lng"],
                        "types": ", ".join(p.get("types", []))
                    })
        
        # Aguarde um pouco para evitar excesso de chamadas na API
        time.sleep(2)
    
    return places

# 🔍 Coletar dados de locais de eventos para todas as cidades
all_places = []
for city in CITIES_PORTUGAL:
    all_places.extend(get_places(city["name"], city["lat"], city["lng"], city["radius"]))

# 📝 Salvar os dados em um arquivo CSV
df_places = pd.DataFrame(all_places)
df_places.to_csv("portugal_event_locations.csv", index=False)

print(f"✅ Collected {len(df_places)} event venues across 10 cities in Portugal! Saved as 'portugal_event_locations.csv'")


📍 Fetching event venues in Lisbon (stadium)...
📍 Fetching event venues in Lisbon (casino)...
📍 Fetching event venues in Lisbon (night_club)...
📍 Fetching event venues in Lisbon (art_gallery)...
📍 Fetching event venues in Lisbon (museum)...
📍 Fetching event venues in Lisbon (tourist_attraction)...
📍 Fetching event venues in Lisbon (amusement_park)...
📍 Fetching event venues in Lisbon (university)...
📍 Fetching event venues in Lisbon (movie_theater)...
📍 Fetching event venues in Porto (stadium)...
📍 Fetching event venues in Porto (casino)...
📍 Fetching event venues in Porto (night_club)...
📍 Fetching event venues in Porto (art_gallery)...
📍 Fetching event venues in Porto (museum)...
📍 Fetching event venues in Porto (tourist_attraction)...
📍 Fetching event venues in Porto (amusement_park)...
📍 Fetching event venues in Porto (university)...
📍 Fetching event venues in Porto (movie_theater)...
📍 Fetching event venues in Sintra (stadium)...
📍 Fetching event venues in Sintra (casino)...
📍 Fetc

### Public Transport Stops

In [28]:
import requests
import pandas as pd
import time

OVERPASS_URL = "https://overpass-api.de/api/interpreter"

# 📍 Top 10 Most Visited Cities in Portugal
CITIES_PORTUGAL = [
    {"name": "Lisbon", "lat": 38.7169, "lon": -9.1399, "radius": 10000},
    {"name": "Porto", "lat": 41.1579, "lon": -8.6291, "radius": 8000},
    {"name": "Sintra", "lat": 38.8029, "lon": -9.3817, "radius": 8000},
    {"name": "Funchal", "lat": 32.6669, "lon": -16.9241, "radius": 8000},
    {"name": "Coimbra", "lat": 40.2033, "lon": -8.4103, "radius": 8000},
    {"name": "Braga", "lat": 41.5454, "lon": -8.4265, "radius": 8000},
    {"name": "Aveiro", "lat": 40.6405, "lon": -8.6538, "radius": 7000},
    {"name": "Nazaré", "lat": 39.6013, "lon": -9.0707, "radius": 7000},
    {"name": "Setúbal", "lat": 38.5243, "lon": -8.8926, "radius": 7000},
    {"name": "Faro", "lat": 37.0194, "lon": -7.9304, "radius": 7000}
]

def get_osm_transit_stops(city):
    """Fetches bus stops, railway stations, and public transport platforms in a specific city in Portugal."""
    query = f"""
    [out:json];
    (
      node["highway"="bus_stop"](around:{city["radius"]},{city["lat"]},{city["lon"]});
      node["railway"="station"](around:{city["radius"]},{city["lat"]},{city["lon"]});
      node["public_transport"="platform"](around:{city["radius"]},{city["lat"]},{city["lon"]});
    );
    out center;
    """

    retries = 3
    while retries > 0:
        response = requests.get(OVERPASS_URL, params={"data": query})

        if response.status_code == 200:
            return response.json().get("elements", [])
        else:
            print(f"⚠️ Error fetching transport data in {city['name']} ({response.status_code}), retrying... ({3 - retries + 1}/3)")
            retries -= 1
            time.sleep(3)

    print(f"❌ Failed to fetch data for {city['name']} after multiple retries.")
    return []

# 🔍 Collect data for all cities
all_stops = []
for city in CITIES_PORTUGAL:
    print(f"📍 Fetching public transport stops in {city['name']}...")
    stops = get_osm_transit_stops(city)

    for stop in stops:
        tags = stop.get("tags", {})

        all_stops.append({
            "city": city["name"],
            "name": tags.get("name", "Unknown"),
            "type": tags.get("highway", tags.get("railway", tags.get("public_transport", "Unknown"))),
            "latitude": stop.get("lat"),
            "longitude": stop.get("lon"),
            "operator": tags.get("operator", "Unknown")  # Transport company if available
        })

    time.sleep(2)  # Avoid API overload

# 📝 Save data to CSV
df_transit = pd.DataFrame(all_stops)
df_transit.to_csv("portugal_transit_stops.csv", index=False)

print(f"✅ Collected {len(df_transit)} public transport stops across 10 cities in Portugal! Saved as 'portugal_transit_stops.csv'")


📍 Fetching public transport stops in Lisbon...
📍 Fetching public transport stops in Porto...
📍 Fetching public transport stops in Sintra...
📍 Fetching public transport stops in Funchal...
📍 Fetching public transport stops in Coimbra...
⚠️ Error fetching transport data in Coimbra (504), retrying... (1/3)
📍 Fetching public transport stops in Braga...
📍 Fetching public transport stops in Aveiro...
📍 Fetching public transport stops in Nazaré...
📍 Fetching public transport stops in Setúbal...
📍 Fetching public transport stops in Faro...
✅ Collected 14771 public transport stops across 10 cities in Portugal! Saved as 'portugal_transit_stops.csv'


### Nature Spots

In [51]:
import requests
import pandas as pd
import time

OVERPASS_URL = "https://overpass-api.de/api/interpreter"

CITIES_PORTUGAL = [
    {"name": "Lisbon", "lat": 38.7169, "lng": -9.1399, "radius": 20000},  
    {"name": "Porto", "lat": 41.1579, "lng": -8.6291, "radius": 18000},  
    {"name": "Sintra", "lat": 38.8029, "lng": -9.3817, "radius": 15000},  
    {"name": "Funchal", "lat": 32.6669, "lng": -16.9241, "radius": 15000},  
    {"name": "Coimbra", "lat": 40.2033, "lng": -8.4103, "radius": 12000},  
    {"name": "Braga", "lat": 41.5454, "lng": -8.4265, "radius": 12000},  
    {"name": "Aveiro", "lat": 40.6405, "lng": -8.6538, "radius": 10000},  
    {"name": "Nazaré", "lat": 39.6013, "lng": -9.0707, "radius": 12000},  
    {"name": "Setúbal", "lat": 38.5243, "lng": -8.8926, "radius": 12000},  
    {"name": "Faro", "lat": 37.0194, "lng": -7.9304, "radius": 15000}  
]

def get_osm_nature_spots(lat, lon):
    query = f"""
    [out:json];
    (
      node["leisure"="nature_reserve"](around:10000,{lat},{lon});
      node["leisure"="park"](around:10000,{lat},{lon});
      node["route"="hiking"](around:10000,{lat},{lon});
      node["piste:type"="downhill"](around:10000,{lat},{lon});
      node["natural"="peak"](around:10000,{lat},{lon});
    );
    out center;
    """
    
    response = requests.get(OVERPASS_URL, params={"data": query})
    return response.json().get("elements", []) if response.status_code == 200 else []

all_nature_spots = []
for city in CITIES_PORTUGAL:
    print(f"📍 Fetching nature spots in {city['name']}...")
    nature_spots = get_osm_nature_spots(city["lat"], city["lng"])

    for spot in nature_spots:
        tags = spot.get("tags", {})

        all_nature_spots.append({
            "city": city["name"],
            "name": tags.get("name", "Unknown"),
            "type": tags.get("leisure", tags.get("route", tags.get("natural", "Unknown"))),
            "latitude": spot.get("lat"),
            "longitude": spot.get("lon"),
            "elevation": tags.get("ele", "Unknown")
        })
    time.sleep(2)

df_nature = pd.DataFrame(all_nature_spots)
df_nature.to_csv("portugal_nature_spots.csv", index=False)

print(f"✅ Collected {len(df_nature)} nature spots across 10 cities in Portugal!")


📍 Fetching nature spots in Lisbon...
📍 Fetching nature spots in Porto...
📍 Fetching nature spots in Sintra...
📍 Fetching nature spots in Funchal...
📍 Fetching nature spots in Coimbra...
📍 Fetching nature spots in Braga...
📍 Fetching nature spots in Aveiro...
📍 Fetching nature spots in Nazaré...
📍 Fetching nature spots in Setúbal...
📍 Fetching nature spots in Faro...
✅ Collected 184 nature spots across 10 cities in Portugal!


### Weather

In [64]:
import os
import requests
import pandas as pd
import time

# Insira sua chave da API do OpenWeather
OPENWEATHER_KEY = input("Enter your OpenWeather API key: ")
WEATHER_URL = "https://api.openweathermap.org/data/2.5/weather"

CITIES_PORTUGAL = [
    {"name": "Lisbon", "lat": 38.7169, "lng": -9.1399},  
    {"name": "Porto", "lat": 41.1579, "lng": -8.6291},  
    {"name": "Sintra", "lat": 38.8029, "lng": -9.3817},  
    {"name": "Funchal", "lat": 32.6669, "lng": -16.9241},  
    {"name": "Coimbra", "lat": 40.2033, "lng": -8.4103},  
    {"name": "Braga", "lat": 41.5454, "lng": -8.4265},  
    {"name": "Aveiro", "lat": 40.6405, "lng": -8.6538},  
    {"name": "Nazaré", "lat": 39.6013, "lng": -9.0707},  
    {"name": "Setúbal", "lat": 38.5243, "lng": -8.8926},  
    {"name": "Faro", "lat": 37.0194, "lng": -7.9304}  
]

# Função para buscar o clima baseado em latitude e longitude
def get_weather(lat, lon):
    params = {
        "lat": lat,
        "lon": lon,
        "appid": OPENWEATHER_KEY,
        "units": "metric"  # Celsius
    }
    response = requests.get(WEATHER_URL, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"❌ Erro ao buscar clima para {lat}, {lon}: {response.status_code}")
        return None

# Coletar dados climáticos para as cidades de Portugal
all_weather_data = []

for city in CITIES_PORTUGAL:
    print(f"Fetching weather data for {city['name']}...")
    weather_data = get_weather(city["lat"], city["lng"])
    
    if weather_data:
        weather_info = {
            "city": city["name"],
            "latitude": city["lat"],
            "longitude": city["lng"],
            "temperature": weather_data["main"]["temp"],
            "weather": weather_data["weather"][0]["description"],
            "humidity": weather_data["main"]["humidity"],
            "wind_speed": weather_data["wind"]["speed"]
        }
        all_weather_data.append(weather_info)
    time.sleep(1)  # Evita exceder limites da API

# Criar DataFrame e salvar os dados climáticos
df_weather = pd.DataFrame(all_weather_data)
df_weather.to_csv("portugal_weather_data.csv", index=False)

print(f"✅ Dados climáticos de {len(df_weather)} cidades de Portugal coletados e salvos!")

Fetching weather data for Lisbon...
Fetching weather data for Porto...
Fetching weather data for Sintra...
Fetching weather data for Funchal...
Fetching weather data for Coimbra...
Fetching weather data for Braga...
Fetching weather data for Aveiro...
Fetching weather data for Nazaré...
Fetching weather data for Setúbal...
Fetching weather data for Faro...
✅ Dados climáticos de 10 cidades de Portugal coletados e salvos!


## Switzerland

### Tourist Attractions

In [25]:
import requests
import pandas as pd
import time

# 🔑 Insert your Google API Key here
API_KEY = input("Enter your Google API key: ")
PLACES_URL = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# 📍 Top 10 Most Visited Cities in Switzerland with Coordinates & Search Radius (in meters)
CITIES_SWITZERLAND = [
    {"name": "Zurich", "lat": 47.3769, "lng": 8.5417, "radius": 20000},  
    {"name": "Geneva", "lat": 46.2044, "lng": 6.1432, "radius": 18000},  
    {"name": "Lucerne", "lat": 47.0505, "lng": 8.3059, "radius": 15000},  
    {"name": "Zermatt", "lat": 46.0207, "lng": 7.7491, "radius": 15000},  
    {"name": "Interlaken", "lat": 46.6863, "lng": 7.8632, "radius": 12000},  
    {"name": "Bern", "lat": 46.9481, "lng": 7.4474, "radius": 12000},  
    {"name": "Lausanne", "lat": 46.5197, "lng": 6.6323, "radius": 10000},  
    {"name": "Basel", "lat": 47.5596, "lng": 7.5886, "radius": 12000},  
    {"name": "Lugano", "lat": 46.005, "lng": 8.9511, "radius": 10000},  
    {"name": "Grindelwald", "lat": 46.6242, "lng": 8.0344, "radius": 15000}  
]

# Function to fetch tourist attractions for a given city
def get_places(city_name, lat, lng, radius, type_place="tourist_attraction"):
    params = {
        "location": f"{lat},{lng}",
        "radius": radius,
        "type": type_place,
        "key": API_KEY
    }
    
    places = []
    next_page_token = None

    print(f"📍 Fetching attractions in {city_name}...")
    
    while True:
        if next_page_token:
            params["pagetoken"] = next_page_token
        
        response = requests.get(PLACES_URL, params=params)
        data = response.json()
        
        if response.status_code != 200 or "results" not in data:
            print(f"⚠️ Error fetching data for {city_name}: {data.get('status', 'Unknown error')}")
            break
        
        places.extend(data["results"])
        
        next_page_token = data.get("next_page_token")
        if not next_page_token:
            break
        
        time.sleep(2)  # Google requires a small delay before using next_page_token

    return places

# Collect data for all cities
all_places = []
seen_places = set()

for city in CITIES_SWITZERLAND:
    places = get_places(city["name"], city["lat"], city["lng"], city["radius"])
    
    for p in places:
        place_id = p["place_id"]
        
        if place_id not in seen_places:
            seen_places.add(place_id)
            all_places.append({
                "city": city["name"],
                "place_id": place_id,
                "name": p.get("name", "Unknown"),
                "rating": p.get("rating", "N/A"),
                "address": p.get("vicinity", "Unknown"),
                "latitude": p["geometry"]["location"]["lat"],
                "longitude": p["geometry"]["location"]["lng"],
                "types": ", ".join(p.get("types", []))
            })

    time.sleep(5)  # Avoid excessive API requests

# Save data to CSV
df_places = pd.DataFrame(all_places)
df_places.to_csv("switzerland_tourist_attractions.csv", index=False)

print(f"✅ Collected {len(df_places)} tourist attractions across 10 cities in Switzerland!")


📍 Fetching attractions in Zurich...
📍 Fetching attractions in Geneva...
📍 Fetching attractions in Lucerne...
📍 Fetching attractions in Zermatt...
📍 Fetching attractions in Interlaken...
📍 Fetching attractions in Bern...
📍 Fetching attractions in Lausanne...
📍 Fetching attractions in Basel...
📍 Fetching attractions in Lugano...
📍 Fetching attractions in Grindelwald...
✅ Collected 592 tourist attractions across 10 cities in Switzerland!


### Events

In [63]:
import requests
import pandas as pd
import time

# 🔑 Solicita a chave da API do usuário
API_KEY = input("Enter your Google API key: ")
PLACES_URL = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# 📍 Top 10 Most Visited Cities in Switzerland with Coordinates & Search Radius (in meters)
CITIES_SWITZERLAND = [
    {"name": "Zurich", "lat": 47.3769, "lng": 8.5417, "radius": 15000},
    {"name": "Geneva", "lat": 46.2044, "lng": 6.1432, "radius": 15000},
    {"name": "Lucerne", "lat": 47.0505, "lng": 8.3059, "radius": 12000},
    {"name": "Zermatt", "lat": 46.0207, "lng": 7.7491, "radius": 10000},
    {"name": "Interlaken", "lat": 46.6863, "lng": 7.8632, "radius": 10000},
    {"name": "Bern", "lat": 46.9481, "lng": 7.4474, "radius": 12000},
    {"name": "Lausanne", "lat": 46.5197, "lng": 6.6323, "radius": 12000},
    {"name": "Basel", "lat": 47.5596, "lng": 7.5886, "radius": 12000},
    {"name": "Lugano", "lat": 46.005, "lng": 8.9511, "radius": 10000},
    {"name": "Grindelwald", "lat": 46.6242, "lng": 8.0344, "radius": 10000}
]

# 🎭 Definir categorias de locais onde eventos podem ocorrer
EVENT_TYPES = [
    "stadium", "casino", "night_club", "art_gallery", "museum",
    "tourist_attraction", "amusement_park", "university", "movie_theater"
]

# Função para buscar locais de eventos para uma cidade
def get_places(city_name, lat, lng, radius):
    places = []
    seen_places = set()

    for place_type in EVENT_TYPES:
        params = {
            "location": f"{lat},{lng}",
            "radius": radius,
            "type": place_type,
            "key": API_KEY
        }
        
        print(f"📍 Fetching event venues in {city_name} ({place_type})...")
        
        response = requests.get(PLACES_URL, params=params)
        data = response.json()

        if response.status_code == 200 and "results" in data:
            for p in data["results"]:
                place_id = p["place_id"]

                if place_id not in seen_places:
                    seen_places.add(place_id)
                    places.append({
                        "city": city_name,
                        "place_id": place_id,
                        "name": p.get("name", "Unknown"),
                        "rating": p.get("rating", "N/A"),
                        "address": p.get("vicinity", "Unknown"),
                        "latitude": p["geometry"]["location"]["lat"],
                        "longitude": p["geometry"]["location"]["lng"],
                        "types": ", ".join(p.get("types", []))
                    })
        
        # Aguarde um pouco para evitar excesso de chamadas na API
        time.sleep(2)
    
    return places

# 🔍 Coletar dados de locais de eventos para todas as cidades
all_places = []
for city in CITIES_SWITZERLAND:
    all_places.extend(get_places(city["name"], city["lat"], city["lng"], city["radius"]))

# 📝 Salvar os dados em um arquivo CSV
df_places = pd.DataFrame(all_places)
df_places.to_csv("switzerland_event_locations.csv", index=False)

print(f"✅ Collected {len(df_places)} event venues across 10 cities in Switzerland! Saved as 'switzerland_event_locations.csv'")


📍 Fetching event venues in Zurich (stadium)...
📍 Fetching event venues in Zurich (casino)...
📍 Fetching event venues in Zurich (night_club)...
📍 Fetching event venues in Zurich (art_gallery)...
📍 Fetching event venues in Zurich (museum)...
📍 Fetching event venues in Zurich (tourist_attraction)...
📍 Fetching event venues in Zurich (amusement_park)...
📍 Fetching event venues in Zurich (university)...
📍 Fetching event venues in Zurich (movie_theater)...
📍 Fetching event venues in Geneva (stadium)...
📍 Fetching event venues in Geneva (casino)...
📍 Fetching event venues in Geneva (night_club)...
📍 Fetching event venues in Geneva (art_gallery)...
📍 Fetching event venues in Geneva (museum)...
📍 Fetching event venues in Geneva (tourist_attraction)...
📍 Fetching event venues in Geneva (amusement_park)...
📍 Fetching event venues in Geneva (university)...
📍 Fetching event venues in Geneva (movie_theater)...
📍 Fetching event venues in Lucerne (stadium)...
📍 Fetching event venues in Lucerne (casino

### Hotels

In [42]:
import requests 
import pandas as pd
import time

OVERPASS_URL = "https://overpass-api.de/api/interpreter"

CITIES_SWITZERLAND = [
    {"name": "Zurich", "lat": 47.3769, "lng": 8.5417, "radius": 20000},  
    {"name": "Geneva", "lat": 46.2044, "lng": 6.1432, "radius": 18000},  
    {"name": "Lucerne", "lat": 47.0505, "lng": 8.3059, "radius": 15000},  
    {"name": "Zermatt", "lat": 46.0207, "lng": 7.7491, "radius": 15000},  
    {"name": "Interlaken", "lat": 46.6863, "lng": 7.8632, "radius": 12000},  
    {"name": "Bern", "lat": 46.9481, "lng": 7.4474, "radius": 12000},  
    {"name": "Lausanne", "lat": 46.5197, "lng": 6.6323, "radius": 10000},  
    {"name": "Basel", "lat": 47.5596, "lng": 7.5886, "radius": 12000},  
    {"name": "Lugano", "lat": 46.005, "lng": 8.9511, "radius": 10000},  
    {"name": "Grindelwald", "lat": 46.6242, "lng": 8.0344, "radius": 15000}  
]

def get_osm_hotels(lat, lon):
    query = f"""
    [out:json];
    (
      node["tourism"="hotel"](around:10000,{lat},{lon});
      node["tourism"="hostel"](around:10000,{lat},{lon});
      node["tourism"="guest_house"](around:10000,{lat},{lon});
    );
    out center;
    """
    
    response = requests.get(OVERPASS_URL, params={"data": query})
    return response.json().get("elements", []) if response.status_code == 200 else []

all_hotels = []
for city in CITIES_SWITZERLAND:
    print(f"📍 Fetching hotels and hostels in {city['name']}...")
    hotels = get_osm_hotels(city["lat"], city["lng"])

    for hotel in hotels:
        tags = hotel.get("tags", {})

        all_hotels.append({
            "city": city["name"],
            "name": tags.get("name", "Unknown"),
            "type": tags.get("tourism", "Unknown"),
            "latitude": hotel.get("lat"),
            "longitude": hotel.get("lon"),
            "stars": tags.get("stars", "Unknown"),
            "operator": tags.get("operator", "Unknown")
        })
    time.sleep(2)

df_hotels = pd.DataFrame(all_hotels)
df_hotels.to_csv("switzerland_hotels_hostels.csv", index=False)

print(f"✅ Collected {len(df_hotels)} hotels and hostels across 10 cities in Switzerland!")


📍 Fetching hotels and hostels in Zurich...
📍 Fetching hotels and hostels in Geneva...
📍 Fetching hotels and hostels in Lucerne...
📍 Fetching hotels and hostels in Zermatt...
📍 Fetching hotels and hostels in Interlaken...
📍 Fetching hotels and hostels in Bern...
📍 Fetching hotels and hostels in Lausanne...
📍 Fetching hotels and hostels in Basel...
📍 Fetching hotels and hostels in Lugano...
📍 Fetching hotels and hostels in Grindelwald...
✅ Collected 990 hotels and hostels across 10 cities in Switzerland!


### Restaurants

In [15]:
import requests
import pandas as pd
import time

OVERPASS_URL = "https://overpass-api.de/api/interpreter"

# 📍 Top 10 Most Visited Cities in Switzerland
CITIES_SWITZERLAND = [
    {"name": "Zurich", "lat": 47.3769, "lon": 8.5417, "radius": 10000},
    {"name": "Geneva", "lat": 46.2044, "lon": 6.1432, "radius": 10000},
    {"name": "Lucerne", "lat": 47.0505, "lon": 8.3059, "radius": 10000},
    {"name": "Zermatt", "lat": 46.0207, "lon": 7.7491, "radius": 8000},
    {"name": "Interlaken", "lat": 46.6863, "lon": 7.8632, "radius": 8000},
    {"name": "Bern", "lat": 46.9481, "lon": 7.4474, "radius": 10000},
    {"name": "Lausanne", "lat": 46.5197, "lon": 6.6323, "radius": 10000},
    {"name": "Basel", "lat": 47.5596, "lon": 7.5886, "radius": 10000},
    {"name": "Lugano", "lat": 46.005, "lon": 8.9511, "radius": 8000},
    {"name": "Grindelwald", "lat": 46.6242, "lon": 8.0344, "radius": 8000}
]

def get_osm_restaurants(city):
    """Fetches restaurants, cafés, and bars in a specific Swiss city with retries."""
    query = f"""
    [out:json];
    (
      node["amenity"="restaurant"](around:{city["radius"]},{city["lat"]},{city["lon"]});
      node["amenity"="cafe"](around:{city["radius"]},{city["lat"]},{city["lon"]});
      node["amenity"="bar"](around:{city["radius"]},{city["lat"]},{city["lon"]});
    );
    out center;
    """

    retries = 3
    while retries > 0:
        response = requests.get(OVERPASS_URL, params={"data": query})
        
        if response.status_code == 200:
            return response.json().get("elements", [])
        else:
            print(f"⚠️ Error fetching restaurants in {city['name']} ({response.status_code}), retrying... ({3 - retries + 1}/3)")
            retries -= 1
            time.sleep(3)  # Aguardar antes de tentar novamente
    
    print(f"❌ Failed to fetch data for {city['name']} after multiple retries.")
    return []

# 🔍 Collect data for all cities
all_places = []
for city in CITIES_SWITZERLAND:
    print(f"📍 Fetching gastronomic locations in {city['name']}...")
    places = get_osm_restaurants(city)

    for place in places:
        tags = place.get("tags", {})

        all_places.append({
            "city": city["name"],
            "name": tags.get("name", "Unknown"),
            "type": tags.get("amenity", "Unknown"),
            "latitude": place.get("lat"),
            "longitude": place.get("lon"),
            "street": tags.get("addr:street", "Unknown"),
            "postcode": tags.get("addr:postcode", "Unknown"),
            "cuisine": tags.get("cuisine", "Not specified")  # Type of food
        })

    time.sleep(2)  # Avoid API rate limits

# 📝 Save data to CSV
df = pd.DataFrame(all_places)
df.to_csv("switzerland_all_restaurants_bars_cafes.csv", index=False)

print(f"✅ Collected {len(df)} gastronomic locations across 10 cities in Switzerland! Saved as 'switzerland_all_restaurants_bars_cafes.csv'")



📍 Fetching gastronomic locations in Zurich...
📍 Fetching gastronomic locations in Geneva...
📍 Fetching gastronomic locations in Lucerne...
📍 Fetching gastronomic locations in Zermatt...
📍 Fetching gastronomic locations in Interlaken...
📍 Fetching gastronomic locations in Bern...
⚠️ Error fetching restaurants in Bern (504), retrying... (1/3)
📍 Fetching gastronomic locations in Lausanne...
📍 Fetching gastronomic locations in Basel...
📍 Fetching gastronomic locations in Lugano...
📍 Fetching gastronomic locations in Grindelwald...
✅ Collected 7282 gastronomic locations across 10 cities in Switzerland! Saved as 'switzerland_all_restaurants_bars_cafes.csv'


### Public Transport Stops

In [29]:
import requests
import pandas as pd
import time

OVERPASS_URL = "https://overpass-api.de/api/interpreter"

CITIES_SWITZERLAND = [
    {"name": "Zurich", "lat": 47.3769, "lon": 8.5417, "radius": 10000},
    {"name": "Geneva", "lat": 46.2044, "lon": 6.1432, "radius": 10000},
    {"name": "Lucerne", "lat": 47.0505, "lon": 8.3059, "radius": 8000},
    {"name": "Zermatt", "lat": 46.0207, "lon": 7.7491, "radius": 8000},
    {"name": "Interlaken", "lat": 46.6863, "lon": 7.8632, "radius": 8000},
    {"name": "Bern", "lat": 46.9481, "lon": 7.4474, "radius": 8000},
    {"name": "Lausanne", "lat": 46.5197, "lon": 6.6323, "radius": 8000},
    {"name": "Basel", "lat": 47.5596, "lon": 7.5886, "radius": 8000},
    {"name": "Lugano", "lat": 46.005, "lon": 8.9511, "radius": 7000},
    {"name": "Grindelwald", "lat": 46.6242, "lon": 8.0344, "radius": 7000}
]

def get_osm_transit_stops(city):
    query = f"""
    [out:json];
    (
      node["highway"="bus_stop"](around:{city["radius"]},{city["lat"]},{city["lon"]});
      node["railway"="station"](around:{city["radius"]},{city["lat"]},{city["lon"]});
      node["public_transport"="platform"](around:{city["radius"]},{city["lat"]},{city["lon"]});
    );
    out center;
    """

    response = requests.get(OVERPASS_URL, params={"data": query})
    return response.json().get("elements", []) if response.status_code == 200 else []

all_stops = []
for city in CITIES_SWITZERLAND:
    print(f"📍 Fetching public transport stops in {city['name']}...")
    stops = get_osm_transit_stops(city)

    for stop in stops:
        tags = stop.get("tags", {})

        all_stops.append({
            "city": city["name"],
            "name": tags.get("name", "Unknown"),
            "type": tags.get("highway", tags.get("railway", tags.get("public_transport", "Unknown"))),
            "latitude": stop.get("lat"),
            "longitude": stop.get("lon"),
            "operator": tags.get("operator", "Unknown")
        })

df_transit = pd.DataFrame(all_stops)
df_transit.to_csv("switzerland_transit_stops.csv", index=False)

print(f"✅ Collected {len(df_transit)} public transport stops across 10 cities in Switzerland!")


📍 Fetching public transport stops in Zurich...
📍 Fetching public transport stops in Geneva...
📍 Fetching public transport stops in Lucerne...
📍 Fetching public transport stops in Zermatt...
📍 Fetching public transport stops in Interlaken...
📍 Fetching public transport stops in Bern...
📍 Fetching public transport stops in Lausanne...
📍 Fetching public transport stops in Basel...
📍 Fetching public transport stops in Lugano...
📍 Fetching public transport stops in Grindelwald...
✅ Collected 8096 public transport stops across 10 cities in Switzerland!


### Nature Spots

In [52]:
import requests
import pandas as pd
import time

OVERPASS_URL = "https://overpass-api.de/api/interpreter"

CITIES_SWITZERLAND = [
    {"name": "Zurich", "lat": 47.3769, "lng": 8.5417, "radius": 20000},  
    {"name": "Geneva", "lat": 46.2044, "lng": 6.1432, "radius": 18000},  
    {"name": "Lucerne", "lat": 47.0505, "lng": 8.3059, "radius": 15000},  
    {"name": "Zermatt", "lat": 46.0207, "lng": 7.7491, "radius": 15000},  
    {"name": "Interlaken", "lat": 46.6863, "lng": 7.8632, "radius": 12000},  
    {"name": "Bern", "lat": 46.9481, "lng": 7.4474, "radius": 12000},  
    {"name": "Lausanne", "lat": 46.5197, "lng": 6.6323, "radius": 10000},  
    {"name": "Basel", "lat": 47.5596, "lng": 7.5886, "radius": 12000},  
    {"name": "Lugano", "lat": 46.005, "lng": 8.9511, "radius": 10000},  
    {"name": "Grindelwald", "lat": 46.6242, "lng": 8.0344, "radius": 15000}  
]

def get_osm_nature_spots(lat, lon):
    query = f"""
    [out:json];
    (
      node["leisure"="nature_reserve"](around:10000,{lat},{lon});
      node["leisure"="park"](around:10000,{lat},{lon});
      node["route"="hiking"](around:10000,{lat},{lon});
      node["piste:type"="downhill"](around:10000,{lat},{lon});
      node["natural"="peak"](around:10000,{lat},{lon});
    );
    out center;
    """
    
    response = requests.get(OVERPASS_URL, params={"data": query})
    return response.json().get("elements", []) if response.status_code == 200 else []

all_nature_spots = []
for city in CITIES_SWITZERLAND:
    print(f"📍 Fetching nature spots in {city['name']}...")
    nature_spots = get_osm_nature_spots(city["lat"], city["lng"])

    for spot in nature_spots:
        tags = spot.get("tags", {})

        all_nature_spots.append({
            "city": city["name"],
            "name": tags.get("name", "Unknown"),
            "type": tags.get("leisure", tags.get("route", tags.get("natural", "Unknown"))),
            "latitude": spot.get("lat"),
            "longitude": spot.get("lon"),
            "elevation": tags.get("ele", "Unknown")
        })
    time.sleep(2)

df_nature = pd.DataFrame(all_nature_spots)
df_nature.to_csv("switzerland_nature_spots.csv", index=False)

print(f"✅ Collected {len(df_nature)} nature spots across 10 cities in Switzerland!")


📍 Fetching nature spots in Zurich...
📍 Fetching nature spots in Geneva...
📍 Fetching nature spots in Lucerne...
📍 Fetching nature spots in Zermatt...
📍 Fetching nature spots in Interlaken...
📍 Fetching nature spots in Bern...
📍 Fetching nature spots in Lausanne...
📍 Fetching nature spots in Basel...
📍 Fetching nature spots in Lugano...
📍 Fetching nature spots in Grindelwald...
✅ Collected 517 nature spots across 10 cities in Switzerland!


### Weather

In [65]:
import os
import requests
import pandas as pd
import time

# Enter your OpenWeather API key
OPENWEATHER_KEY = input("Enter your OpenWeather API key: ")
WEATHER_URL = "https://api.openweathermap.org/data/2.5/weather"

CITIES_SWITZERLAND = [
    {"name": "Zurich", "lat": 47.3769, "lng": 8.5417},  
    {"name": "Geneva", "lat": 46.2044, "lng": 6.1432},  
    {"name": "Lucerne", "lat": 47.0505, "lng": 8.3059},  
    {"name": "Zermatt", "lat": 46.0207, "lng": 7.7491},  
    {"name": "Interlaken", "lat": 46.6863, "lng": 7.8632},  
    {"name": "Bern", "lat": 46.9481, "lng": 7.4474},  
    {"name": "Lausanne", "lat": 46.5197, "lng": 6.6323},  
    {"name": "Basel", "lat": 47.5596, "lng": 7.5886},  
    {"name": "Lugano", "lat": 46.005, "lng": 8.9511},  
    {"name": "Grindelwald", "lat": 46.6242, "lng": 8.0344}  
]

# Function to fetch weather data based on latitude and longitude
def get_weather(lat, lon):
    params = {
        "lat": lat,
        "lon": lon,
        "appid": OPENWEATHER_KEY,
        "units": "metric"  # Celsius
    }
    response = requests.get(WEATHER_URL, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"❌ Error fetching weather for {lat}, {lon}: {response.status_code}")
        return None

# Collect weather data for Swiss cities
all_weather_data = []

for city in CITIES_SWITZERLAND:
    print(f"Fetching weather data for {city['name']}...")
    weather_data = get_weather(city["lat"], city["lng"])
    
    if weather_data:
        weather_info = {
            "city": city["name"],
            "latitude": city["lat"],
            "longitude": city["lng"],
            "temperature": weather_data["main"]["temp"],
            "weather": weather_data["weather"][0]["description"],
            "humidity": weather_data["main"]["humidity"],
            "wind_speed": weather_data["wind"]["speed"]
        }
        all_weather_data.append(weather_info)
    time.sleep(1)  # Avoid exceeding API limits

# Create DataFrame and save weather data
df_weather = pd.DataFrame(all_weather_data)
df_weather.to_csv("switzerland_weather_data.csv", index=False)

print(f"✅ Weather data for {len(df_weather)} Swiss cities collected and saved!")


Fetching weather data for Zurich...
Fetching weather data for Geneva...
Fetching weather data for Lucerne...
Fetching weather data for Zermatt...
Fetching weather data for Interlaken...
Fetching weather data for Bern...
Fetching weather data for Lausanne...
Fetching weather data for Basel...
Fetching weather data for Lugano...
Fetching weather data for Grindelwald...
✅ Weather data for 10 Swiss cities collected and saved!


## Greece

### Tourist Attractions

In [61]:
import requests
import pandas as pd
import time

# 🔑 Insert your Google API Key here
API_KEY = input("Enter your Google API key: ")
PLACES_URL = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# 📍 Top 10 Most Visited Cities in Greece with Coordinates & Search Radius (in meters)
CITIES_GREECE = [
    {"name": "Athens", "lat": 37.9838, "lng": 23.7275, "radius": 20000},  
    {"name": "Thessaloniki", "lat": 40.6401, "lng": 22.9444, "radius": 18000},  
    {"name": "Santorini", "lat": 36.3932, "lng": 25.4615, "radius": 15000},  
    {"name": "Mykonos", "lat": 37.4467, "lng": 25.3289, "radius": 15000},  
    {"name": "Rhodes", "lat": 36.4349, "lng": 28.2176, "radius": 12000},  
    {"name": "Corfu", "lat": 39.6249, "lng": 19.9223, "radius": 12000},  
    {"name": "Heraklion", "lat": 35.3387, "lng": 25.1442, "radius": 10000},  
    {"name": "Chania", "lat": 35.5112, "lng": 24.0297, "radius": 12000},  
    {"name": "Nafplio", "lat": 37.5671, "lng": 22.8047, "radius": 10000},  
    {"name": "Delphi", "lat": 38.4828, "lng": 22.5010, "radius": 15000}  
]

# Function to fetch tourist attractions for a given city
def get_places(city_name, lat, lng, radius, type_place="tourist_attraction"):
    params = {
        "location": f"{lat},{lng}",
        "radius": radius,
        "type": type_place,
        "key": API_KEY
    }
    
    places = []
    next_page_token = None

    print(f"📍 Fetching attractions in {city_name}...")
    
    while True:
        if next_page_token:
            params["pagetoken"] = next_page_token
        
        response = requests.get(PLACES_URL, params=params)
        data = response.json()
        
        if response.status_code != 200 or "results" not in data:
            print(f"⚠️ Error fetching data for {city_name}: {data.get('status', 'Unknown error')}")
            break
        
        places.extend(data["results"])
        
        next_page_token = data.get("next_page_token")
        if not next_page_token:
            break
        
        time.sleep(2)  # Google requires a small delay before using next_page_token

    return places

# Collect data for all cities
all_places = []
seen_places = set()

for city in CITIES_GREECE:
    places = get_places(city["name"], city["lat"], city["lng"], city["radius"])
    
    for p in places:
        place_id = p["place_id"]
        
        if place_id not in seen_places:
            seen_places.add(place_id)
            all_places.append({
                "city": city["name"],
                "place_id": place_id,
                "name": p.get("name", "Unknown"),
                "rating": p.get("rating", "N/A"),
                "address": p.get("vicinity", "Unknown"),
                "latitude": p["geometry"]["location"]["lat"],
                "longitude": p["geometry"]["location"]["lng"],
                "types": ", ".join(p.get("types", []))
            })

    time.sleep(5)  # Avoid excessive API requests

# Save data to CSV
df_places = pd.DataFrame(all_places)
df_places.to_csv("greece_tourist_attractions.csv", index=False)

print(f"✅ Collected {len(df_places)} tourist attractions across 10 cities in Greece!")


📍 Fetching attractions in Athens...
📍 Fetching attractions in Thessaloniki...
📍 Fetching attractions in Santorini...
📍 Fetching attractions in Mykonos...


KeyboardInterrupt: 

### Events

In [62]:
import requests
import pandas as pd
import time

# 🔑 Solicita a chave da API do usuário
API_KEY = input("Enter your Google API key: ")
PLACES_URL = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# 📍 Top 10 Most Visited Cities in Greece with Coordinates & Search Radius (in meters)
CITIES_GREECE = [
    {"name": "Athens", "lat": 37.9838, "lng": 23.7275, "radius": 15000},
    {"name": "Thessaloniki", "lat": 40.6401, "lng": 22.9444, "radius": 12000},
    {"name": "Santorini", "lat": 36.3932, "lng": 25.4615, "radius": 10000},
    {"name": "Mykonos", "lat": 37.4467, "lng": 25.3289, "radius": 10000},
    {"name": "Rhodes", "lat": 36.4341, "lng": 28.2176, "radius": 12000},
    {"name": "Corfu", "lat": 39.6243, "lng": 19.9217, "radius": 12000},
    {"name": "Heraklion", "lat": 35.3387, "lng": 25.1442, "radius": 12000},
    {"name": "Chania", "lat": 35.5138, "lng": 24.0180, "radius": 12000},
    {"name": "Nafplio", "lat": 37.5639, "lng": 22.8091, "radius": 10000},
    {"name": "Delphi", "lat": 38.4828, "lng": 22.5010, "radius": 10000}
]

# 🎭 Define categories of places where events may occur
EVENT_TYPES = [
    "stadium", "casino", "night_club", "art_gallery", "museum",
    "tourist_attraction", "amusement_park", "university", "movie_theater"
]

# Function to fetch event venues for a city
def get_places(city_name, lat, lng, radius):
    places = []
    seen_places = set()

    for place_type in EVENT_TYPES:
        params = {
            "location": f"{lat},{lng}",
            "radius": radius,
            "type": place_type,
            "key": API_KEY
        }
        
        print(f"📍 Fetching event venues in {city_name} ({place_type})...")
        
        response = requests.get(PLACES_URL, params=params)
        data = response.json()

        if response.status_code == 200 and "results" in data:
            for p in data["results"]:
                place_id = p["place_id"]

                if place_id not in seen_places:
                    seen_places.add(place_id)
                    places.append({
                        "city": city_name,
                        "place_id": place_id,
                        "name": p.get("name", "Unknown"),
                        "rating": p.get("rating", "N/A"),
                        "address": p.get("vicinity", "Unknown"),
                        "latitude": p["geometry"]["location"]["lat"],
                        "longitude": p["geometry"]["location"]["lng"],
                        "types": ", ".join(p.get("types", []))
                    })
        
        # Wait a bit to avoid exceeding API request limits
        time.sleep(2)
    
    return places

# 🔍 Collect event venue data for all cities
all_places = []
for city in CITIES_GREECE:
    all_places.extend(get_places(city["name"], city["lat"], city["lng"], city["radius"]))

# 📝 Save data to CSV
df_places = pd.DataFrame(all_places)
df_places.to_csv("greece_event_locations.csv", index=False)

print(f"✅ Collected {len(df_places)} event venues across 10 cities in Greece! Saved as 'greece_event_locations.csv'")


📍 Fetching event venues in Athens (stadium)...
📍 Fetching event venues in Athens (casino)...
📍 Fetching event venues in Athens (night_club)...
📍 Fetching event venues in Athens (art_gallery)...
📍 Fetching event venues in Athens (museum)...
📍 Fetching event venues in Athens (tourist_attraction)...
📍 Fetching event venues in Athens (amusement_park)...
📍 Fetching event venues in Athens (university)...
📍 Fetching event venues in Athens (movie_theater)...
📍 Fetching event venues in Thessaloniki (stadium)...
📍 Fetching event venues in Thessaloniki (casino)...
📍 Fetching event venues in Thessaloniki (night_club)...
📍 Fetching event venues in Thessaloniki (art_gallery)...
📍 Fetching event venues in Thessaloniki (museum)...
📍 Fetching event venues in Thessaloniki (tourist_attraction)...
📍 Fetching event venues in Thessaloniki (amusement_park)...
📍 Fetching event venues in Thessaloniki (university)...
📍 Fetching event venues in Thessaloniki (movie_theater)...
📍 Fetching event venues in Santorini

### Hotels

In [ ]:
import requests 
import pandas as pd
import time

OVERPASS_URL = "https://overpass-api.de/api/interpreter"

CITIES_GREECE = [
    {"name": "Athens", "lat": 37.9838, "lng": 23.7275, "radius": 20000},  
    {"name": "Thessaloniki", "lat": 40.6401, "lng": 22.9444, "radius": 18000},  
    {"name": "Santorini", "lat": 36.3932, "lng": 25.4615, "radius": 15000},  
    {"name": "Mykonos", "lat": 37.4467, "lng": 25.3289, "radius": 15000},  
    {"name": "Rhodes", "lat": 36.4349, "lng": 28.2176, "radius": 12000},  
    {"name": "Corfu", "lat": 39.6249, "lng": 19.9223, "radius": 12000},  
    {"name": "Heraklion", "lat": 35.3387, "lng": 25.1442, "radius": 10000},  
    {"name": "Chania", "lat": 35.5112, "lng": 24.0297, "radius": 12000},  
    {"name": "Nafplio", "lat": 37.5671, "lng": 22.8047, "radius": 10000},  
    {"name": "Delphi", "lat": 38.4828, "lng": 22.5010, "radius": 15000}  
]

def get_osm_hotels(lat, lon):
    query = f"""
    [out:json];
    (
      node["tourism"="hotel"](around:10000,{lat},{lon});
      node["tourism"="hostel"](around:10000,{lat},{lon});
      node["tourism"="guest_house"](around:10000,{lat},{lon});
    );
    out center;
    """
    
    response = requests.get(OVERPASS_URL, params={"data": query})
    return response.json().get("elements", []) if response.status_code == 200 else []

all_hotels = []
for city in CITIES_GREECE:
    print(f"📍 Fetching hotels and hostels in {city['name']}...")
    hotels = get_osm_hotels(city["lat"], city["lng"])

    for hotel in hotels:
        tags = hotel.get("tags", {})

        all_hotels.append({
            "city": city["name"],
            "name": tags.get("name", "Unknown"),
            "type": tags.get("tourism", "Unknown"),
            "latitude": hotel.get("lat"),
            "longitude": hotel.get("lon"),
            "stars": tags.get("stars", "Unknown"),
            "operator": tags.get("operator", "Unknown")
        })
    time.sleep(2)

df_hotels = pd.DataFrame(all_hotels)
df_hotels.to_csv("greece_hotels_hostels.csv", index=False)

print(f"✅ Collected {len(df_hotels)} hotels and hostels across 10 cities in Greece!")


📍 Fetching hotels and hostels in Athens...
📍 Fetching hotels and hostels in Thessaloniki...
📍 Fetching hotels and hostels in Santorini...
📍 Fetching hotels and hostels in Mykonos...
📍 Fetching hotels and hostels in Rhodes...
📍 Fetching hotels and hostels in Corfu...
📍 Fetching hotels and hostels in Heraklion...
📍 Fetching hotels and hostels in Chania...
📍 Fetching hotels and hostels in Nafplio...
📍 Fetching hotels and hostels in Delphi...
✅ Collected 1211 hotels and hostels across 10 cities in Greece!


### Restaurants

In [ ]:
import requests
import pandas as pd
import time

OVERPASS_URL = "https://overpass-api.de/api/interpreter"

# 📍 Top 10 Most Visited Cities in Greece
CITIES_GREECE = [
    {"name": "Athens", "lat": 37.9838, "lon": 23.7275, "radius": 10000},
    {"name": "Thessaloniki", "lat": 40.6401, "lon": 22.9444, "radius": 10000},
    {"name": "Santorini", "lat": 36.3932, "lon": 25.4615, "radius": 7000},
    {"name": "Mykonos", "lat": 37.4467, "lon": 25.3289, "radius": 7000},
    {"name": "Rhodes", "lat": 36.4344, "lon": 28.2170, "radius": 7000},
    {"name": "Corfu", "lat": 39.6249, "lon": 19.9223, "radius": 7000},
    {"name": "Heraklion", "lat": 35.3387, "lon": 25.1442, "radius": 10000},
    {"name": "Chania", "lat": 35.5138, "lon": 24.0180, "radius": 8000},
    {"name": "Nafplio", "lat": 37.5639, "lon": 22.8075, "radius": 8000},
    {"name": "Delphi", "lat": 38.4826, "lon": 22.5010, "radius": 7000}
]

def get_osm_restaurants(city):
    """Fetches restaurants, cafés, and bars in a specific Greek city with retries."""
    query = f"""
    [out:json];
    (
      node["amenity"="restaurant"](around:{city["radius"]},{city["lat"]},{city["lon"]});
      node["amenity"="cafe"](around:{city["radius"]},{city["lat"]},{city["lon"]});
      node["amenity"="bar"](around:{city["radius"]},{city["lat"]},{city["lon"]});
    );
    out center;
    """

    retries = 3
    while retries > 0:
        response = requests.get(OVERPASS_URL, params={"data": query})
        
        if response.status_code == 200:
            return response.json().get("elements", [])
        else:
            print(f"⚠️ Error fetching restaurants in {city['name']} ({response.status_code}), retrying... ({3 - retries + 1}/3)")
            retries -= 1
            time.sleep(3)  # Aguardar antes de tentar novamente
    
    print(f"❌ Failed to fetch data for {city['name']} after multiple retries.")
    return []

# 🔍 Collect data for all cities
all_places = []
for city in CITIES_GREECE:
    print(f"📍 Fetching gastronomic locations in {city['name']}...")
    places = get_osm_restaurants(city)

    for place in places:
        tags = place.get("tags", {})

        all_places.append({
            "city": city["name"],
            "name": tags.get("name", "Unknown"),
            "type": tags.get("amenity", "Unknown"),
            "latitude": place.get("lat"),
            "longitude": place.get("lon"),
            "street": tags.get("addr:street", "Unknown"),
            "postcode": tags.get("addr:postcode", "Unknown"),
            "cuisine": tags.get("cuisine", "Not specified")  # Type of food
        })

    time.sleep(2)  # Avoid API rate limits

# 📝 Save data to CSV
df = pd.DataFrame(all_places)
df.to_csv("greece_all_restaurants_bars_cafes.csv", index=False)

print(f"✅ Collected {len(df)} gastronomic locations across 10 cities in Greece! Saved as 'greece_all_restaurants_bars_cafes.csv'")


📍 Fetching gastronomic locations in Athens...
📍 Fetching gastronomic locations in Thessaloniki...
📍 Fetching gastronomic locations in Santorini...
📍 Fetching gastronomic locations in Mykonos...
📍 Fetching gastronomic locations in Rhodes...
📍 Fetching gastronomic locations in Corfu...
📍 Fetching gastronomic locations in Heraklion...
📍 Fetching gastronomic locations in Chania...
📍 Fetching gastronomic locations in Nafplio...
📍 Fetching gastronomic locations in Delphi...
⚠️ Error fetching restaurants in Delphi (504), retrying... (1/3)
✅ Collected 8310 gastronomic locations across 10 cities in Greece! Saved as 'greece_all_restaurants_bars_cafes.csv'


### Public Transport Stops

In [ ]:
import requests
import pandas as pd

OVERPASS_URL = "https://overpass-api.de/api/interpreter"

CITIES_GREECE = [
    {"name": "Athens", "lat": 37.9838, "lon": 23.7275, "radius": 10000},
    {"name": "Thessaloniki", "lat": 40.6401, "lon": 22.9444, "radius": 8000},
    {"name": "Santorini", "lat": 36.3932, "lon": 25.4615, "radius": 7000},
    {"name": "Mykonos", "lat": 37.4467, "lon": 25.3289, "radius": 7000},
    {"name": "Rhodes", "lat": 36.4344, "lon": 28.2170, "radius": 7000},
    {"name": "Corfu", "lat": 39.6249, "lon": 19.9224, "radius": 7000},
    {"name": "Heraklion", "lat": 35.3403, "lon": 25.1333, "radius": 7000},
    {"name": "Chania", "lat": 35.5112, "lon": 24.0292, "radius": 7000},
    {"name": "Nafplio", "lat": 37.5639, "lon": 22.8075, "radius": 7000},
    {"name": "Delphi", "lat": 38.4801, "lon": 22.5010, "radius": 7000}
]

def get_osm_transit_stops(city):
    query = f"""
    [out:json];
    (
      node["highway"="bus_stop"](around:{city["radius"]},{city["lat"]},{city["lon"]});
      node["railway"="station"](around:{city["radius"]},{city["lat"]},{city["lon"]});
      node["public_transport"="platform"](around:{city["radius"]},{city["lat"]},{city["lon"]});
    );
    out center;
    """
    
    response = requests.get(OVERPASS_URL, params={"data": query})
    return response.json().get("elements", []) if response.status_code == 200 else []

all_stops = []
for city in CITIES_GREECE:
    print(f"📍 Fetching public transport stops in {city['name']}...")
    stops = get_osm_transit_stops(city)

    for stop in stops:
        tags = stop.get("tags", {})

        all_stops.append({
            "city": city["name"],
            "name": tags.get("name", "Unknown"),
            "type": tags.get("highway", tags.get("railway", tags.get("public_transport", "Unknown"))),
            "latitude": stop.get("lat"),
            "longitude": stop.get("lon"),
            "operator": tags.get("operator", "Unknown")
        })

df_transit = pd.DataFrame(all_stops)
df_transit.to_csv("greece_transit_stops.csv", index=False)

print(f"✅ Collected {len(df_transit)} public transport stops across 10 cities in Greece!")


📍 Fetching public transport stops in Athens...
📍 Fetching public transport stops in Thessaloniki...
📍 Fetching public transport stops in Santorini...
📍 Fetching public transport stops in Mykonos...
📍 Fetching public transport stops in Rhodes...
📍 Fetching public transport stops in Corfu...
📍 Fetching public transport stops in Heraklion...
📍 Fetching public transport stops in Chania...
📍 Fetching public transport stops in Nafplio...
📍 Fetching public transport stops in Delphi...
✅ Collected 7694 public transport stops across 10 cities in Greece!


### Nature Spots

In [ ]:
import requests
import pandas as pd
import time

OVERPASS_URL = "https://overpass-api.de/api/interpreter"

CITIES_GREECE = [
    {"name": "Athens", "lat": 37.9838, "lng": 23.7275, "radius": 20000},  
    {"name": "Thessaloniki", "lat": 40.6401, "lng": 22.9444, "radius": 18000},  
    {"name": "Santorini", "lat": 36.3932, "lng": 25.4615, "radius": 15000},  
    {"name": "Mykonos", "lat": 37.4467, "lng": 25.3289, "radius": 15000},  
    {"name": "Rhodes", "lat": 36.4349, "lng": 28.2176, "radius": 12000},  
    {"name": "Corfu", "lat": 39.6249, "lng": 19.9223, "radius": 12000},  
    {"name": "Heraklion", "lat": 35.3387, "lng": 25.1442, "radius": 10000},  
    {"name": "Chania", "lat": 35.5112, "lng": 24.0297, "radius": 12000},  
    {"name": "Nafplio", "lat": 37.5671, "lng": 22.8047, "radius": 10000},  
    {"name": "Delphi", "lat": 38.4828, "lng": 22.5010, "radius": 15000}  
]

def get_osm_nature_spots(lat, lon):
    query = f"""
    [out:json];
    (
      node["leisure"="nature_reserve"](around:10000,{lat},{lon});
      node["leisure"="park"](around:10000,{lat},{lon});
      node["route"="hiking"](around:10000,{lat},{lon});
      node["piste:type"="downhill"](around:10000,{lat},{lon});
      node["natural"="peak"](around:10000,{lat},{lon});
    );
    out center;
    """
    
    response = requests.get(OVERPASS_URL, params={"data": query})
    return response.json().get("elements", []) if response.status_code == 200 else []

all_nature_spots = []
for city in CITIES_GREECE:
    print(f"📍 Fetching nature spots in {city['name']}...")
    nature_spots = get_osm_nature_spots(city["lat"], city["lng"])

    for spot in nature_spots:
        tags = spot.get("tags", {})

        all_nature_spots.append({
            "city": city["name"],
            "name": tags.get("name", "Unknown"),
            "type": tags.get("leisure", tags.get("route", tags.get("natural", "Unknown"))),
            "latitude": spot.get("lat"),
            "longitude": spot.get("lon"),
            "elevation": tags.get("ele", "Unknown")
        })
    time.sleep(2)

df_nature = pd.DataFrame(all_nature_spots)
df_nature.to_csv("greece_nature_spots.csv", index=False)

print(f"✅ Collected {len(df_nature)} nature spots across 10 cities in Greece!")

📍 Fetching nature spots in Athens...
📍 Fetching nature spots in Thessaloniki...
📍 Fetching nature spots in Santorini...
📍 Fetching nature spots in Mykonos...
📍 Fetching nature spots in Rhodes...
📍 Fetching nature spots in Corfu...
📍 Fetching nature spots in Heraklion...
📍 Fetching nature spots in Chania...
📍 Fetching nature spots in Nafplio...
📍 Fetching nature spots in Delphi...
✅ Collected 218 nature spots across 10 cities in Greece!


### Weather

In [66]:
import os
import requests
import pandas as pd
import time

# Enter your OpenWeather API key
OPENWEATHER_KEY = input("Enter your OpenWeather API key: ")
WEATHER_URL = "https://api.openweathermap.org/data/2.5/weather"

CITIES_GREECE = [
    {"name": "Athens", "lat": 37.9838, "lng": 23.7275},  
    {"name": "Thessaloniki", "lat": 40.6401, "lng": 22.9444},  
    {"name": "Santorini", "lat": 36.3932, "lng": 25.4615},  
    {"name": "Mykonos", "lat": 37.4467, "lng": 25.3289},  
    {"name": "Rhodes", "lat": 36.4349, "lng": 28.2176},  
    {"name": "Corfu", "lat": 39.6249, "lng": 19.9223},  
    {"name": "Heraklion", "lat": 35.3387, "lng": 25.1442},  
    {"name": "Chania", "lat": 35.5112, "lng": 24.0297},  
    {"name": "Nafplio", "lat": 37.5671, "lng": 22.8047},  
    {"name": "Delphi", "lat": 38.4828, "lng": 22.5010}  
]

# Function to fetch weather data based on latitude and longitude
def get_weather(lat, lon):
    params = {
        "lat": lat,
        "lon": lon,
        "appid": OPENWEATHER_KEY,
        "units": "metric"  # Celsius
    }
    response = requests.get(WEATHER_URL, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"❌ Error fetching weather for {lat}, {lon}: {response.status_code}")
        return None

# Collect weather data for Greek cities
all_weather_data = []

for city in CITIES_GREECE:
    print(f"Fetching weather data for {city['name']}...")
    weather_data = get_weather(city["lat"], city["lng"])
    
    if weather_data:
        weather_info = {
            "city": city["name"],
            "latitude": city["lat"],
            "longitude": city["lng"],
            "temperature": weather_data["main"]["temp"],
            "weather": weather_data["weather"][0]["description"],
            "humidity": weather_data["main"]["humidity"],
            "wind_speed": weather_data["wind"]["speed"]
        }
        all_weather_data.append(weather_info)
    time.sleep(1)  # Avoid exceeding API limits

# Create DataFrame and save weather data
df_weather = pd.DataFrame(all_weather_data)
df_weather.to_csv("greece_weather_data.csv", index=False)

print(f"✅ Weather data for {len(df_weather)} Greek cities collected and saved!")


Fetching weather data for Athens...
Fetching weather data for Thessaloniki...
Fetching weather data for Santorini...
Fetching weather data for Mykonos...
Fetching weather data for Rhodes...
Fetching weather data for Corfu...
Fetching weather data for Heraklion...
Fetching weather data for Chania...
Fetching weather data for Nafplio...
Fetching weather data for Delphi...
✅ Weather data for 10 Greek cities collected and saved!


## Italy

### Tourist Attractions

In [27]:
import requests
import pandas as pd
import time

# 🔑 Insert your Google API Key here
API_KEY = input("Enter your Google API key: ")
PLACES_URL = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# 📍 Top 10 Most Visited Cities in Italy with Coordinates & Search Radius (in meters)
CITIES_ITALY = [
    {"name": "Rome", "lat": 41.9028, "lng": 12.4964, "radius": 20000},  
    {"name": "Milan", "lat": 45.4642, "lng": 9.1900, "radius": 18000},  
    {"name": "Florence", "lat": 43.7696, "lng": 11.2558, "radius": 15000},  
    {"name": "Venice", "lat": 45.4408, "lng": 12.3155, "radius": 15000},  
    {"name": "Naples", "lat": 40.8518, "lng": 14.2681, "radius": 12000},  
    {"name": "Verona", "lat": 45.4384, "lng": 10.9916, "radius": 12000},  
    {"name": "Bologna", "lat": 44.4949, "lng": 11.3426, "radius": 10000},  
    {"name": "Turin", "lat": 45.0703, "lng": 7.6869, "radius": 12000},  
    {"name": "Pisa", "lat": 43.7162, "lng": 10.3966, "radius": 10000},  
    {"name": "Siena", "lat": 43.3188, "lng": 11.3308, "radius": 15000}  
]

# Function to fetch tourist attractions for a given city
def get_places(city_name, lat, lng, radius, type_place="tourist_attraction"):
    params = {
        "location": f"{lat},{lng}",
        "radius": radius,
        "type": type_place,
        "key": API_KEY
    }
    
    places = []
    next_page_token = None

    print(f"📍 Fetching attractions in {city_name}...")
    
    while True:
        if next_page_token:
            params["pagetoken"] = next_page_token
        
        response = requests.get(PLACES_URL, params=params)
        data = response.json()
        
        if response.status_code != 200 or "results" not in data:
            print(f"⚠️ Error fetching data for {city_name}: {data.get('status', 'Unknown error')}")
            break
        
        places.extend(data["results"])
        
        next_page_token = data.get("next_page_token")
        if not next_page_token:
            break
        
        time.sleep(2)  # Google requires a small delay before using next_page_token

    return places

# Collect data for all cities
all_places = []
seen_places = set()

for city in CITIES_ITALY:
    places = get_places(city["name"], city["lat"], city["lng"], city["radius"])
    
    for p in places:
        place_id = p["place_id"]
        
        if place_id not in seen_places:
            seen_places.add(place_id)
            all_places.append({
                "city": city["name"],
                "place_id": place_id,
                "name": p.get("name", "Unknown"),
                "rating": p.get("rating", "N/A"),
                "address": p.get("vicinity", "Unknown"),
                "latitude": p["geometry"]["location"]["lat"],
                "longitude": p["geometry"]["location"]["lng"],
                "types": ", ".join(p.get("types", []))
            })

    time.sleep(5)  # Avoid excessive API requests

# Save data to CSV
df_places = pd.DataFrame(all_places)
df_places.to_csv("italy_tourist_attractions.csv", index=False)

print(f"✅ Collected {len(df_places)} tourist attractions across 10 cities in Italy!")


📍 Fetching attractions in Rome...
📍 Fetching attractions in Milan...
📍 Fetching attractions in Florence...
📍 Fetching attractions in Venice...
📍 Fetching attractions in Naples...
📍 Fetching attractions in Verona...
📍 Fetching attractions in Bologna...
📍 Fetching attractions in Turin...
📍 Fetching attractions in Pisa...
📍 Fetching attractions in Siena...
✅ Collected 600 tourist attractions across 10 cities in Italy!


### Events

In [10]:
import requests
import pandas as pd
import time

# 🔑 Solicita a chave da API do usuário
API_KEY = input("Enter your Google API key: ")
PLACES_URL = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# 📍 Top 10 Most Visited Cities in Italy with Coordinates & Search Radius (in meters)
CITIES_ITALY = [
    {"name": "Rome", "lat": 41.9028, "lng": 12.4964, "radius": 15000},
    {"name": "Milan", "lat": 45.4642, "lng": 9.1900, "radius": 12000},
    {"name": "Florence", "lat": 43.7696, "lng": 11.2558, "radius": 10000},
    {"name": "Venice", "lat": 45.4408, "lng": 12.3155, "radius": 10000},
    {"name": "Naples", "lat": 40.8518, "lng": 14.2681, "radius": 12000},
    {"name": "Verona", "lat": 45.4384, "lng": 10.9916, "radius": 10000},
    {"name": "Bologna", "lat": 44.4949, "lng": 11.3426, "radius": 10000},
    {"name": "Turin", "lat": 45.0703, "lng": 7.6869, "radius": 10000},
    {"name": "Pisa", "lat": 43.7159, "lng": 10.4017, "radius": 10000},
    {"name": "Siena", "lat": 43.3182, "lng": 11.3306, "radius": 10000}
]

# 🎭 Define categories of places where events may occur
EVENT_TYPES = [
    "stadium", "casino", "night_club", "art_gallery", "museum",
    "tourist_attraction", "amusement_park", "university", "movie_theater"
]

# Function to fetch event venues for a city
def get_places(city_name, lat, lng, radius):
    places = []
    seen_places = set()

    for place_type in EVENT_TYPES:
        params = {
            "location": f"{lat},{lng}",
            "radius": radius,
            "type": place_type,
            "key": API_KEY
        }
        
        print(f"📍 Fetching event venues in {city_name} ({place_type})...")
        
        response = requests.get(PLACES_URL, params=params)
        data = response.json()

        if response.status_code == 200 and "results" in data:
            for p in data["results"]:
                place_id = p["place_id"]

                if place_id not in seen_places:
                    seen_places.add(place_id)
                    places.append({
                        "city": city_name,
                        "place_id": place_id,
                        "name": p.get("name", "Unknown"),
                        "rating": p.get("rating", "N/A"),
                        "address": p.get("vicinity", "Unknown"),
                        "latitude": p["geometry"]["location"]["lat"],
                        "longitude": p["geometry"]["location"]["lng"],
                        "types": ", ".join(p.get("types", []))
                    })
        
        # Wait a bit to avoid exceeding API request limits
        time.sleep(2)
    
    return places

# 🔍 Collect event venue data for all cities
all_places = []
for city in CITIES_ITALY:
    all_places.extend(get_places(city["name"], city["lat"], city["lng"], city["radius"]))

# 📝 Save data to CSV
df_places = pd.DataFrame(all_places)
df_places.to_csv("italy_event_locations.csv", index=False)

print(f"✅ Collected {len(df_places)} event venues across 10 cities in Italy! Saved as 'italy_event_locations.csv'")


📍 Fetching event venues in Rome (stadium)...
📍 Fetching event venues in Rome (casino)...
📍 Fetching event venues in Rome (night_club)...
📍 Fetching event venues in Rome (art_gallery)...
📍 Fetching event venues in Rome (museum)...
📍 Fetching event venues in Rome (tourist_attraction)...
📍 Fetching event venues in Rome (amusement_park)...
📍 Fetching event venues in Rome (university)...
📍 Fetching event venues in Rome (movie_theater)...
📍 Fetching event venues in Milan (stadium)...
📍 Fetching event venues in Milan (casino)...
📍 Fetching event venues in Milan (night_club)...
📍 Fetching event venues in Milan (art_gallery)...
📍 Fetching event venues in Milan (museum)...
📍 Fetching event venues in Milan (tourist_attraction)...
📍 Fetching event venues in Milan (amusement_park)...
📍 Fetching event venues in Milan (university)...
📍 Fetching event venues in Milan (movie_theater)...
📍 Fetching event venues in Florence (stadium)...
📍 Fetching event venues in Florence (casino)...
📍 Fetching event ven

### Hotels

In [44]:
import requests 
import pandas as pd
import time

OVERPASS_URL = "https://overpass-api.de/api/interpreter"

CITIES_ITALY = [
    {"name": "Rome", "lat": 41.9028, "lng": 12.4964, "radius": 20000},  
    {"name": "Milan", "lat": 45.4642, "lng": 9.1900, "radius": 18000},  
    {"name": "Florence", "lat": 43.7696, "lng": 11.2558, "radius": 15000},  
    {"name": "Venice", "lat": 45.4408, "lng": 12.3155, "radius": 15000},  
    {"name": "Naples", "lat": 40.8518, "lng": 14.2681, "radius": 12000},  
    {"name": "Verona", "lat": 45.4384, "lng": 10.9916, "radius": 12000},  
    {"name": "Bologna", "lat": 44.4949, "lng": 11.3426, "radius": 10000},  
    {"name": "Turin", "lat": 45.0703, "lng": 7.6869, "radius": 12000},  
    {"name": "Pisa", "lat": 43.7162, "lng": 10.3966, "radius": 10000},  
    {"name": "Siena", "lat": 43.3188, "lng": 11.3308, "radius": 15000}  
]

def get_osm_hotels(lat, lon):
    query = f"""
    [out:json];
    (
      node["tourism"="hotel"](around:10000,{lat},{lon});
      node["tourism"="hostel"](around:10000,{lat},{lon});
      node["tourism"="guest_house"](around:10000,{lat},{lon});
    );
    out center;
    """
    
    response = requests.get(OVERPASS_URL, params={"data": query})
    return response.json().get("elements", []) if response.status_code == 200 else []

all_hotels = []
for city in CITIES_ITALY:
    print(f"📍 Fetching hotels and hostels in {city['name']}...")
    hotels = get_osm_hotels(city["lat"], city["lng"])

    for hotel in hotels:
        tags = hotel.get("tags", {})

        all_hotels.append({
            "city": city["name"],
            "name": tags.get("name", "Unknown"),
            "type": tags.get("tourism", "Unknown"),
            "latitude": hotel.get("lat"),
            "longitude": hotel.get("lon"),
            "stars": tags.get("stars", "Unknown"),
            "operator": tags.get("operator", "Unknown")
        })
    time.sleep(2)

df_hotels = pd.DataFrame(all_hotels)
df_hotels.to_csv("italy_hotels_hostels.csv", index=False)

print(f"✅ Collected {len(df_hotels)} hotels and hostels across 10 cities in Italy!")


📍 Fetching hotels and hostels in Rome...
📍 Fetching hotels and hostels in Milan...
📍 Fetching hotels and hostels in Florence...
📍 Fetching hotels and hostels in Venice...
📍 Fetching hotels and hostels in Naples...
📍 Fetching hotels and hostels in Verona...
📍 Fetching hotels and hostels in Bologna...
📍 Fetching hotels and hostels in Turin...
📍 Fetching hotels and hostels in Pisa...
📍 Fetching hotels and hostels in Siena...
✅ Collected 3180 hotels and hostels across 10 cities in Italy!


### Restaurants

In [17]:
import requests
import pandas as pd
import time

OVERPASS_URL = "https://overpass-api.de/api/interpreter"

# 📍 Top 10 Most Visited Cities in Italy
CITIES_ITALY = [
    {"name": "Rome", "lat": 41.9028, "lon": 12.4964, "radius": 10000},
    {"name": "Milan", "lat": 45.4642, "lon": 9.1900, "radius": 10000},
    {"name": "Florence", "lat": 43.7696, "lon": 11.2558, "radius": 8000},
    {"name": "Venice", "lat": 45.4408, "lon": 12.3155, "radius": 7000},
    {"name": "Naples", "lat": 40.8518, "lon": 14.2681, "radius": 10000},
    {"name": "Verona", "lat": 45.4384, "lon": 10.9916, "radius": 7000},
    {"name": "Bologna", "lat": 44.4949, "lon": 11.3426, "radius": 8000},
    {"name": "Turin", "lat": 45.0703, "lon": 7.6869, "radius": 8000},
    {"name": "Pisa", "lat": 43.7167, "lon": 10.4000, "radius": 7000},
    {"name": "Siena", "lat": 43.3188, "lon": 11.3306, "radius": 7000}
]

def get_osm_restaurants(city):
    """Fetches restaurants, cafés, and bars in a specific Italian city with retries."""
    query = f"""
    [out:json];
    (
      node["amenity"="restaurant"](around:{city["radius"]},{city["lat"]},{city["lon"]});
      node["amenity"="cafe"](around:{city["radius"]},{city["lat"]},{city["lon"]});
      node["amenity"="bar"](around:{city["radius"]},{city["lat"]},{city["lon"]});
    );
    out center;
    """

    retries = 3
    while retries > 0:
        response = requests.get(OVERPASS_URL, params={"data": query})
        
        if response.status_code == 200:
            return response.json().get("elements", [])
        else:
            print(f"⚠️ Error fetching restaurants in {city['name']} ({response.status_code}), retrying... ({3 - retries + 1}/3)")
            retries -= 1
            time.sleep(3)  # Aguardar antes de tentar novamente
    
    print(f"❌ Failed to fetch data for {city['name']} after multiple retries.")
    return []

# 🔍 Collect data for all cities
all_places = []
for city in CITIES_ITALY:
    print(f"📍 Fetching gastronomic locations in {city['name']}...")
    places = get_osm_restaurants(city)

    for place in places:
        tags = place.get("tags", {})

        all_places.append({
            "city": city["name"],
            "name": tags.get("name", "Unknown"),
            "type": tags.get("amenity", "Unknown"),
            "latitude": place.get("lat"),
            "longitude": place.get("lon"),
            "street": tags.get("addr:street", "Unknown"),
            "postcode": tags.get("addr:postcode", "Unknown"),
            "cuisine": tags.get("cuisine", "Not specified")  # Tipo de comida
        })

    time.sleep(2)  # Evitar sobrecarga na API

# 📝 Save data to CSV
df = pd.DataFrame(all_places)
df.to_csv("italy_all_restaurants_bars_cafes.csv", index=False)

print(f"✅ Collected {len(df)} gastronomic locations across 10 cities in Italy! Saved as 'italy_all_restaurants_bars_cafes.csv'")


📍 Fetching gastronomic locations in Rome...
📍 Fetching gastronomic locations in Milan...
📍 Fetching gastronomic locations in Florence...
📍 Fetching gastronomic locations in Venice...
📍 Fetching gastronomic locations in Naples...
📍 Fetching gastronomic locations in Verona...
📍 Fetching gastronomic locations in Bologna...
📍 Fetching gastronomic locations in Turin...
⚠️ Error fetching restaurants in Turin (504), retrying... (1/3)
📍 Fetching gastronomic locations in Pisa...
📍 Fetching gastronomic locations in Siena...
✅ Collected 19103 gastronomic locations across 10 cities in Italy! Saved as 'italy_all_restaurants_bars_cafes.csv'


### Public Transport Stops

In [31]:
import requests
import pandas as pd

OVERPASS_URL = "https://overpass-api.de/api/interpreter"

CITIES_ITALY = [
    {"name": "Rome", "lat": 41.9028, "lon": 12.4964, "radius": 10000},
    {"name": "Milan", "lat": 45.4642, "lon": 9.1900, "radius": 8000},
    {"name": "Florence", "lat": 43.7696, "lon": 11.2558, "radius": 8000},
    {"name": "Venice", "lat": 45.4408, "lon": 12.3155, "radius": 7000},
    {"name": "Naples", "lat": 40.8518, "lon": 14.2681, "radius": 7000},
    {"name": "Verona", "lat": 45.4384, "lon": 10.9916, "radius": 7000},
    {"name": "Bologna", "lat": 44.4949, "lon": 11.3426, "radius": 7000},
    {"name": "Turin", "lat": 45.0703, "lon": 7.6869, "radius": 7000},
    {"name": "Pisa", "lat": 43.7164, "lon": 10.3966, "radius": 7000},
    {"name": "Siena", "lat": 43.3188, "lon": 11.3308, "radius": 7000}
]

def get_osm_transit_stops(city):
    query = f"""
    [out:json];
    (
      node["highway"="bus_stop"](around:{city["radius"]},{city["lat"]},{city["lon"]});
      node["railway"="station"](around:{city["radius"]},{city["lat"]},{city["lon"]});
      node["public_transport"="platform"](around:{city["radius"]},{city["lat"]},{city["lon"]});
    );
    out center;
    """
    
    response = requests.get(OVERPASS_URL, params={"data": query})
    return response.json().get("elements", []) if response.status_code == 200 else []

all_stops = []
for city in CITIES_ITALY:
    print(f"📍 Fetching public transport stops in {city['name']}...")
    stops = get_osm_transit_stops(city)

    for stop in stops:
        tags = stop.get("tags", {})

        all_stops.append({
            "city": city["name"],
            "name": tags.get("name", "Unknown"),
            "type": tags.get("highway", tags.get("railway", tags.get("public_transport", "Unknown"))),
            "latitude": stop.get("lat"),
            "longitude": stop.get("lon"),
            "operator": tags.get("operator", "Unknown")
        })

df_transit = pd.DataFrame(all_stops)
df_transit.to_csv("italy_transit_stops.csv", index=False)

print(f"✅ Collected {len(df_transit)} public transport stops across 10 cities in Italy!")


📍 Fetching public transport stops in Rome...
📍 Fetching public transport stops in Milan...
📍 Fetching public transport stops in Florence...
📍 Fetching public transport stops in Venice...
📍 Fetching public transport stops in Naples...
📍 Fetching public transport stops in Verona...
📍 Fetching public transport stops in Bologna...
📍 Fetching public transport stops in Turin...
📍 Fetching public transport stops in Pisa...
📍 Fetching public transport stops in Siena...
✅ Collected 17422 public transport stops across 10 cities in Italy!


### Nature Spots

In [54]:
import requests
import pandas as pd
import time

OVERPASS_URL = "https://overpass-api.de/api/interpreter"

CITIES_ITALY = [
    {"name": "Rome", "lat": 41.9028, "lng": 12.4964, "radius": 20000},  
    {"name": "Milan", "lat": 45.4642, "lng": 9.1900, "radius": 18000},  
    {"name": "Florence", "lat": 43.7696, "lng": 11.2558, "radius": 15000},  
    {"name": "Venice", "lat": 45.4408, "lng": 12.3155, "radius": 15000},  
    {"name": "Naples", "lat": 40.8518, "lng": 14.2681, "radius": 12000},  
    {"name": "Verona", "lat": 45.4384, "lng": 10.9916, "radius": 12000},  
    {"name": "Bologna", "lat": 44.4949, "lng": 11.3426, "radius": 10000},  
    {"name": "Turin", "lat": 45.0703, "lng": 7.6869, "radius": 12000},  
    {"name": "Pisa", "lat": 43.7162, "lng": 10.3966, "radius": 10000},  
    {"name": "Siena", "lat": 43.3188, "lng": 11.3308, "radius": 15000}  
]

def get_osm_nature_spots(lat, lon):
    query = f"""
    [out:json];
    (
      node["leisure"="nature_reserve"](around:10000,{lat},{lon});
      node["leisure"="park"](around:10000,{lat},{lon});
      node["route"="hiking"](around:10000,{lat},{lon});
      node["piste:type"="downhill"](around:10000,{lat},{lon});
      node["natural"="peak"](around:10000,{lat},{lon});
    );
    out center;
    """
    
    response = requests.get(OVERPASS_URL, params={"data": query})
    return response.json().get("elements", []) if response.status_code == 200 else []

all_nature_spots = []
for city in CITIES_ITALY:
    print(f"📍 Fetching nature spots in {city['name']}...")
    nature_spots = get_osm_nature_spots(city["lat"], city["lng"])

    for spot in nature_spots:
        tags = spot.get("tags", {})

        all_nature_spots.append({
            "city": city["name"],
            "name": tags.get("name", "Unknown"),
            "type": tags.get("leisure", tags.get("route", tags.get("natural", "Unknown"))),
            "latitude": spot.get("lat"),
            "longitude": spot.get("lon"),
            "elevation": tags.get("ele", "Unknown")
        })
    time.sleep(2)

df_nature = pd.DataFrame(all_nature_spots)
df_nature.to_csv("italy_nature_spots.csv", index=False)

print(f"✅ Collected {len(df_nature)} nature spots across 10 cities in Italy!")

📍 Fetching nature spots in Rome...
📍 Fetching nature spots in Milan...
📍 Fetching nature spots in Florence...
📍 Fetching nature spots in Venice...
📍 Fetching nature spots in Naples...
📍 Fetching nature spots in Verona...
📍 Fetching nature spots in Bologna...
📍 Fetching nature spots in Turin...
📍 Fetching nature spots in Pisa...
📍 Fetching nature spots in Siena...
✅ Collected 228 nature spots across 10 cities in Italy!


### Weather

In [67]:
import os
import requests
import pandas as pd
import time

# Enter your OpenWeather API key
OPENWEATHER_KEY = input("Enter your OpenWeather API key: ")
WEATHER_URL = "https://api.openweathermap.org/data/2.5/weather"

CITIES_ITALY = [
    {"name": "Rome", "lat": 41.9028, "lng": 12.4964},  
    {"name": "Milan", "lat": 45.4642, "lng": 9.1900},  
    {"name": "Florence", "lat": 43.7696, "lng": 11.2558},  
    {"name": "Venice", "lat": 45.4408, "lng": 12.3155},  
    {"name": "Naples", "lat": 40.8518, "lng": 14.2681},  
    {"name": "Verona", "lat": 45.4384, "lng": 10.9916},  
    {"name": "Bologna", "lat": 44.4949, "lng": 11.3426},  
    {"name": "Turin", "lat": 45.0703, "lng": 7.6869},  
    {"name": "Pisa", "lat": 43.7162, "lng": 10.3966},  
    {"name": "Siena", "lat": 43.3188, "lng": 11.3308}  
]

# Function to fetch weather data based on latitude and longitude
def get_weather(lat, lon):
    params = {
        "lat": lat,
        "lon": lon,
        "appid": OPENWEATHER_KEY,
        "units": "metric"  # Celsius
    }
    response = requests.get(WEATHER_URL, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"❌ Error fetching weather for {lat}, {lon}: {response.status_code}")
        return None

# Collect weather data for Italian cities
all_weather_data = []

for city in CITIES_ITALY:
    print(f"Fetching weather data for {city['name']}...")
    weather_data = get_weather(city["lat"], city["lng"])
    
    if weather_data:
        weather_info = {
            "city": city["name"],
            "latitude": city["lat"],
            "longitude": city["lng"],
            "temperature": weather_data["main"]["temp"],
            "weather": weather_data["weather"][0]["description"],
            "humidity": weather_data["main"]["humidity"],
            "wind_speed": weather_data["wind"]["speed"]
        }
        all_weather_data.append(weather_info)
    time.sleep(1)  # Avoid exceeding API limits

# Create DataFrame and save weather data
df_weather = pd.DataFrame(all_weather_data)
df_weather.to_csv("italy_weather_data.csv", index=False)

print(f"✅ Weather data for {len(df_weather)} Italian cities collected and saved!")


Fetching weather data for Rome...
Fetching weather data for Milan...
Fetching weather data for Florence...
Fetching weather data for Venice...
Fetching weather data for Naples...
Fetching weather data for Verona...
Fetching weather data for Bologna...
Fetching weather data for Turin...
Fetching weather data for Pisa...
Fetching weather data for Siena...
✅ Weather data for 10 Italian cities collected and saved!


## Spain

### Tourist Attractions

In [28]:
import requests
import pandas as pd
import time

# 🔑 Insert your Google API Key here
API_KEY = input("Enter your Google API key: ")
PLACES_URL = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# 📍 Top 10 Most Visited Cities in Spain with Coordinates & Search Radius (in meters)
CITIES_SPAIN = [
    {"name": "Madrid", "lat": 40.4168, "lng": -3.7038, "radius": 20000},  
    {"name": "Barcelona", "lat": 41.3851, "lng": 2.1734, "radius": 18000},  
    {"name": "Seville", "lat": 37.3886, "lng": -5.9823, "radius": 15000},  
    {"name": "Valencia", "lat": 39.4699, "lng": -0.3763, "radius": 15000},  
    {"name": "Granada", "lat": 37.1773, "lng": -3.5986, "radius": 12000},  
    {"name": "Bilbao", "lat": 43.2630, "lng": -2.9340, "radius": 12000},  
    {"name": "Málaga", "lat": 36.7213, "lng": -4.4210, "radius": 10000},  
    {"name": "Santiago de Compostela", "lat": 42.8782, "lng": -8.5448, "radius": 12000},  
    {"name": "San Sebastián", "lat": 43.3183, "lng": -1.9812, "radius": 10000},  
    {"name": "Toledo", "lat": 39.8628, "lng": -4.0273, "radius": 15000}  
]

# Function to fetch tourist attractions for a given city
def get_places(city_name, lat, lng, radius, type_place="tourist_attraction"):
    params = {
        "location": f"{lat},{lng}",
        "radius": radius,
        "type": type_place,
        "key": API_KEY
    }
    
    places = []
    next_page_token = None

    print(f"📍 Fetching attractions in {city_name}...")
    
    while True:
        if next_page_token:
            params["pagetoken"] = next_page_token
        
        response = requests.get(PLACES_URL, params=params)
        data = response.json()
        
        if response.status_code != 200 or "results" not in data:
            print(f"⚠️ Error fetching data for {city_name}: {data.get('status', 'Unknown error')}")
            break
        
        places.extend(data["results"])
        
        next_page_token = data.get("next_page_token")
        if not next_page_token:
            break
        
        time.sleep(2)  # Google requires a small delay before using next_page_token

    return places

# Collect data for all cities
all_places = []
seen_places = set()

for city in CITIES_SPAIN:
    places = get_places(city["name"], city["lat"], city["lng"], city["radius"])
    
    for p in places:
        place_id = p["place_id"]
        
        if place_id not in seen_places:
            seen_places.add(place_id)
            all_places.append({
                "city": city["name"],
                "place_id": place_id,
                "name": p.get("name", "Unknown"),
                "rating": p.get("rating", "N/A"),
                "address": p.get("vicinity", "Unknown"),
                "latitude": p["geometry"]["location"]["lat"],
                "longitude": p["geometry"]["location"]["lng"],
                "types": ", ".join(p.get("types", []))
            })

    time.sleep(5)  # Avoid excessive API requests

# Save data to CSV
df_places = pd.DataFrame(all_places)
df_places.to_csv("spain_tourist_attractions.csv", index=False)

print(f"✅ Collected {len(df_places)} tourist attractions across 10 cities in Spain!")


📍 Fetching attractions in Madrid...
📍 Fetching attractions in Barcelona...
📍 Fetching attractions in Seville...
📍 Fetching attractions in Valencia...
📍 Fetching attractions in Granada...
📍 Fetching attractions in Bilbao...
📍 Fetching attractions in Málaga...
📍 Fetching attractions in Santiago de Compostela...
📍 Fetching attractions in San Sebastián...
📍 Fetching attractions in Toledo...
✅ Collected 600 tourist attractions across 10 cities in Spain!


### Events

In [11]:
import requests
import pandas as pd
import time

# 🔑 Solicita a chave da API do usuário
API_KEY = input("Enter your Google API key: ")
PLACES_URL = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# 📍 Top 10 Most Visited Cities in Spain with Coordinates & Search Radius (in meters)
CITIES_SPAIN = [
    {"name": "Madrid", "lat": 40.4168, "lng": -3.7038, "radius": 15000},
    {"name": "Barcelona", "lat": 41.3874, "lng": 2.1686, "radius": 12000},
    {"name": "Seville", "lat": 37.3886, "lng": -5.9823, "radius": 10000},
    {"name": "Valencia", "lat": 39.4699, "lng": -0.3763, "radius": 10000},
    {"name": "Granada", "lat": 37.1773, "lng": -3.5986, "radius": 10000},
    {"name": "Bilbao", "lat": 43.2630, "lng": -2.9350, "radius": 10000},
    {"name": "Málaga", "lat": 36.7213, "lng": -4.4216, "radius": 10000},
    {"name": "Santiago de Compostela", "lat": 42.8782, "lng": -8.5448, "radius": 10000},
    {"name": "San Sebastián", "lat": 43.3183, "lng": -1.9812, "radius": 10000},
    {"name": "Toledo", "lat": 39.8628, "lng": -4.0273, "radius": 10000}
]

# 🎭 Define categories of places where events may occur
EVENT_TYPES = [
    "stadium", "casino", "night_club", "art_gallery", "museum",
    "tourist_attraction", "amusement_park", "university", "movie_theater"
]

# Function to fetch event venues for a city
def get_places(city_name, lat, lng, radius):
    places = []
    seen_places = set()

    for place_type in EVENT_TYPES:
        params = {
            "location": f"{lat},{lng}",
            "radius": radius,
            "type": place_type,
            "key": API_KEY
        }
        
        print(f"📍 Fetching event venues in {city_name} ({place_type})...")
        
        response = requests.get(PLACES_URL, params=params)
        data = response.json()

        if response.status_code == 200 and "results" in data:
            for p in data["results"]:
                place_id = p["place_id"]

                if place_id not in seen_places:
                    seen_places.add(place_id)
                    places.append({
                        "city": city_name,
                        "place_id": place_id,
                        "name": p.get("name", "Unknown"),
                        "rating": p.get("rating", "N/A"),
                        "address": p.get("vicinity", "Unknown"),
                        "latitude": p["geometry"]["location"]["lat"],
                        "longitude": p["geometry"]["location"]["lng"],
                        "types": ", ".join(p.get("types", []))
                    })
        
        # Wait a bit to avoid exceeding API request limits
        time.sleep(2)
    
    return places

# 🔍 Collect event venue data for all cities
all_places = []
for city in CITIES_SPAIN:
    all_places.extend(get_places(city["name"], city["lat"], city["lng"], city["radius"]))

# 📝 Save data to CSV
df_places = pd.DataFrame(all_places)
df_places.to_csv("spain_event_locations.csv", index=False)

print(f"✅ Collected {len(df_places)} event venues across 10 cities in Spain! Saved as 'spain_event_locations.csv'")


📍 Fetching event venues in Madrid (stadium)...
📍 Fetching event venues in Madrid (casino)...
📍 Fetching event venues in Madrid (night_club)...
📍 Fetching event venues in Madrid (art_gallery)...
📍 Fetching event venues in Madrid (museum)...
📍 Fetching event venues in Madrid (tourist_attraction)...
📍 Fetching event venues in Madrid (amusement_park)...
📍 Fetching event venues in Madrid (university)...
📍 Fetching event venues in Madrid (movie_theater)...
📍 Fetching event venues in Barcelona (stadium)...
📍 Fetching event venues in Barcelona (casino)...
📍 Fetching event venues in Barcelona (night_club)...
📍 Fetching event venues in Barcelona (art_gallery)...
📍 Fetching event venues in Barcelona (museum)...
📍 Fetching event venues in Barcelona (tourist_attraction)...
📍 Fetching event venues in Barcelona (amusement_park)...
📍 Fetching event venues in Barcelona (university)...
📍 Fetching event venues in Barcelona (movie_theater)...
📍 Fetching event venues in Seville (stadium)...
📍 Fetching even

### Hotels

In [45]:
import requests 
import pandas as pd
import time

OVERPASS_URL = "https://overpass-api.de/api/interpreter"

CITIES_SPAIN = [
    {"name": "Madrid", "lat": 40.4168, "lng": -3.7038, "radius": 20000},  
    {"name": "Barcelona", "lat": 41.3851, "lng": 2.1734, "radius": 18000},  
    {"name": "Seville", "lat": 37.3886, "lng": -5.9823, "radius": 15000},  
    {"name": "Valencia", "lat": 39.4699, "lng": -0.3763, "radius": 15000},  
    {"name": "Granada", "lat": 37.1773, "lng": -3.5986, "radius": 12000},  
    {"name": "Bilbao", "lat": 43.2630, "lng": -2.9340, "radius": 12000},  
    {"name": "Málaga", "lat": 36.7213, "lng": -4.4210, "radius": 10000},  
    {"name": "Santiago de Compostela", "lat": 42.8782, "lng": -8.5448, "radius": 12000},  
    {"name": "San Sebastián", "lat": 43.3183, "lng": -1.9812, "radius": 10000},  
    {"name": "Toledo", "lat": 39.8628, "lng": -4.0273, "radius": 15000}  
]

def get_osm_hotels(lat, lon):
    query = f"""
    [out:json];
    (
      node["tourism"="hotel"](around:10000,{lat},{lon});
      node["tourism"="hostel"](around:10000,{lat},{lon});
      node["tourism"="guest_house"](around:10000,{lat},{lon});
    );
    out center;
    """
    
    response = requests.get(OVERPASS_URL, params={"data": query})
    return response.json().get("elements", []) if response.status_code == 200 else []

all_hotels = []
for city in CITIES_SPAIN:
    print(f"📍 Fetching hotels and hostels in {city['name']}...")
    hotels = get_osm_hotels(city["lat"], city["lng"])

    for hotel in hotels:
        tags = hotel.get("tags", {})

        all_hotels.append({
            "city": city["name"],
            "name": tags.get("name", "Unknown"),
            "type": tags.get("tourism", "Unknown"),
            "latitude": hotel.get("lat"),
            "longitude": hotel.get("lon"),
            "stars": tags.get("stars", "Unknown"),
            "operator": tags.get("operator", "Unknown")
        })
    time.sleep(2)

df_hotels = pd.DataFrame(all_hotels)
df_hotels.to_csv("spain_hotels_hostels.csv", index=False)

print(f"✅ Collected {len(df_hotels)} hotels and hostels across 10 cities in Spain!")


📍 Fetching hotels and hostels in Madrid...
📍 Fetching hotels and hostels in Barcelona...
📍 Fetching hotels and hostels in Seville...
📍 Fetching hotels and hostels in Valencia...
📍 Fetching hotels and hostels in Granada...
📍 Fetching hotels and hostels in Bilbao...
📍 Fetching hotels and hostels in Málaga...
📍 Fetching hotels and hostels in Santiago de Compostela...
📍 Fetching hotels and hostels in San Sebastián...
📍 Fetching hotels and hostels in Toledo...
✅ Collected 2002 hotels and hostels across 10 cities in Spain!


### Restaurants

In [18]:
import requests
import pandas as pd
import time

OVERPASS_URL = "https://overpass-api.de/api/interpreter"

# 📍 Top 10 Most Visited Cities in Spain
CITIES_SPAIN = [
    {"name": "Madrid", "lat": 40.4168, "lon": -3.7038, "radius": 10000},
    {"name": "Barcelona", "lat": 41.3851, "lon": 2.1734, "radius": 10000},
    {"name": "Seville", "lat": 37.3891, "lon": -5.9845, "radius": 8000},
    {"name": "Valencia", "lat": 39.4699, "lon": -0.3763, "radius": 8000},
    {"name": "Granada", "lat": 37.1773, "lon": -3.5986, "radius": 7000},
    {"name": "Bilbao", "lat": 43.263, "lon": -2.935, "radius": 7000},
    {"name": "Málaga", "lat": 36.7213, "lon": -4.4214, "radius": 7000},
    {"name": "Santiago de Compostela", "lat": 42.8782, "lon": -8.5448, "radius": 7000},
    {"name": "San Sebastián", "lat": 43.3183, "lon": -1.9812, "radius": 7000},
    {"name": "Toledo", "lat": 39.8628, "lon": -4.0273, "radius": 7000}
]

def get_osm_restaurants(city):
    """Fetches restaurants, cafés, and bars in a specific Spanish city with retries."""
    query = f"""
    [out:json];
    (
      node["amenity"="restaurant"](around:{city["radius"]},{city["lat"]},{city["lon"]});
      node["amenity"="cafe"](around:{city["radius"]},{city["lat"]},{city["lon"]});
      node["amenity"="bar"](around:{city["radius"]},{city["lat"]},{city["lon"]});
    );
    out center;
    """

    retries = 3
    while retries > 0:
        response = requests.get(OVERPASS_URL, params={"data": query})
        
        if response.status_code == 200:
            return response.json().get("elements", [])
        else:
            print(f"⚠️ Error fetching restaurants in {city['name']} ({response.status_code}), retrying... ({3 - retries + 1}/3)")
            retries -= 1
            time.sleep(3)  # Aguardar antes de tentar novamente
    
    print(f"❌ Failed to fetch data for {city['name']} after multiple retries.")
    return []

# 🔍 Collect data for all cities
all_places = []
for city in CITIES_SPAIN:
    print(f"📍 Fetching gastronomic locations in {city['name']}...")
    places = get_osm_restaurants(city)

    for place in places:
        tags = place.get("tags", {})

        all_places.append({
            "city": city["name"],
            "name": tags.get("name", "Unknown"),
            "type": tags.get("amenity", "Unknown"),
            "latitude": place.get("lat"),
            "longitude": place.get("lon"),
            "street": tags.get("addr:street", "Unknown"),
            "postcode": tags.get("addr:postcode", "Unknown"),
            "cuisine": tags.get("cuisine", "Not specified")  # Tipo de comida
        })

    time.sleep(2)  # Evitar sobrecarga na API

# 📝 Save data to CSV
df = pd.DataFrame(all_places)
df.to_csv("spain_all_restaurants_bars_cafes.csv", index=False)

print(f"✅ Collected {len(df)} gastronomic locations across 10 cities in Spain! Saved as 'spain_all_restaurants_bars_cafes.csv'")


📍 Fetching gastronomic locations in Madrid...
📍 Fetching gastronomic locations in Barcelona...
📍 Fetching gastronomic locations in Seville...
📍 Fetching gastronomic locations in Valencia...
📍 Fetching gastronomic locations in Granada...
📍 Fetching gastronomic locations in Bilbao...
📍 Fetching gastronomic locations in Málaga...
📍 Fetching gastronomic locations in Santiago de Compostela...
📍 Fetching gastronomic locations in San Sebastián...
📍 Fetching gastronomic locations in Toledo...
✅ Collected 22393 gastronomic locations across 10 cities in Spain! Saved as 'spain_all_restaurants_bars_cafes.csv'


### Public Transport Stops

In [32]:
import requests
import pandas as pd

OVERPASS_URL = "https://overpass-api.de/api/interpreter"

CITIES_SPAIN = [
    {"name": "Madrid", "lat": 40.4168, "lon": -3.7038, "radius": 20000},  
    {"name": "Barcelona", "lat": 41.3851, "lon": 2.1734, "radius": 18000},  
    {"name": "Seville", "lat": 37.3886, "lon": -5.9823, "radius": 15000},  
    {"name": "Valencia", "lat": 39.4699, "lon": -0.3763, "radius": 15000},  
    {"name": "Granada", "lat": 37.1773, "lon": -3.5986, "radius": 12000},  
    {"name": "Bilbao", "lat": 43.2630, "lon": -2.9340, "radius": 12000},  
    {"name": "Málaga", "lat": 36.7213, "lon": -4.4210, "radius": 10000},  
    {"name": "Santiago de Compostela", "lat": 42.8782, "lon": -8.5448, "radius": 12000},  
    {"name": "San Sebastián", "lat": 43.3183, "lon": -1.9812, "radius": 10000},  
    {"name": "Toledo", "lat": 39.8628, "lon": -4.0273, "radius": 15000}  
]

def get_osm_transit_stops(city):
    query = f"""
    [out:json];
    (
      node["highway"="bus_stop"](around:{city["radius"]},{city["lat"]},{city["lon"]});
      node["railway"="station"](around:{city["radius"]},{city["lat"]},{city["lon"]});
      node["public_transport"="platform"](around:{city["radius"]},{city["lat"]},{city["lon"]});
    );
    out center;
    """
    
    response = requests.get(OVERPASS_URL, params={"data": query})
    return response.json().get("elements", []) if response.status_code == 200 else []

all_stops = []
for city in CITIES_SPAIN:
    print(f"📍 Fetching public transport stops in {city['name']}...")
    stops = get_osm_transit_stops(city)

    for stop in stops:
        tags = stop.get("tags", {})

        all_stops.append({
            "city": city["name"],
            "name": tags.get("name", "Unknown"),
            "type": tags.get("highway", tags.get("railway", tags.get("public_transport", "Unknown"))),
            "latitude": stop.get("lat"),
            "longitude": stop.get("lon"),
            "operator": tags.get("operator", "Unknown")
        })

df_transit = pd.DataFrame(all_stops)
df_transit.to_csv("spain_transit_stops.csv", index=False)

print(f"✅ Collected {len(df_transit)} public transport stops across 10 cities in Spain!")


📍 Fetching public transport stops in Madrid...
📍 Fetching public transport stops in Barcelona...
📍 Fetching public transport stops in Seville...
📍 Fetching public transport stops in Valencia...
📍 Fetching public transport stops in Granada...
📍 Fetching public transport stops in Bilbao...
📍 Fetching public transport stops in Málaga...
📍 Fetching public transport stops in Santiago de Compostela...
📍 Fetching public transport stops in San Sebastián...
📍 Fetching public transport stops in Toledo...
✅ Collected 30124 public transport stops across 10 cities in Spain!


### Nature Spots

In [55]:
import requests
import pandas as pd
import time

OVERPASS_URL = "https://overpass-api.de/api/interpreter"

CITIES_SPAIN = [
    {"name": "Madrid", "lat": 40.4168, "lng": -3.7038, "radius": 20000},  
    {"name": "Barcelona", "lat": 41.3851, "lng": 2.1734, "radius": 18000},  
    {"name": "Seville", "lat": 37.3886, "lng": -5.9823, "radius": 15000},  
    {"name": "Valencia", "lat": 39.4699, "lng": -0.3763, "radius": 15000},  
    {"name": "Granada", "lat": 37.1773, "lng": -3.5986, "radius": 12000},  
    {"name": "Bilbao", "lat": 43.2630, "lng": -2.9340, "radius": 12000},  
    {"name": "Málaga", "lat": 36.7213, "lng": -4.4210, "radius": 10000},  
    {"name": "Santiago de Compostela", "lat": 42.8782, "lng": -8.5448, "radius": 12000},  
    {"name": "San Sebastián", "lat": 43.3183, "lng": -1.9812, "radius": 10000},  
    {"name": "Toledo", "lat": 39.8628, "lng": -4.0273, "radius": 15000}  
]

def get_osm_nature_spots(lat, lon):
    query = f"""
    [out:json];
    (
      node["leisure"="nature_reserve"](around:10000,{lat},{lon});
      node["leisure"="park"](around:10000,{lat},{lon});
      node["route"="hiking"](around:10000,{lat},{lon});
      node["piste:type"="downhill"](around:10000,{lat},{lon});
      node["natural"="peak"](around:10000,{lat},{lon});
    );
    out center;
    """
    
    response = requests.get(OVERPASS_URL, params={"data": query})
    return response.json().get("elements", []) if response.status_code == 200 else []

all_nature_spots = []
for city in CITIES_SPAIN:
    print(f"📍 Fetching nature spots in {city['name']}...")
    nature_spots = get_osm_nature_spots(city["lat"], city["lng"])

    for spot in nature_spots:
        tags = spot.get("tags", {})

        all_nature_spots.append({
            "city": city["name"],
            "name": tags.get("name", "Unknown"),
            "type": tags.get("leisure", tags.get("route", tags.get("natural", "Unknown"))),
            "latitude": spot.get("lat"),
            "longitude": spot.get("lon"),
            "elevation": tags.get("ele", "Unknown")
        })
    time.sleep(2)

df_nature = pd.DataFrame(all_nature_spots)
df_nature.to_csv("spain_nature_spots.csv", index=False)

print(f"✅ Collected {len(df_nature)} nature spots across 10 cities in Spain!")

📍 Fetching nature spots in Madrid...
📍 Fetching nature spots in Barcelona...
📍 Fetching nature spots in Seville...
📍 Fetching nature spots in Valencia...
📍 Fetching nature spots in Granada...
📍 Fetching nature spots in Bilbao...
📍 Fetching nature spots in Málaga...
📍 Fetching nature spots in Santiago de Compostela...
📍 Fetching nature spots in San Sebastián...
📍 Fetching nature spots in Toledo...
✅ Collected 1079 nature spots across 10 cities in Spain!


### Weather

In [74]:
import os
import requests
import pandas as pd
import time

# Enter your OpenWeather API key
OPENWEATHER_KEY = input("Enter your OpenWeather API key: ")
WEATHER_URL = "https://api.openweathermap.org/data/2.5/weather"

CITIES_SPAIN = [
    {"name": "Madrid", "lat": 40.4168, "lng": -3.7038},  
    {"name": "Barcelona", "lat": 41.3851, "lng": 2.1734},  
    {"name": "Seville", "lat": 37.3886, "lng": -5.9823},  
    {"name": "Valencia", "lat": 39.4699, "lng": -0.3763},  
    {"name": "Granada", "lat": 37.1773, "lng": -3.5986},  
    {"name": "Bilbao", "lat": 43.2630, "lng": -2.9340},  
    {"name": "Málaga", "lat": 36.7213, "lng": -4.4210},  
    {"name": "Santiago de Compostela", "lat": 42.8782, "lng": -8.5448},  
    {"name": "San Sebastián", "lat": 43.3183, "lng": -1.9812},  
    {"name": "Toledo", "lat": 39.8628, "lng": -4.0273}  
]

# Function to fetch weather data based on latitude and longitude
def get_weather(lat, lon):
    params = {
        "lat": lat,
        "lon": lon,
        "appid": OPENWEATHER_KEY,
        "units": "metric"  # Celsius
    }
    response = requests.get(WEATHER_URL, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"❌ Error fetching weather for {lat}, {lon}: {response.status_code}")
        return None

# Collect weather data for Spanish cities
all_weather_data = []

for city in CITIES_SPAIN:
    print(f"Fetching weather data for {city['name']}...")
    weather_data = get_weather(city["lat"], city["lng"])
    
    if weather_data:
        weather_info = {
            "city": city["name"],
            "latitude": city["lat"],
            "longitude": city["lng"],
            "temperature": weather_data["main"]["temp"],
            "weather": weather_data["weather"][0]["description"],
            "humidity": weather_data["main"]["humidity"],
            "wind_speed": weather_data["wind"]["speed"]
        }
        all_weather_data.append(weather_info)
    time.sleep(1)  # Avoid exceeding API limits

# Create DataFrame and save weather data
df_weather = pd.DataFrame(all_weather_data)
df_weather.to_csv("spain_weather_data.csv", index=False)

print(f"✅ Weather data for {len(df_weather)} Spanish cities collected and saved!")


Fetching weather data for Madrid...
Fetching weather data for Barcelona...
Fetching weather data for Seville...
Fetching weather data for Valencia...
Fetching weather data for Granada...
Fetching weather data for Bilbao...
Fetching weather data for Málaga...
Fetching weather data for Santiago de Compostela...
Fetching weather data for San Sebastián...
Fetching weather data for Toledo...
✅ Weather data for 10 Spanish cities collected and saved!


## England

### Tourist Attractions

In [29]:
import requests
import pandas as pd
import time

# 🔑 Insert your Google API Key here
API_KEY = input("Enter your Google API key: ")
PLACES_URL = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# 📍 Top 10 Most Visited Cities in England with Coordinates & Search Radius (in meters)
CITIES_ENGLAND = [
    {"name": "London", "lat": 51.5074, "lng": -0.1278, "radius": 20000},  
    {"name": "Manchester", "lat": 53.483, "lng": -2.244, "radius": 18000},  
    {"name": "Birmingham", "lat": 52.4862, "lng": -1.8904, "radius": 15000},  
    {"name": "Liverpool", "lat": 53.4084, "lng": -2.9916, "radius": 15000},  
    {"name": "Oxford", "lat": 51.752, "lng": -1.2577, "radius": 12000},  
    {"name": "Cambridge", "lat": 52.2053, "lng": 0.1218, "radius": 12000},  
    {"name": "Bristol", "lat": 51.4545, "lng": -2.5879, "radius": 10000},  
    {"name": "York", "lat": 53.959, "lng": -1.0815, "radius": 12000},  
    {"name": "Bath", "lat": 51.3758, "lng": -2.3599, "radius": 10000},  
    {"name": "Brighton", "lat": 50.8225, "lng": -0.1372, "radius": 15000}  
]

# Function to fetch tourist attractions for a given city
def get_places(city_name, lat, lng, radius, type_place="tourist_attraction"):
    params = {
        "location": f"{lat},{lng}",
        "radius": radius,
        "type": type_place,
        "key": API_KEY
    }
    
    places = []
    next_page_token = None

    print(f"📍 Fetching attractions in {city_name}...")
    
    while True:
        if next_page_token:
            params["pagetoken"] = next_page_token
        
        response = requests.get(PLACES_URL, params=params)
        data = response.json()
        
        if response.status_code != 200 or "results" not in data:
            print(f"⚠️ Error fetching data for {city_name}: {data.get('status', 'Unknown error')}")
            break
        
        places.extend(data["results"])
        
        next_page_token = data.get("next_page_token")
        if not next_page_token:
            break
        
        time.sleep(2)  # Google requires a small delay before using next_page_token

    return places

# Collect data for all cities
all_places = []
seen_places = set()

for city in CITIES_ENGLAND:
    places = get_places(city["name"], city["lat"], city["lng"], city["radius"])
    
    for p in places:
        place_id = p["place_id"]
        
        if place_id not in seen_places:
            seen_places.add(place_id)
            all_places.append({
                "city": city["name"],
                "place_id": place_id,
                "name": p.get("name", "Unknown"),
                "rating": p.get("rating", "N/A"),
                "address": p.get("vicinity", "Unknown"),
                "latitude": p["geometry"]["location"]["lat"],
                "longitude": p["geometry"]["location"]["lng"],
                "types": ", ".join(p.get("types", []))
            })

    time.sleep(5)  # Avoid excessive API requests

# Save data to CSV
df_places = pd.DataFrame(all_places)
df_places.to_csv("england_tourist_attractions.csv", index=False)

print(f"✅ Collected {len(df_places)} tourist attractions across 10 cities in England!")


📍 Fetching attractions in London...
📍 Fetching attractions in Manchester...
📍 Fetching attractions in Birmingham...
📍 Fetching attractions in Liverpool...
📍 Fetching attractions in Oxford...
📍 Fetching attractions in Cambridge...
📍 Fetching attractions in Bristol...
📍 Fetching attractions in York...
📍 Fetching attractions in Bath...
📍 Fetching attractions in Brighton...
✅ Collected 599 tourist attractions across 10 cities in England!


### Events

In [12]:
import requests
import pandas as pd
import time

# 🔑 Solicita a chave da API do usuário
API_KEY = input("Enter your Google API key: ")
PLACES_URL = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# 📍 Top 10 Most Visited Cities in England with Coordinates & Search Radius (in meters)
CITIES_ENGLAND = [
    {"name": "London", "lat": 51.5074, "lng": -0.1278, "radius": 15000},
    {"name": "Manchester", "lat": 53.4808, "lng": -2.2426, "radius": 12000},
    {"name": "Birmingham", "lat": 52.4862, "lng": -1.8904, "radius": 12000},
    {"name": "Liverpool", "lat": 53.4084, "lng": -2.9916, "radius": 10000},
    {"name": "Oxford", "lat": 51.7520, "lng": -1.2577, "radius": 10000},
    {"name": "Cambridge", "lat": 52.2053, "lng": 0.1218, "radius": 10000},
    {"name": "Bristol", "lat": 51.4545, "lng": -2.5879, "radius": 10000},
    {"name": "York", "lat": 53.9590, "lng": -1.0815, "radius": 10000},
    {"name": "Bath", "lat": 51.3758, "lng": -2.3599, "radius": 10000},
    {"name": "Brighton", "lat": 50.8225, "lng": -0.1372, "radius": 10000}
]

# 🎭 Define categories of places where events may occur
EVENT_TYPES = [
    "stadium", "casino", "night_club", "art_gallery", "museum",
    "tourist_attraction", "amusement_park", "university", "movie_theater"
]

# Function to fetch event venues for a city
def get_places(city_name, lat, lng, radius):
    places = []
    seen_places = set()

    for place_type in EVENT_TYPES:
        params = {
            "location": f"{lat},{lng}",
            "radius": radius,
            "type": place_type,
            "key": API_KEY
        }
        
        print(f"📍 Fetching event venues in {city_name} ({place_type})...")
        
        response = requests.get(PLACES_URL, params=params)
        data = response.json()

        if response.status_code == 200 and "results" in data:
            for p in data["results"]:
                place_id = p["place_id"]

                if place_id not in seen_places:
                    seen_places.add(place_id)
                    places.append({
                        "city": city_name,
                        "place_id": place_id,
                        "name": p.get("name", "Unknown"),
                        "rating": p.get("rating", "N/A"),
                        "address": p.get("vicinity", "Unknown"),
                        "latitude": p["geometry"]["location"]["lat"],
                        "longitude": p["geometry"]["location"]["lng"],
                        "types": ", ".join(p.get("types", []))
                    })
        
        # Wait a bit to avoid exceeding API request limits
        time.sleep(2)
    
    return places

# 🔍 Collect event venue data for all cities
all_places = []
for city in CITIES_ENGLAND:
    all_places.extend(get_places(city["name"], city["lat"], city["lng"], city["radius"]))

# 📝 Save data to CSV
df_places = pd.DataFrame(all_places)
df_places.to_csv("england_event_locations.csv", index=False)

print(f"✅ Collected {len(df_places)} event venues across 10 cities in England! Saved as 'england_event_locations.csv'")


📍 Fetching event venues in London (stadium)...
📍 Fetching event venues in London (casino)...
📍 Fetching event venues in London (night_club)...
📍 Fetching event venues in London (art_gallery)...
📍 Fetching event venues in London (museum)...
📍 Fetching event venues in London (tourist_attraction)...
📍 Fetching event venues in London (amusement_park)...
📍 Fetching event venues in London (university)...
📍 Fetching event venues in London (movie_theater)...
📍 Fetching event venues in Manchester (stadium)...
📍 Fetching event venues in Manchester (casino)...
📍 Fetching event venues in Manchester (night_club)...
📍 Fetching event venues in Manchester (art_gallery)...
📍 Fetching event venues in Manchester (museum)...
📍 Fetching event venues in Manchester (tourist_attraction)...
📍 Fetching event venues in Manchester (amusement_park)...
📍 Fetching event venues in Manchester (university)...
📍 Fetching event venues in Manchester (movie_theater)...
📍 Fetching event venues in Birmingham (stadium)...
📍 F

### Hotels

In [46]:
import requests 
import pandas as pd
import time

OVERPASS_URL = "https://overpass-api.de/api/interpreter"

CITIES_ENGLAND = [
    {"name": "London", "lat": 51.5074, "lng": -0.1278, "radius": 20000},  
    {"name": "Manchester", "lat": 53.483, "lng": -2.244, "radius": 18000},  
    {"name": "Birmingham", "lat": 52.4862, "lng": -1.8904, "radius": 15000},  
    {"name": "Liverpool", "lat": 53.4084, "lng": -2.9916, "radius": 15000},  
    {"name": "Oxford", "lat": 51.752, "lng": -1.2577, "radius": 12000},  
    {"name": "Cambridge", "lat": 52.2053, "lng": 0.1218, "radius": 12000},  
    {"name": "Bristol", "lat": 51.4545, "lng": -2.5879, "radius": 10000},  
    {"name": "York", "lat": 53.959, "lng": -1.0815, "radius": 12000},  
    {"name": "Bath", "lat": 51.3758, "lng": -2.3599, "radius": 10000},  
    {"name": "Brighton", "lat": 50.8225, "lng": -0.1372, "radius": 15000}  
]

def get_osm_hotels(lat, lon):
    query = f"""
    [out:json];
    (
      node["tourism"="hotel"](around:10000,{lat},{lon});
      node["tourism"="hostel"](around:10000,{lat},{lon});
      node["tourism"="guest_house"](around:10000,{lat},{lon});
    );
    out center;
    """
    
    response = requests.get(OVERPASS_URL, params={"data": query})
    return response.json().get("elements", []) if response.status_code == 200 else []

all_hotels = []
for city in CITIES_ENGLAND:
    print(f"📍 Fetching hotels and hostels in {city['name']}...")
    hotels = get_osm_hotels(city["lat"], city["lng"])

    for hotel in hotels:
        tags = hotel.get("tags", {})

        all_hotels.append({
            "city": city["name"],
            "name": tags.get("name", "Unknown"),
            "type": tags.get("tourism", "Unknown"),
            "latitude": hotel.get("lat"),
            "longitude": hotel.get("lon"),
            "stars": tags.get("stars", "Unknown"),
            "operator": tags.get("operator", "Unknown")
        })
    time.sleep(2)

df_hotels = pd.DataFrame(all_hotels)
df_hotels.to_csv("england_hotels_hostels.csv", index=False)

print(f"✅ Collected {len(df_hotels)} hotels and hostels across 10 cities in England!")


📍 Fetching hotels and hostels in London...
📍 Fetching hotels and hostels in Manchester...
📍 Fetching hotels and hostels in Birmingham...
📍 Fetching hotels and hostels in Liverpool...
📍 Fetching hotels and hostels in Oxford...
📍 Fetching hotels and hostels in Cambridge...
📍 Fetching hotels and hostels in Bristol...
📍 Fetching hotels and hostels in York...
📍 Fetching hotels and hostels in Bath...
📍 Fetching hotels and hostels in Brighton...
✅ Collected 849 hotels and hostels across 10 cities in England!


### Restaurants

In [19]:
import requests
import pandas as pd
import time

OVERPASS_URL = "https://overpass-api.de/api/interpreter"

# 📍 Top 10 Most Visited Cities in England
CITIES_ENGLAND = [
    {"name": "London", "lat": 51.5074, "lon": -0.1278, "radius": 10000},
    {"name": "Manchester", "lat": 53.4808, "lon": -2.2426, "radius": 8000},
    {"name": "Birmingham", "lat": 52.4862, "lon": -1.8904, "radius": 8000},
    {"name": "Liverpool", "lat": 53.4084, "lon": -2.9916, "radius": 7000},
    {"name": "Oxford", "lat": 51.7520, "lon": -1.2577, "radius": 7000},
    {"name": "Cambridge", "lat": 52.2053, "lon": 0.1218, "radius": 7000},
    {"name": "Bristol", "lat": 51.4545, "lon": -2.5879, "radius": 7000},
    {"name": "York", "lat": 53.9590, "lon": -1.0815, "radius": 7000},
    {"name": "Bath", "lat": 51.3758, "lon": -2.3599, "radius": 7000},
    {"name": "Brighton", "lat": 50.8225, "lon": -0.1372, "radius": 7000}
]

def get_osm_restaurants(city):
    """Fetches restaurants, cafés, and bars in a specific English city with retries."""
    query = f"""
    [out:json];
    (
      node["amenity"="restaurant"](around:{city["radius"]},{city["lat"]},{city["lon"]});
      node["amenity"="cafe"](around:{city["radius"]},{city["lat"]},{city["lon"]});
      node["amenity"="bar"](around:{city["radius"]},{city["lat"]},{city["lon"]});
    );
    out center;
    """

    retries = 3
    while retries > 0:
        response = requests.get(OVERPASS_URL, params={"data": query})
        
        if response.status_code == 200:
            return response.json().get("elements", [])
        else:
            print(f"⚠️ Error fetching restaurants in {city['name']} ({response.status_code}), retrying... ({3 - retries + 1}/3)")
            retries -= 1
            time.sleep(3)  # Aguardar antes de tentar novamente
    
    print(f"❌ Failed to fetch data for {city['name']} after multiple retries.")
    return []

# 🔍 Collect data for all cities
all_places = []
for city in CITIES_ENGLAND:
    print(f"📍 Fetching gastronomic locations in {city['name']}...")
    places = get_osm_restaurants(city)

    for place in places:
        tags = place.get("tags", {})

        all_places.append({
            "city": city["name"],
            "name": tags.get("name", "Unknown"),
            "type": tags.get("amenity", "Unknown"),
            "latitude": place.get("lat"),
            "longitude": place.get("lon"),
            "street": tags.get("addr:street", "Unknown"),
            "postcode": tags.get("addr:postcode", "Unknown"),
            "cuisine": tags.get("cuisine", "Not specified")  # Tipo de comida
        })

    time.sleep(2)  # Evitar sobrecarga na API

# 📝 Save data to CSV
df = pd.DataFrame(all_places)
df.to_csv("england_all_restaurants_bars_cafes.csv", index=False)

print(f"✅ Collected {len(df)} gastronomic locations across 10 cities in England! Saved as 'england_all_restaurants_bars_cafes.csv'")


📍 Fetching gastronomic locations in London...
📍 Fetching gastronomic locations in Manchester...
📍 Fetching gastronomic locations in Birmingham...
📍 Fetching gastronomic locations in Liverpool...
📍 Fetching gastronomic locations in Oxford...
📍 Fetching gastronomic locations in Cambridge...
📍 Fetching gastronomic locations in Bristol...
📍 Fetching gastronomic locations in York...
📍 Fetching gastronomic locations in Bath...
📍 Fetching gastronomic locations in Brighton...
✅ Collected 11768 gastronomic locations across 10 cities in England! Saved as 'england_all_restaurants_bars_cafes.csv'


### Public Transport Stops

In [33]:
import requests
import pandas as pd

OVERPASS_URL = "https://overpass-api.de/api/interpreter"

CITIES_ENGLAND = [
    {"name": "London", "lat": 51.5074, "lon": -0.1278, "radius": 20000},  
    {"name": "Manchester", "lat": 53.483, "lon": -2.244, "radius": 18000},  
    {"name": "Birmingham", "lat": 52.4862, "lon": -1.8904, "radius": 15000},  
    {"name": "Liverpool", "lat": 53.4084, "lon": -2.9916, "radius": 15000},  
    {"name": "Oxford", "lat": 51.752, "lon": -1.2577, "radius": 12000},  
    {"name": "Cambridge", "lat": 52.2053, "lon": 0.1218, "radius": 12000},  
    {"name": "Bristol", "lat": 51.4545, "lon": -2.5879, "radius": 10000},  
    {"name": "York", "lat": 53.959, "lon": -1.0815, "radius": 12000},  
    {"name": "Bath", "lat": 51.3758, "lon": -2.3599, "radius": 10000},  
    {"name": "Brighton", "lat": 50.8225, "lon": -0.1372, "radius": 15000}  
]

def get_osm_transit_stops(city):
    query = f"""
    [out:json];
    (
      node["highway"="bus_stop"](around:{city["radius"]},{city["lat"]},{city["lon"]});
      node["railway"="station"](around:{city["radius"]},{city["lat"]},{city["lon"]});
      node["public_transport"="platform"](around:{city["radius"]},{city["lat"]},{city["lon"]});
    );
    out center;
    """
    
    response = requests.get(OVERPASS_URL, params={"data": query})
    return response.json().get("elements", []) if response.status_code == 200 else []

all_stops = []
for city in CITIES_ENGLAND:
    print(f"📍 Fetching public transport stops in {city['name']}...")
    stops = get_osm_transit_stops(city)

    for stop in stops:
        tags = stop.get("tags", {})

        all_stops.append({
            "city": city["name"],
            "name": tags.get("name", "Unknown"),
            "type": tags.get("highway", tags.get("railway", tags.get("public_transport", "Unknown"))),
            "latitude": stop.get("lat"),
            "longitude": stop.get("lon"),
            "operator": tags.get("operator", "Unknown")
        })

df_transit = pd.DataFrame(all_stops)
df_transit.to_csv("england_transit_stops.csv", index=False)

print(f"✅ Collected {len(df_transit)} public transport stops across 10 cities in England!")

📍 Fetching public transport stops in London...
📍 Fetching public transport stops in Manchester...
📍 Fetching public transport stops in Birmingham...
📍 Fetching public transport stops in Liverpool...
📍 Fetching public transport stops in Oxford...
📍 Fetching public transport stops in Cambridge...
📍 Fetching public transport stops in Bristol...
📍 Fetching public transport stops in York...
📍 Fetching public transport stops in Bath...
📍 Fetching public transport stops in Brighton...
✅ Collected 49585 public transport stops across 10 cities in England!


### Nature Spots

In [56]:
import requests
import pandas as pd
import time

OVERPASS_URL = "https://overpass-api.de/api/interpreter"

CITIES_ENGLAND = [
    {"name": "London", "lat": 51.5074, "lng": -0.1278, "radius": 20000},  
    {"name": "Manchester", "lat": 53.483, "lng": -2.244, "radius": 18000},  
    {"name": "Birmingham", "lat": 52.4862, "lng": -1.8904, "radius": 15000},  
    {"name": "Liverpool", "lat": 53.4084, "lng": -2.9916, "radius": 15000},  
    {"name": "Oxford", "lat": 51.752, "lng": -1.2577, "radius": 12000},  
    {"name": "Cambridge", "lat": 52.2053, "lng": 0.1218, "radius": 12000},  
    {"name": "Bristol", "lat": 51.4545, "lng": -2.5879, "radius": 10000},  
    {"name": "York", "lat": 53.959, "lng": -1.0815, "radius": 12000},  
    {"name": "Bath", "lat": 51.3758, "lng": -2.3599, "radius": 10000},  
    {"name": "Brighton", "lat": 50.8225, "lng": -0.1372, "radius": 15000}  
]

def get_osm_nature_spots(lat, lon):
    query = f"""
    [out:json];
    (
      node["leisure"="nature_reserve"](around:10000,{lat},{lon});
      node["leisure"="park"](around:10000,{lat},{lon});
      node["route"="hiking"](around:10000,{lat},{lon});
      node["piste:type"="downhill"](around:10000,{lat},{lon});
      node["natural"="peak"](around:10000,{lat},{lon});
    );
    out center;
    """
    
    response = requests.get(OVERPASS_URL, params={"data": query})
    return response.json().get("elements", []) if response.status_code == 200 else []

all_nature_spots = []
for city in CITIES_ENGLAND:
    print(f"📍 Fetching nature spots in {city['name']}...")
    nature_spots = get_osm_nature_spots(city["lat"], city["lng"])

    for spot in nature_spots:
        tags = spot.get("tags", {})

        all_nature_spots.append({
            "city": city["name"],
            "name": tags.get("name", "Unknown"),
            "type": tags.get("leisure", tags.get("route", tags.get("natural", "Unknown"))),
            "latitude": spot.get("lat"),
            "longitude": spot.get("lon"),
            "elevation": tags.get("ele", "Unknown")
        })
    time.sleep(2)

df_nature = pd.DataFrame(all_nature_spots)
df_nature.to_csv("england_nature_spots.csv", index=False)

print(f"✅ Collected {len(df_nature)} nature spots across 10 cities in England!")


📍 Fetching nature spots in London...
📍 Fetching nature spots in Manchester...
📍 Fetching nature spots in Birmingham...
📍 Fetching nature spots in Liverpool...
📍 Fetching nature spots in Oxford...
📍 Fetching nature spots in Cambridge...
📍 Fetching nature spots in Bristol...
📍 Fetching nature spots in York...
📍 Fetching nature spots in Bath...
📍 Fetching nature spots in Brighton...
✅ Collected 189 nature spots across 10 cities in England!


### Weather

In [73]:
import os
import requests
import pandas as pd
import time

# Enter your OpenWeather API key
OPENWEATHER_KEY = input("Enter your OpenWeather API key: ")
WEATHER_URL = "https://api.openweathermap.org/data/2.5/weather"

CITIES_ENGLAND = [
    {"name": "London", "lat": 51.5074, "lng": -0.1278},  
    {"name": "Manchester", "lat": 53.483, "lng": -2.244},  
    {"name": "Birmingham", "lat": 52.4862, "lng": -1.8904},  
    {"name": "Liverpool", "lat": 53.4084, "lng": -2.9916},  
    {"name": "Oxford", "lat": 51.752, "lng": -1.2577},  
    {"name": "Cambridge", "lat": 52.2053, "lng": 0.1218},  
    {"name": "Bristol", "lat": 51.4545, "lng": -2.5879},  
    {"name": "York", "lat": 53.959, "lng": -1.0815},  
    {"name": "Bath", "lat": 51.3758, "lng": -2.3599},  
    {"name": "Brighton", "lat": 50.8225, "lng": -0.1372}  
]

# Function to fetch weather data based on latitude and longitude
def get_weather(lat, lon):
    params = {
        "lat": lat,
        "lon": lon,
        "appid": OPENWEATHER_KEY,
        "units": "metric"  # Celsius
    }
    response = requests.get(WEATHER_URL, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"❌ Error fetching weather for {lat}, {lon}: {response.status_code}")
        return None

# Collect weather data for English cities
all_weather_data = []

for city in CITIES_ENGLAND:
    print(f"Fetching weather data for {city['name']}...")
    weather_data = get_weather(city["lat"], city["lng"])
    
    if weather_data:
        weather_info = {
            "city": city["name"],
            "latitude": city["lat"],
            "longitude": city["lng"],
            "temperature": weather_data["main"]["temp"],
            "weather": weather_data["weather"][0]["description"],
            "humidity": weather_data["main"]["humidity"],
            "wind_speed": weather_data["wind"]["speed"]
        }
        all_weather_data.append(weather_info)
    time.sleep(1)  # Avoid exceeding API limits

# Create DataFrame and save weather data
df_weather = pd.DataFrame(all_weather_data)
df_weather.to_csv("england_weather_data.csv", index=False)

print(f"✅ Weather data for {len(df_weather)} English cities collected and saved!")


Fetching weather data for London...
Fetching weather data for Manchester...
Fetching weather data for Birmingham...
Fetching weather data for Liverpool...
Fetching weather data for Oxford...
Fetching weather data for Cambridge...
Fetching weather data for Bristol...
Fetching weather data for York...
Fetching weather data for Bath...
Fetching weather data for Brighton...
✅ Weather data for 10 English cities collected and saved!


## Norway

### Tourist Attractions

In [30]:
import requests
import pandas as pd
import time

# 🔑 Insert your Google API Key here
API_KEY = input("Enter your Google API key: ")
PLACES_URL = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# 📍 Top 10 Most Visited Cities in Norway with Coordinates & Search Radius (in meters)
CITIES_NORWAY = [
    {"name": "Oslo", "lat": 59.9139, "lng": 10.7522, "radius": 20000},  
    {"name": "Bergen", "lat": 60.3913, "lng": 5.3221, "radius": 18000},  
    {"name": "Trondheim", "lat": 63.4305, "lng": 10.3951, "radius": 15000},  
    {"name": "Stavanger", "lat": 58.9700, "lng": 5.7331, "radius": 15000},  
    {"name": "Tromsø", "lat": 69.6496, "lng": 18.9560, "radius": 12000},  
    {"name": "Alesund", "lat": 62.4722, "lng": 6.1495, "radius": 12000},  
    {"name": "Kristiansand", "lat": 58.1467, "lng": 7.9956, "radius": 10000},  
    {"name": "Lillehammer", "lat": 61.1153, "lng": 10.4663, "radius": 12000},  
    {"name": "Bodø", "lat": 67.2804, "lng": 14.4049, "radius": 10000},  
    {"name": "Narvik", "lat": 68.4385, "lng": 17.4270, "radius": 15000}  
]

# Function to fetch tourist attractions for a given city
def get_places(city_name, lat, lng, radius, type_place="tourist_attraction"):
    params = {
        "location": f"{lat},{lng}",
        "radius": radius,
        "type": type_place,
        "key": API_KEY
    }
    
    places = []
    next_page_token = None

    print(f"📍 Fetching attractions in {city_name}...")
    
    while True:
        if next_page_token:
            params["pagetoken"] = next_page_token
        
        response = requests.get(PLACES_URL, params=params)
        data = response.json()
        
        if response.status_code != 200 or "results" not in data:
            print(f"⚠️ Error fetching data for {city_name}: {data.get('status', 'Unknown error')}")
            break
        
        places.extend(data["results"])
        
        next_page_token = data.get("next_page_token")
        if not next_page_token:
            break
        
        time.sleep(2)  # Google requires a small delay before using next_page_token

    return places

# Collect data for all cities
all_places = []
seen_places = set()

for city in CITIES_NORWAY:
    places = get_places(city["name"], city["lat"], city["lng"], city["radius"])
    
    for p in places:
        place_id = p["place_id"]
        
        if place_id not in seen_places:
            seen_places.add(place_id)
            all_places.append({
                "city": city["name"],
                "place_id": place_id,
                "name": p.get("name", "Unknown"),
                "rating": p.get("rating", "N/A"),
                "address": p.get("vicinity", "Unknown"),
                "latitude": p["geometry"]["location"]["lat"],
                "longitude": p["geometry"]["location"]["lng"],
                "types": ", ".join(p.get("types", []))
            })

    time.sleep(5)  # Avoid excessive API requests

# Save data to CSV
df_places = pd.DataFrame(all_places)
df_places.to_csv("norway_tourist_attractions.csv", index=False)

print(f"✅ Collected {len(df_places)} tourist attractions across 10 cities in Norway!")


📍 Fetching attractions in Oslo...
📍 Fetching attractions in Bergen...
📍 Fetching attractions in Trondheim...
📍 Fetching attractions in Stavanger...
📍 Fetching attractions in Tromsø...
📍 Fetching attractions in Alesund...
📍 Fetching attractions in Kristiansand...
📍 Fetching attractions in Lillehammer...
📍 Fetching attractions in Bodø...
📍 Fetching attractions in Narvik...
✅ Collected 482 tourist attractions across 10 cities in Norway!


### Events

In [20]:
import requests
import pandas as pd
import time

# 🔑 Solicita a chave da API do usuário
API_KEY = input("Enter your Google API key: ")
PLACES_URL = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# 📍 Top 10 Most Visited Cities in Norway with Coordinates & Search Radius (in meters)
CITIES_NORWAY = [
    {"name": "Oslo", "lat": 59.9139, "lng": 10.7522, "radius": 15000},
    {"name": "Bergen", "lat": 60.3913, "lng": 5.3221, "radius": 12000},
    {"name": "Trondheim", "lat": 63.4305, "lng": 10.3951, "radius": 12000},
    {"name": "Stavanger", "lat": 58.9690, "lng": 5.7331, "radius": 10000},
    {"name": "Tromsø", "lat": 69.6496, "lng": 18.9560, "radius": 10000},
    {"name": "Alesund", "lat": 62.4722, "lng": 6.1549, "radius": 10000},
    {"name": "Kristiansand", "lat": 58.1465, "lng": 7.9954, "radius": 10000},
    {"name": "Lillehammer", "lat": 61.1153, "lng": 10.4662, "radius": 10000},
    {"name": "Bodø", "lat": 67.2804, "lng": 14.4049, "radius": 10000},
    {"name": "Narvik", "lat": 68.4385, "lng": 17.4270, "radius": 10000}
]

# 🎭 Define categories of places where events may occur
EVENT_TYPES = [
    "stadium", "casino", "night_club", "art_gallery", "museum",
    "tourist_attraction", "amusement_park", "university", "movie_theater"
]

# Function to fetch event venues for a city
def get_places(city_name, lat, lng, radius):
    places = []
    seen_places = set()

    for place_type in EVENT_TYPES:
        params = {
            "location": f"{lat},{lng}",
            "radius": radius,
            "type": place_type,
            "key": API_KEY
        }
        
        print(f"📍 Fetching event venues in {city_name} ({place_type})...")
        
        response = requests.get(PLACES_URL, params=params)
        data = response.json()

        if response.status_code == 200 and "results" in data:
            for p in data["results"]:
                place_id = p["place_id"]

                if place_id not in seen_places:
                    seen_places.add(place_id)
                    places.append({
                        "city": city_name,
                        "place_id": place_id,
                        "name": p.get("name", "Unknown"),
                        "rating": p.get("rating", "N/A"),
                        "address": p.get("vicinity", "Unknown"),
                        "latitude": p["geometry"]["location"]["lat"],
                        "longitude": p["geometry"]["location"]["lng"],
                        "types": ", ".join(p.get("types", []))
                    })
        
        # Wait a bit to avoid exceeding API request limits
        time.sleep(2)
    
    return places

# 🔍 Collect event venue data for all cities
all_places = []
for city in CITIES_NORWAY:
    all_places.extend(get_places(city["name"], city["lat"], city["lng"], city["radius"]))

# 📝 Save data to CSV
df_places = pd.DataFrame(all_places)
df_places.to_csv("norway_event_locations.csv", index=False)

print(f"✅ Collected {len(df_places)} event venues across 10 cities in Norway! Saved as 'norway_event_locations.csv'")


📍 Fetching event venues in Oslo (stadium)...
📍 Fetching event venues in Oslo (casino)...
📍 Fetching event venues in Oslo (night_club)...
📍 Fetching event venues in Oslo (art_gallery)...
📍 Fetching event venues in Oslo (museum)...
📍 Fetching event venues in Oslo (tourist_attraction)...
📍 Fetching event venues in Oslo (amusement_park)...
📍 Fetching event venues in Oslo (university)...
📍 Fetching event venues in Oslo (movie_theater)...
📍 Fetching event venues in Bergen (stadium)...
📍 Fetching event venues in Bergen (casino)...
📍 Fetching event venues in Bergen (night_club)...
📍 Fetching event venues in Bergen (art_gallery)...
📍 Fetching event venues in Bergen (museum)...
📍 Fetching event venues in Bergen (tourist_attraction)...
📍 Fetching event venues in Bergen (amusement_park)...
📍 Fetching event venues in Bergen (university)...
📍 Fetching event venues in Bergen (movie_theater)...
📍 Fetching event venues in Trondheim (stadium)...
📍 Fetching event venues in Trondheim (casino)...
📍 Fetchin

### Hotels

In [47]:
import requests 
import pandas as pd
import time

OVERPASS_URL = "https://overpass-api.de/api/interpreter"

CITIES_NORWAY = [
    {"name": "Oslo", "lat": 59.9139, "lng": 10.7522, "radius": 20000},  
    {"name": "Bergen", "lat": 60.3913, "lng": 5.3221, "radius": 18000},  
    {"name": "Trondheim", "lat": 63.4305, "lng": 10.3951, "radius": 15000},  
    {"name": "Stavanger", "lat": 58.9700, "lng": 5.7331, "radius": 15000},  
    {"name": "Tromsø", "lat": 69.6496, "lng": 18.9560, "radius": 12000},  
    {"name": "Alesund", "lat": 62.4722, "lng": 6.1495, "radius": 12000},  
    {"name": "Kristiansand", "lat": 58.1467, "lng": 7.9956, "radius": 10000},  
    {"name": "Lillehammer", "lat": 61.1153, "lng": 10.4663, "radius": 12000},  
    {"name": "Bodø", "lat": 67.2804, "lng": 14.4049, "radius": 10000},  
    {"name": "Narvik", "lat": 68.4385, "lng": 17.4270, "radius": 15000}  
]

def get_osm_hotels(lat, lon):
    query = f"""
    [out:json];
    (
      node["tourism"="hotel"](around:10000,{lat},{lon});
      node["tourism"="hostel"](around:10000,{lat},{lon});
      node["tourism"="guest_house"](around:10000,{lat},{lon});
    );
    out center;
    """
    
    response = requests.get(OVERPASS_URL, params={"data": query})
    return response.json().get("elements", []) if response.status_code == 200 else []

all_hotels = []
for city in CITIES_NORWAY:
    print(f"📍 Fetching hotels and hostels in {city['name']}...")
    hotels = get_osm_hotels(city["lat"], city["lng"])

    for hotel in hotels:
        tags = hotel.get("tags", {})

        all_hotels.append({
            "city": city["name"],
            "name": tags.get("name", "Unknown"),
            "type": tags.get("tourism", "Unknown"),
            "latitude": hotel.get("lat"),
            "longitude": hotel.get("lon"),
            "stars": tags.get("stars", "Unknown"),
            "operator": tags.get("operator", "Unknown")
        })
    time.sleep(2)

df_hotels = pd.DataFrame(all_hotels)
df_hotels.to_csv("norway_hotels_hostels.csv", index=False)

print(f"✅ Collected {len(df_hotels)} hotels and hostels across 10 cities in Norway!")

📍 Fetching hotels and hostels in Oslo...
📍 Fetching hotels and hostels in Bergen...
📍 Fetching hotels and hostels in Trondheim...
📍 Fetching hotels and hostels in Stavanger...
📍 Fetching hotels and hostels in Tromsø...
📍 Fetching hotels and hostels in Alesund...
📍 Fetching hotels and hostels in Kristiansand...
📍 Fetching hotels and hostels in Lillehammer...
📍 Fetching hotels and hostels in Bodø...
📍 Fetching hotels and hostels in Narvik...
✅ Collected 211 hotels and hostels across 10 cities in Norway!


### Restaurants

In [21]:
import requests
import pandas as pd
import time

OVERPASS_URL = "https://overpass-api.de/api/interpreter"

# 📍 Top 10 Most Visited Cities in Norway
CITIES_NORWAY = [
    {"name": "Oslo", "lat": 59.9139, "lon": 10.7522, "radius": 10000},
    {"name": "Bergen", "lat": 60.3913, "lon": 5.3221, "radius": 8000},
    {"name": "Trondheim", "lat": 63.4305, "lon": 10.3951, "radius": 8000},
    {"name": "Stavanger", "lat": 58.9690, "lon": 5.7331, "radius": 7000},
    {"name": "Tromsø", "lat": 69.6496, "lon": 18.9560, "radius": 7000},
    {"name": "Alesund", "lat": 62.4722, "lon": 6.1495, "radius": 7000},
    {"name": "Kristiansand", "lat": 58.1465, "lon": 7.9955, "radius": 7000},
    {"name": "Lillehammer", "lat": 61.1153, "lon": 10.4662, "radius": 7000},
    {"name": "Bodø", "lat": 67.2804, "lon": 14.4049, "radius": 7000},
    {"name": "Narvik", "lat": 68.4385, "lon": 17.4270, "radius": 7000}
]

def get_osm_restaurants(city):
    """Fetches restaurants, cafés, and bars in a specific Norwegian city with retries."""
    query = f"""
    [out:json];
    (
      node["amenity"="restaurant"](around:{city["radius"]},{city["lat"]},{city["lon"]});
      node["amenity"="cafe"](around:{city["radius"]},{city["lat"]},{city["lon"]});
      node["amenity"="bar"](around:{city["radius"]},{city["lat"]},{city["lon"]});
    );
    out center;
    """

    retries = 3
    while retries > 0:
        response = requests.get(OVERPASS_URL, params={"data": query})
        
        if response.status_code == 200:
            return response.json().get("elements", [])
        else:
            print(f"⚠️ Error fetching restaurants in {city['name']} ({response.status_code}), retrying... ({3 - retries + 1}/3)")
            retries -= 1
            time.sleep(3)  # Aguardar antes de tentar novamente
    
    print(f"❌ Failed to fetch data for {city['name']} after multiple retries.")
    return []

# 🔍 Collect data for all cities
all_places = []
for city in CITIES_NORWAY:
    print(f"📍 Fetching gastronomic locations in {city['name']}...")
    places = get_osm_restaurants(city)

    for place in places:
        tags = place.get("tags", {})

        all_places.append({
            "city": city["name"],
            "name": tags.get("name", "Unknown"),
            "type": tags.get("amenity", "Unknown"),
            "latitude": place.get("lat"),
            "longitude": place.get("lon"),
            "street": tags.get("addr:street", "Unknown"),
            "postcode": tags.get("addr:postcode", "Unknown"),
            "cuisine": tags.get("cuisine", "Not specified")  # Tipo de comida
        })

    time.sleep(2)  # Evitar sobrecarga na API

# 📝 Save data to CSV
df = pd.DataFrame(all_places)
df.to_csv("norway_all_restaurants_bars_cafes.csv", index=False)

print(f"✅ Collected {len(df)} gastronomic locations across 10 cities in Norway! Saved as 'norway_all_restaurants_bars_cafes.csv'")


📍 Fetching gastronomic locations in Oslo...
📍 Fetching gastronomic locations in Bergen...
📍 Fetching gastronomic locations in Trondheim...
📍 Fetching gastronomic locations in Stavanger...
📍 Fetching gastronomic locations in Tromsø...
📍 Fetching gastronomic locations in Alesund...
📍 Fetching gastronomic locations in Kristiansand...
📍 Fetching gastronomic locations in Lillehammer...
📍 Fetching gastronomic locations in Bodø...
📍 Fetching gastronomic locations in Narvik...
✅ Collected 2216 gastronomic locations across 10 cities in Norway! Saved as 'norway_all_restaurants_bars_cafes.csv'


### Public Transport Stops

In [34]:
import requests
import pandas as pd

OVERPASS_URL = "https://overpass-api.de/api/interpreter"

CITIES_NORWAY = [
    {"name": "Oslo", "lat": 59.9139, "lon": 10.7522, "radius": 20000},  
    {"name": "Bergen", "lat": 60.3913, "lon": 5.3221, "radius": 18000},  
    {"name": "Trondheim", "lat": 63.4305, "lon": 10.3951, "radius": 15000},  
    {"name": "Stavanger", "lat": 58.9700, "lon": 5.7331, "radius": 15000},  
    {"name": "Tromsø", "lat": 69.6496, "lon": 18.9560, "radius": 12000},  
    {"name": "Alesund", "lat": 62.4722, "lon": 6.1495, "radius": 12000},  
    {"name": "Kristiansand", "lat": 58.1467, "lon": 7.9956, "radius": 10000},  
    {"name": "Lillehammer", "lat": 61.1153, "lon": 10.4663, "radius": 12000},  
    {"name": "Bodø", "lat": 67.2804, "lon": 14.4049, "radius": 10000},  
    {"name": "Narvik", "lat": 68.4385, "lon": 17.4270, "radius": 15000}  
]

def get_osm_transit_stops(city):
    query = f"""
    [out:json];
    (
      node["highway"="bus_stop"](around:{city["radius"]},{city["lat"]},{city["lon"]});
      node["railway"="station"](around:{city["radius"]},{city["lat"]},{city["lon"]});
      node["public_transport"="platform"](around:{city["radius"]},{city["lat"]},{city["lon"]});
    );
    out center;
    """
    
    response = requests.get(OVERPASS_URL, params={"data": query})
    return response.json().get("elements", []) if response.status_code == 200 else []

all_stops = []
for city in CITIES_NORWAY:
    print(f"📍 Fetching public transport stops in {city['name']}...")
    stops = get_osm_transit_stops(city)

    for stop in stops:
        tags = stop.get("tags", {})

        all_stops.append({
            "city": city["name"],
            "name": tags.get("name", "Unknown"),
            "type": tags.get("highway", tags.get("railway", tags.get("public_transport", "Unknown"))),
            "latitude": stop.get("lat"),
            "longitude": stop.get("lon"),
            "operator": tags.get("operator", "Unknown")
        })

df_transit = pd.DataFrame(all_stops)
df_transit.to_csv("norway_transit_stops.csv", index=False)

print(f"✅ Collected {len(df_transit)} public transport stops across 10 cities in Norway!")


📍 Fetching public transport stops in Oslo...
📍 Fetching public transport stops in Bergen...
📍 Fetching public transport stops in Trondheim...
📍 Fetching public transport stops in Stavanger...
📍 Fetching public transport stops in Tromsø...
📍 Fetching public transport stops in Alesund...
📍 Fetching public transport stops in Kristiansand...
📍 Fetching public transport stops in Lillehammer...
📍 Fetching public transport stops in Bodø...
📍 Fetching public transport stops in Narvik...
✅ Collected 10915 public transport stops across 10 cities in Norway!


### Nature Spots

In [57]:
import requests
import pandas as pd
import time

OVERPASS_URL = "https://overpass-api.de/api/interpreter"

CITIES_NORWAY = [
    {"name": "Oslo", "lat": 59.9139, "lng": 10.7522, "radius": 20000},  
    {"name": "Bergen", "lat": 60.3913, "lng": 5.3221, "radius": 18000},  
    {"name": "Trondheim", "lat": 63.4305, "lng": 10.3951, "radius": 15000},  
    {"name": "Stavanger", "lat": 58.9700, "lng": 5.7331, "radius": 15000},  
    {"name": "Tromsø", "lat": 69.6496, "lng": 18.9560, "radius": 12000},  
    {"name": "Alesund", "lat": 62.4722, "lng": 6.1495, "radius": 12000},  
    {"name": "Kristiansand", "lat": 58.1467, "lng": 7.9956, "radius": 10000},  
    {"name": "Lillehammer", "lat": 61.1153, "lng": 10.4663, "radius": 12000},  
    {"name": "Bodø", "lat": 67.2804, "lng": 14.4049, "radius": 10000},  
    {"name": "Narvik", "lat": 68.4385, "lng": 17.4270, "radius": 15000}  
]

def get_osm_nature_spots(lat, lon):
    query = f"""
    [out:json];
    (
      node["leisure"="nature_reserve"](around:10000,{lat},{lon});
      node["leisure"="park"](around:10000,{lat},{lon});
      node["route"="hiking"](around:10000,{lat},{lon});
      node["piste:type"="downhill"](around:10000,{lat},{lon});
      node["natural"="peak"](around:10000,{lat},{lon});
    );
    out center;
    """
    
    response = requests.get(OVERPASS_URL, params={"data": query})
    return response.json().get("elements", []) if response.status_code == 200 else []

all_nature_spots = []
for city in CITIES_NORWAY:
    print(f"📍 Fetching nature spots in {city['name']}...")
    nature_spots = get_osm_nature_spots(city["lat"], city["lng"])

    for spot in nature_spots:
        tags = spot.get("tags", {})

        all_nature_spots.append({
            "city": city["name"],
            "name": tags.get("name", "Unknown"),
            "type": tags.get("leisure", tags.get("route", tags.get("natural", "Unknown"))),
            "latitude": spot.get("lat"),
            "longitude": spot.get("lon"),
            "elevation": tags.get("ele", "Unknown")
        })
    time.sleep(2)

df_nature = pd.DataFrame(all_nature_spots)
df_nature.to_csv("norway_nature_spots.csv", index=False)

print(f"✅ Collected {len(df_nature)} nature spots across 10 cities in Norway!")


📍 Fetching nature spots in Oslo...
📍 Fetching nature spots in Bergen...
📍 Fetching nature spots in Trondheim...
📍 Fetching nature spots in Stavanger...
📍 Fetching nature spots in Tromsø...
📍 Fetching nature spots in Alesund...
📍 Fetching nature spots in Kristiansand...
📍 Fetching nature spots in Lillehammer...
📍 Fetching nature spots in Bodø...
📍 Fetching nature spots in Narvik...
✅ Collected 557 nature spots across 10 cities in Norway!


### Weather

In [69]:
import os
import requests
import pandas as pd
import time

# Enter your OpenWeather API key
OPENWEATHER_KEY = input("Enter your OpenWeather API key: ")
WEATHER_URL = "https://api.openweathermap.org/data/2.5/weather"

CITIES_NORWAY = [
    {"name": "Oslo", "lat": 59.9139, "lng": 10.7522},  
    {"name": "Bergen", "lat": 60.3913, "lng": 5.3221},  
    {"name": "Trondheim", "lat": 63.4305, "lng": 10.3951},  
    {"name": "Stavanger", "lat": 58.9700, "lng": 5.7331},  
    {"name": "Tromsø", "lat": 69.6496, "lng": 18.9560},  
    {"name": "Alesund", "lat": 62.4722, "lng": 6.1495},  
    {"name": "Kristiansand", "lat": 58.1467, "lng": 7.9956},  
    {"name": "Lillehammer", "lat": 61.1153, "lng": 10.4663},  
    {"name": "Bodø", "lat": 67.2804, "lng": 14.4049},  
    {"name": "Narvik", "lat": 68.4385, "lng": 17.4270}  
]

# Function to fetch weather data based on latitude and longitude
def get_weather(lat, lon):
    params = {
        "lat": lat,
        "lon": lon,
        "appid": OPENWEATHER_KEY,
        "units": "metric"  # Celsius
    }
    response = requests.get(WEATHER_URL, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"❌ Error fetching weather for {lat}, {lon}: {response.status_code}")
        return None

# Collect weather data for Norwegian cities
all_weather_data = []

for city in CITIES_NORWAY:
    print(f"Fetching weather data for {city['name']}...")
    weather_data = get_weather(city["lat"], city["lng"])
    
    if weather_data:
        weather_info = {
            "city": city["name"],
            "latitude": city["lat"],
            "longitude": city["lng"],
            "temperature": weather_data["main"]["temp"],
            "weather": weather_data["weather"][0]["description"],
            "humidity": weather_data["main"]["humidity"],
            "wind_speed": weather_data["wind"]["speed"]
        }
        all_weather_data.append(weather_info)
    time.sleep(1)  # Avoid exceeding API limits

# Create DataFrame and save weather data
df_weather = pd.DataFrame(all_weather_data)
df_weather.to_csv("norway_weather_data.csv", index=False)

print(f"✅ Weather data for {len(df_weather)} Norwegian cities collected and saved!")


Fetching weather data for Oslo...
Fetching weather data for Bergen...
Fetching weather data for Trondheim...
Fetching weather data for Stavanger...
Fetching weather data for Tromsø...
Fetching weather data for Alesund...
Fetching weather data for Kristiansand...
Fetching weather data for Lillehammer...
Fetching weather data for Bodø...
Fetching weather data for Narvik...
✅ Weather data for 10 Norwegian cities collected and saved!


## Denmark

### Tourist Attractions

In [31]:
import requests
import pandas as pd
import time

# 🔑 Insert your Google API Key here
API_KEY = input("Enter your Google API key: ")
PLACES_URL = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# 📍 Top 10 Most Visited Cities in Denmark with Coordinates & Search Radius (in meters)
CITIES_DENMARK = [
    {"name": "Copenhagen", "lat": 55.6761, "lng": 12.5683, "radius": 20000},  
    {"name": "Aarhus", "lat": 56.1629, "lng": 10.2039, "radius": 18000},  
    {"name": "Odense", "lat": 55.4038, "lng": 10.4024, "radius": 15000},  
    {"name": "Aalborg", "lat": 57.0488, "lng": 9.9217, "radius": 15000},  
    {"name": "Esbjerg", "lat": 55.4765, "lng": 8.4594, "radius": 12000},  
    {"name": "Roskilde", "lat": 55.6415, "lng": 12.0803, "radius": 12000},  
    {"name": "Helsingør", "lat": 56.0366, "lng": 12.6136, "radius": 10000},  
    {"name": "Vejle", "lat": 55.7093, "lng": 9.5363, "radius": 12000},  
    {"name": "Herning", "lat": 56.1365, "lng": 8.9784, "radius": 10000},  
    {"name": "Silkeborg", "lat": 56.1697, "lng": 9.5451, "radius": 15000}  
]

# Function to fetch tourist attractions for a given city
def get_places(city_name, lat, lng, radius, type_place="tourist_attraction"):
    params = {
        "location": f"{lat},{lng}",
        "radius": radius,
        "type": type_place,
        "key": API_KEY
    }
    
    places = []
    next_page_token = None

    print(f"📍 Fetching attractions in {city_name}...")
    
    while True:
        if next_page_token:
            params["pagetoken"] = next_page_token
        
        response = requests.get(PLACES_URL, params=params)
        data = response.json()
        
        if response.status_code != 200 or "results" not in data:
            print(f"⚠️ Error fetching data for {city_name}: {data.get('status', 'Unknown error')}")
            break
        
        places.extend(data["results"])
        
        next_page_token = data.get("next_page_token")
        if not next_page_token:
            break
        
        time.sleep(2)  # Google requires a small delay before using next_page_token

    return places

# Collect data for all cities
all_places = []
seen_places = set()

for city in CITIES_DENMARK:
    places = get_places(city["name"], city["lat"], city["lng"], city["radius"])
    
    for p in places:
        place_id = p["place_id"]
        
        if place_id not in seen_places:
            seen_places.add(place_id)
            all_places.append({
                "city": city["name"],
                "place_id": place_id,
                "name": p.get("name", "Unknown"),
                "rating": p.get("rating", "N/A"),
                "address": p.get("vicinity", "Unknown"),
                "latitude": p["geometry"]["location"]["lat"],
                "longitude": p["geometry"]["location"]["lng"],
                "types": ", ".join(p.get("types", []))
            })

    time.sleep(5)  # Avoid excessive API requests

# Save data to CSV
df_places = pd.DataFrame(all_places)
df_places.to_csv("denmark_tourist_attractions.csv", index=False)

print(f"✅ Collected {len(df_places)} tourist attractions across 10 cities in Denmark!")


📍 Fetching attractions in Copenhagen...
📍 Fetching attractions in Aarhus...
📍 Fetching attractions in Odense...
📍 Fetching attractions in Aalborg...
📍 Fetching attractions in Esbjerg...
📍 Fetching attractions in Roskilde...
📍 Fetching attractions in Helsingør...
📍 Fetching attractions in Vejle...
📍 Fetching attractions in Herning...
📍 Fetching attractions in Silkeborg...
✅ Collected 526 tourist attractions across 10 cities in Denmark!


### Events

In [22]:
import requests
import pandas as pd
import time

# 🔑 Solicita a chave da API do usuário
API_KEY = input("Enter your Google API key: ")
PLACES_URL = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# 📍 Top 10 Most Visited Cities in Denmark with Coordinates & Search Radius (in meters)
CITIES_DENMARK = [
    {"name": "Copenhagen", "lat": 55.6761, "lng": 12.5683, "radius": 15000},
    {"name": "Aarhus", "lat": 56.1629, "lng": 10.2039, "radius": 12000},
    {"name": "Odense", "lat": 55.4038, "lng": 10.4024, "radius": 10000},
    {"name": "Aalborg", "lat": 57.0488, "lng": 9.9217, "radius": 10000},
    {"name": "Esbjerg", "lat": 55.4703, "lng": 8.4519, "radius": 8000},
    {"name": "Roskilde", "lat": 55.6415, "lng": 12.0803, "radius": 8000},
    {"name": "Helsingør", "lat": 56.0361, "lng": 12.6136, "radius": 8000},
    {"name": "Vejle", "lat": 55.7093, "lng": 9.5364, "radius": 8000},
    {"name": "Herning", "lat": 56.1393, "lng": 8.9738, "radius": 8000},
    {"name": "Silkeborg", "lat": 56.1697, "lng": 9.5446, "radius": 8000}
]

# 🎭 Define categories of places where events may occur
EVENT_TYPES = [
    "stadium", "casino", "night_club", "art_gallery", "museum",
    "tourist_attraction", "amusement_park", "university", "movie_theater"
]

# Function to fetch event venues for a city
def get_places(city_name, lat, lng, radius):
    places = []
    seen_places = set()

    for place_type in EVENT_TYPES:
        params = {
            "location": f"{lat},{lng}",
            "radius": radius,
            "type": place_type,
            "key": API_KEY
        }
        
        print(f"📍 Fetching event venues in {city_name} ({place_type})...")
        
        response = requests.get(PLACES_URL, params=params)
        data = response.json()

        if response.status_code == 200 and "results" in data:
            for p in data["results"]:
                place_id = p["place_id"]

                if place_id not in seen_places:
                    seen_places.add(place_id)
                    places.append({
                        "city": city_name,
                        "place_id": place_id,
                        "name": p.get("name", "Unknown"),
                        "rating": p.get("rating", "N/A"),
                        "address": p.get("vicinity", "Unknown"),
                        "latitude": p["geometry"]["location"]["lat"],
                        "longitude": p["geometry"]["location"]["lng"],
                        "types": ", ".join(p.get("types", []))
                    })
        
        # Wait a bit to avoid exceeding API request limits
        time.sleep(2)
    
    return places

# 🔍 Collect event venue data for all cities
all_places = []
for city in CITIES_DENMARK:
    all_places.extend(get_places(city["name"], city["lat"], city["lng"], city["radius"]))

# 📝 Save data to CSV
df_places = pd.DataFrame(all_places)
df_places.to_csv("denmark_event_locations.csv", index=False)

print(f"✅ Collected {len(df_places)} event venues across 10 cities in Denmark! Saved as 'denmark_event_locations.csv'")


📍 Fetching event venues in Copenhagen (stadium)...
📍 Fetching event venues in Copenhagen (casino)...
📍 Fetching event venues in Copenhagen (night_club)...
📍 Fetching event venues in Copenhagen (art_gallery)...
📍 Fetching event venues in Copenhagen (museum)...
📍 Fetching event venues in Copenhagen (tourist_attraction)...
📍 Fetching event venues in Copenhagen (amusement_park)...
📍 Fetching event venues in Copenhagen (university)...
📍 Fetching event venues in Copenhagen (movie_theater)...
📍 Fetching event venues in Aarhus (stadium)...
📍 Fetching event venues in Aarhus (casino)...
📍 Fetching event venues in Aarhus (night_club)...
📍 Fetching event venues in Aarhus (art_gallery)...
📍 Fetching event venues in Aarhus (museum)...
📍 Fetching event venues in Aarhus (tourist_attraction)...
📍 Fetching event venues in Aarhus (amusement_park)...
📍 Fetching event venues in Aarhus (university)...
📍 Fetching event venues in Aarhus (movie_theater)...
📍 Fetching event venues in Odense (stadium)...
📍 Fetch

### Hotels

In [48]:
import requests 
import pandas as pd
import time

OVERPASS_URL = "https://overpass-api.de/api/interpreter"

CITIES_DENMARK = [
    {"name": "Copenhagen", "lat": 55.6761, "lng": 12.5683, "radius": 20000},  
    {"name": "Aarhus", "lat": 56.1629, "lng": 10.2039, "radius": 18000},  
    {"name": "Odense", "lat": 55.4038, "lng": 10.4024, "radius": 15000},  
    {"name": "Aalborg", "lat": 57.0488, "lng": 9.9217, "radius": 15000},  
    {"name": "Esbjerg", "lat": 55.4765, "lng": 8.4594, "radius": 12000},  
    {"name": "Roskilde", "lat": 55.6415, "lng": 12.0803, "radius": 12000},  
    {"name": "Helsingør", "lat": 56.0366, "lng": 12.6136, "radius": 10000},  
    {"name": "Vejle", "lat": 55.7093, "lng": 9.5363, "radius": 12000},  
    {"name": "Herning", "lat": 56.1365, "lng": 8.9784, "radius": 10000},  
    {"name": "Silkeborg", "lat": 56.1697, "lng": 9.5451, "radius": 15000}  
]

def get_osm_hotels(lat, lon):
    query = f"""
    [out:json];
    (
      node["tourism"="hotel"](around:10000,{lat},{lon});
      node["tourism"="hostel"](around:10000,{lat},{lon});
      node["tourism"="guest_house"](around:10000,{lat},{lon});
    );
    out center;
    """
    
    response = requests.get(OVERPASS_URL, params={"data": query})
    return response.json().get("elements", []) if response.status_code == 200 else []

all_hotels = []
for city in CITIES_DENMARK:
    print(f"📍 Fetching hotels and hostels in {city['name']}...")
    hotels = get_osm_hotels(city["lat"], city["lng"])

    for hotel in hotels:
        tags = hotel.get("tags", {})

        all_hotels.append({
            "city": city["name"],
            "name": tags.get("name", "Unknown"),
            "type": tags.get("tourism", "Unknown"),
            "latitude": hotel.get("lat"),
            "longitude": hotel.get("lon"),
            "stars": tags.get("stars", "Unknown"),
            "operator": tags.get("operator", "Unknown")
        })
    time.sleep(2)

df_hotels = pd.DataFrame(all_hotels)
df_hotels.to_csv("denmark_hotels_hostels.csv", index=False)

print(f"✅ Collected {len(df_hotels)} hotels and hostels across 10 cities in Denmark!")

📍 Fetching hotels and hostels in Copenhagen...
📍 Fetching hotels and hostels in Aarhus...
📍 Fetching hotels and hostels in Odense...
📍 Fetching hotels and hostels in Aalborg...
📍 Fetching hotels and hostels in Esbjerg...
📍 Fetching hotels and hostels in Roskilde...
📍 Fetching hotels and hostels in Helsingør...
📍 Fetching hotels and hostels in Vejle...
📍 Fetching hotels and hostels in Herning...
📍 Fetching hotels and hostels in Silkeborg...
✅ Collected 255 hotels and hostels across 10 cities in Denmark!


### Restaurants

In [23]:
import requests
import pandas as pd
import time

OVERPASS_URL = "https://overpass-api.de/api/interpreter"

# 📍 Top 10 Most Visited Cities in Denmark
CITIES_DENMARK = [
    {"name": "Copenhagen", "lat": 55.6761, "lon": 12.5683, "radius": 10000},
    {"name": "Aarhus", "lat": 56.1629, "lon": 10.2039, "radius": 8000},
    {"name": "Odense", "lat": 55.4038, "lon": 10.4024, "radius": 7000},
    {"name": "Aalborg", "lat": 57.0488, "lon": 9.9217, "radius": 7000},
    {"name": "Esbjerg", "lat": 55.4671, "lon": 8.4496, "radius": 7000},
    {"name": "Roskilde", "lat": 55.6415, "lon": 12.0803, "radius": 6000},
    {"name": "Helsingør", "lat": 56.0361, "lon": 12.6136, "radius": 6000},
    {"name": "Vejle", "lat": 55.7093, "lon": 9.5357, "radius": 6000},
    {"name": "Herning", "lat": 56.1368, "lon": 8.9766, "radius": 6000},
    {"name": "Silkeborg", "lat": 56.1697, "lon": 9.5451, "radius": 6000}
]

def get_osm_restaurants(city):
    """Fetches restaurants, cafés, and bars in a specific Danish city with retries."""
    query = f"""
    [out:json];
    (
      node["amenity"="restaurant"](around:{city["radius"]},{city["lat"]},{city["lon"]});
      node["amenity"="cafe"](around:{city["radius"]},{city["lat"]},{city["lon"]});
      node["amenity"="bar"](around:{city["radius"]},{city["lat"]},{city["lon"]});
    );
    out center;
    """

    retries = 3
    while retries > 0:
        response = requests.get(OVERPASS_URL, params={"data": query})
        
        if response.status_code == 200:
            return response.json().get("elements", [])
        else:
            print(f"⚠️ Error fetching restaurants in {city['name']} ({response.status_code}), retrying... ({3 - retries + 1}/3)")
            retries -= 1
            time.sleep(3)  # Aguardar antes de tentar novamente
    
    print(f"❌ Failed to fetch data for {city['name']} after multiple retries.")
    return []

# 🔍 Collect data for all cities
all_places = []
for city in CITIES_DENMARK:
    print(f"📍 Fetching gastronomic locations in {city['name']}...")
    places = get_osm_restaurants(city)

    for place in places:
        tags = place.get("tags", {})

        all_places.append({
            "city": city["name"],
            "name": tags.get("name", "Unknown"),
            "type": tags.get("amenity", "Unknown"),
            "latitude": place.get("lat"),
            "longitude": place.get("lon"),
            "street": tags.get("addr:street", "Unknown"),
            "postcode": tags.get("addr:postcode", "Unknown"),
            "cuisine": tags.get("cuisine", "Not specified")  # Tipo de comida
        })

    time.sleep(2)  # Evitar sobrecarga na API

# 📝 Save data to CSV
df = pd.DataFrame(all_places)
df.to_csv("denmark_all_restaurants_bars_cafes.csv", index=False)

print(f"✅ Collected {len(df)} gastronomic locations across 10 cities in Denmark! Saved as 'denmark_all_restaurants_bars_cafes.csv'")


📍 Fetching gastronomic locations in Copenhagen...
📍 Fetching gastronomic locations in Aarhus...
📍 Fetching gastronomic locations in Odense...
📍 Fetching gastronomic locations in Aalborg...
📍 Fetching gastronomic locations in Esbjerg...
📍 Fetching gastronomic locations in Roskilde...
📍 Fetching gastronomic locations in Helsingør...
📍 Fetching gastronomic locations in Vejle...
📍 Fetching gastronomic locations in Herning...
📍 Fetching gastronomic locations in Silkeborg...
✅ Collected 3997 gastronomic locations across 10 cities in Denmark! Saved as 'denmark_all_restaurants_bars_cafes.csv'


### Public Transport Stops

In [35]:
import requests
import pandas as pd

OVERPASS_URL = "https://overpass-api.de/api/interpreter"

CITIES_DENMARK = [
    {"name": "Copenhagen", "lat": 55.6761, "lon": 12.5683, "radius": 10000},
    {"name": "Aarhus", "lat": 56.1629, "lon": 10.2039, "radius": 8000},
    {"name": "Odense", "lat": 55.4038, "lon": 10.4024, "radius": 7000},
    {"name": "Aalborg", "lat": 57.0488, "lon": 9.9217, "radius": 7000},
    {"name": "Esbjerg", "lat": 55.4671, "lon": 8.4496, "radius": 7000},
    {"name": "Roskilde", "lat": 55.6415, "lon": 12.0803, "radius": 6000},
    {"name": "Helsingør", "lat": 56.0361, "lon": 12.6136, "radius": 6000},
    {"name": "Vejle", "lat": 55.7093, "lon": 9.5357, "radius": 6000},
    {"name": "Herning", "lat": 56.1368, "lon": 8.9766, "radius": 6000},
    {"name": "Silkeborg", "lat": 56.1697, "lon": 9.5451, "radius": 6000}
]

def get_osm_transit_stops(city):
    query = f"""
    [out:json];
    (
      node["highway"="bus_stop"](around:{city["radius"]},{city["lat"]},{city["lon"]});
      node["railway"="station"](around:{city["radius"]},{city["lat"]},{city["lon"]});
      node["public_transport"="platform"](around:{city["radius"]},{city["lat"]},{city["lon"]});
    );
    out center;
    """
    
    response = requests.get(OVERPASS_URL, params={"data": query})
    return response.json().get("elements", []) if response.status_code == 200 else []

all_stops = []
for city in CITIES_DENMARK:
    print(f"📍 Fetching public transport stops in {city['name']}...")
    stops = get_osm_transit_stops(city)

    for stop in stops:
        tags = stop.get("tags", {})

        all_stops.append({
            "city": city["name"],
            "name": tags.get("name", "Unknown"),
            "type": tags.get("highway", tags.get("railway", tags.get("public_transport", "Unknown"))),
            "latitude": stop.get("lat"),
            "longitude": stop.get("lon"),
            "operator": tags.get("operator", "Unknown")
        })

df_transit = pd.DataFrame(all_stops)
df_transit.to_csv("denmark_transit_stops.csv", index=False)

print(f"✅ Collected {len(df_transit)} public transport stops across 10 cities in Denmark!")


📍 Fetching public transport stops in Copenhagen...
📍 Fetching public transport stops in Aarhus...
📍 Fetching public transport stops in Odense...
📍 Fetching public transport stops in Aalborg...
📍 Fetching public transport stops in Esbjerg...
📍 Fetching public transport stops in Roskilde...
📍 Fetching public transport stops in Helsingør...
📍 Fetching public transport stops in Vejle...
📍 Fetching public transport stops in Herning...
📍 Fetching public transport stops in Silkeborg...
✅ Collected 5697 public transport stops across 10 cities in Denmark!


### Nature Spots

In [58]:
import requests
import pandas as pd
import time

OVERPASS_URL = "https://overpass-api.de/api/interpreter"

CITIES_DENMARK = [
    {"name": "Copenhagen", "lat": 55.6761, "lng": 12.5683, "radius": 20000},  
    {"name": "Aarhus", "lat": 56.1629, "lng": 10.2039, "radius": 18000},  
    {"name": "Odense", "lat": 55.4038, "lng": 10.4024, "radius": 15000},  
    {"name": "Aalborg", "lat": 57.0488, "lng": 9.9217, "radius": 15000},  
    {"name": "Esbjerg", "lat": 55.4765, "lng": 8.4594, "radius": 12000},  
    {"name": "Roskilde", "lat": 55.6415, "lng": 12.0803, "radius": 12000},  
    {"name": "Helsingør", "lat": 56.0366, "lng": 12.6136, "radius": 10000},  
    {"name": "Vejle", "lat": 55.7093, "lng": 9.5363, "radius": 12000},  
    {"name": "Herning", "lat": 56.1365, "lng": 8.9784, "radius": 10000},  
    {"name": "Silkeborg", "lat": 56.1697, "lng": 9.5451, "radius": 15000}  
]

def get_osm_nature_spots(lat, lon):
    query = f"""
    [out:json];
    (
      node["leisure"="nature_reserve"](around:10000,{lat},{lon});
      node["leisure"="park"](around:10000,{lat},{lon});
      node["route"="hiking"](around:10000,{lat},{lon});
      node["piste:type"="downhill"](around:10000,{lat},{lon});
      node["natural"="peak"](around:10000,{lat},{lon});
    );
    out center;
    """
    
    response = requests.get(OVERPASS_URL, params={"data": query})
    return response.json().get("elements", []) if response.status_code == 200 else []

all_nature_spots = []
for city in CITIES_DENMARK:
    print(f"📍 Fetching nature spots in {city['name']}...")
    nature_spots = get_osm_nature_spots(city["lat"], city["lng"])

    for spot in nature_spots:
        tags = spot.get("tags", {})

        all_nature_spots.append({
            "city": city["name"],
            "name": tags.get("name", "Unknown"),
            "type": tags.get("leisure", tags.get("route", tags.get("natural", "Unknown"))),
            "latitude": spot.get("lat"),
            "longitude": spot.get("lon"),
            "elevation": tags.get("ele", "Unknown")
        })
    time.sleep(2)

df_nature = pd.DataFrame(all_nature_spots)
df_nature.to_csv("denmark_nature_spots.csv", index=False)

print(f"✅ Collected {len(df_nature)} nature spots across 10 cities in Denmark!")

📍 Fetching nature spots in Copenhagen...
📍 Fetching nature spots in Aarhus...
📍 Fetching nature spots in Odense...
📍 Fetching nature spots in Aalborg...
📍 Fetching nature spots in Esbjerg...
📍 Fetching nature spots in Roskilde...
📍 Fetching nature spots in Helsingør...
📍 Fetching nature spots in Vejle...
📍 Fetching nature spots in Herning...
📍 Fetching nature spots in Silkeborg...
✅ Collected 123 nature spots across 10 cities in Denmark!


### Weather

In [70]:
import os
import requests
import pandas as pd
import time

# Enter your OpenWeather API key
OPENWEATHER_KEY = input("Enter your OpenWeather API key: ")
WEATHER_URL = "https://api.openweathermap.org/data/2.5/weather"

CITIES_DENMARK = [
    {"name": "Copenhagen", "lat": 55.6761, "lng": 12.5683},  
    {"name": "Aarhus", "lat": 56.1629, "lng": 10.2039},  
    {"name": "Odense", "lat": 55.4038, "lng": 10.4024},  
    {"name": "Aalborg", "lat": 57.0488, "lng": 9.9217},  
    {"name": "Esbjerg", "lat": 55.4765, "lng": 8.4594},  
    {"name": "Roskilde", "lat": 55.6415, "lng": 12.0803},  
    {"name": "Helsingør", "lat": 56.0366, "lng": 12.6136},  
    {"name": "Vejle", "lat": 55.7093, "lng": 9.5363},  
    {"name": "Herning", "lat": 56.1365, "lng": 8.9784},  
    {"name": "Silkeborg", "lat": 56.1697, "lng": 9.5451}  
]

# Function to fetch weather data based on latitude and longitude
def get_weather(lat, lon):
    params = {
        "lat": lat,
        "lon": lon,
        "appid": OPENWEATHER_KEY,
        "units": "metric"  # Celsius
    }
    response = requests.get(WEATHER_URL, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"❌ Error fetching weather for {lat}, {lon}: {response.status_code}")
        return None

# Collect weather data for Danish cities
all_weather_data = []

for city in CITIES_DENMARK:
    print(f"Fetching weather data for {city['name']}...")
    weather_data = get_weather(city["lat"], city["lng"])
    
    if weather_data:
        weather_info = {
            "city": city["name"],
            "latitude": city["lat"],
            "longitude": city["lng"],
            "temperature": weather_data["main"]["temp"],
            "weather": weather_data["weather"][0]["description"],
            "humidity": weather_data["main"]["humidity"],
            "wind_speed": weather_data["wind"]["speed"]
        }
        all_weather_data.append(weather_info)
    time.sleep(1)  # Avoid exceeding API limits

# Create DataFrame and save weather data
df_weather = pd.DataFrame(all_weather_data)
df_weather.to_csv("denmark_weather_data.csv", index=False)

print(f"✅ Weather data for {len(df_weather)} Danish cities collected and saved!")


Fetching weather data for Copenhagen...
Fetching weather data for Aarhus...
Fetching weather data for Odense...
Fetching weather data for Aalborg...
Fetching weather data for Esbjerg...
Fetching weather data for Roskilde...
Fetching weather data for Helsingør...
Fetching weather data for Vejle...
Fetching weather data for Herning...
Fetching weather data for Silkeborg...
✅ Weather data for 10 Danish cities collected and saved!


## France

### Tourist Attractions

In [32]:
import requests
import pandas as pd
import time

# 🔑 Insert your Google API Key here
API_KEY = input("Enter your Google API key: ")
PLACES_URL = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# 📍 Top 10 Most Visited Cities in France with Coordinates & Search Radius (in meters)
CITIES_FRANCE = [
    {"name": "Paris", "lat": 48.8566, "lng": 2.3522, "radius": 20000},  
    {"name": "Nice", "lat": 43.7102, "lng": 7.2620, "radius": 18000},  
    {"name": "Lyon", "lat": 45.7640, "lng": 4.8357, "radius": 15000},  
    {"name": "Marseille", "lat": 43.2965, "lng": 5.3698, "radius": 15000},  
    {"name": "Toulouse", "lat": 43.6047, "lng": 1.4442, "radius": 12000},  
    {"name": "Bordeaux", "lat": 44.8378, "lng": -0.5792, "radius": 12000},  
    {"name": "Lille", "lat": 50.6292, "lng": 3.0573, "radius": 10000},  
    {"name": "Strasbourg", "lat": 48.5734, "lng": 7.7521, "radius": 12000},  
    {"name": "Montpellier", "lat": 43.6108, "lng": 3.8767, "radius": 10000},  
    {"name": "Cannes", "lat": 43.5528, "lng": 7.0174, "radius": 15000}  
]

# Function to fetch tourist attractions for a given city
def get_places(city_name, lat, lng, radius, type_place="tourist_attraction"):
    params = {
        "location": f"{lat},{lng}",
        "radius": radius,
        "type": type_place,
        "key": API_KEY
    }
    
    places = []
    next_page_token = None

    print(f"📍 Fetching attractions in {city_name}...")
    
    while True:
        if next_page_token:
            params["pagetoken"] = next_page_token
        
        response = requests.get(PLACES_URL, params=params)
        data = response.json()
        
        if response.status_code != 200 or "results" not in data:
            print(f"⚠️ Error fetching data for {city_name}: {data.get('status', 'Unknown error')}")
            break
        
        places.extend(data["results"])
        
        next_page_token = data.get("next_page_token")
        if not next_page_token:
            break
        
        time.sleep(2)  # Google requires a small delay before using next_page_token

    return places

# Collect data for all cities
all_places = []
seen_places = set()

for city in CITIES_FRANCE:
    places = get_places(city["name"], city["lat"], city["lng"], city["radius"])
    
    for p in places:
        place_id = p["place_id"]
        
        if place_id not in seen_places:
            seen_places.add(place_id)
            all_places.append({
                "city": city["name"],
                "place_id": place_id,
                "name": p.get("name", "Unknown"),
                "rating": p.get("rating", "N/A"),
                "address": p.get("vicinity", "Unknown"),
                "latitude": p["geometry"]["location"]["lat"],
                "longitude": p["geometry"]["location"]["lng"],
                "types": ", ".join(p.get("types", []))
            })

    time.sleep(5)  # Avoid excessive API requests

# Save data to CSV
df_places = pd.DataFrame(all_places)
df_places.to_csv("france_tourist_attractions.csv", index=False)

print(f"✅ Collected {len(df_places)} tourist attractions across 10 cities in France!")


📍 Fetching attractions in Paris...
📍 Fetching attractions in Nice...
📍 Fetching attractions in Lyon...
📍 Fetching attractions in Marseille...
📍 Fetching attractions in Toulouse...
📍 Fetching attractions in Bordeaux...
📍 Fetching attractions in Lille...
📍 Fetching attractions in Strasbourg...
📍 Fetching attractions in Montpellier...
📍 Fetching attractions in Cannes...
✅ Collected 595 tourist attractions across 10 cities in France!


### Events

In [24]:
import requests
import pandas as pd
import time

# 🔑 Solicita a chave da API do usuário
API_KEY = input("Enter your Google API key: ")
PLACES_URL = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# 📍 Top 10 Most Visited Cities in France with Coordinates & Search Radius (in meters)
CITIES_FRANCE = [
    {"name": "Paris", "lat": 48.8566, "lng": 2.3522, "radius": 20000},
    {"name": "Nice", "lat": 43.7102, "lng": 7.2620, "radius": 15000},
    {"name": "Lyon", "lat": 45.7640, "lng": 4.8357, "radius": 15000},
    {"name": "Marseille", "lat": 43.2965, "lng": 5.3698, "radius": 15000},
    {"name": "Toulouse", "lat": 43.6045, "lng": 1.4442, "radius": 12000},
    {"name": "Bordeaux", "lat": 44.8378, "lng": -0.5792, "radius": 12000},
    {"name": "Lille", "lat": 50.6292, "lng": 3.0573, "radius": 10000},
    {"name": "Strasbourg", "lat": 48.5734, "lng": 7.7521, "radius": 10000},
    {"name": "Montpellier", "lat": 43.6108, "lng": 3.8767, "radius": 10000},
    {"name": "Cannes", "lat": 43.5528, "lng": 7.0174, "radius": 8000}
]

# 🎭 Define categories of places where events may occur
EVENT_TYPES = [
    "stadium", "casino", "night_club", "art_gallery", "museum",
    "tourist_attraction", "amusement_park", "university", "movie_theater"
]

# Function to fetch event venues for a city
def get_places(city_name, lat, lng, radius):
    places = []
    seen_places = set()

    for place_type in EVENT_TYPES:
        params = {
            "location": f"{lat},{lng}",
            "radius": radius,
            "type": place_type,
            "key": API_KEY
        }
        
        print(f"📍 Fetching event venues in {city_name} ({place_type})...")
        
        response = requests.get(PLACES_URL, params=params)
        data = response.json()

        if response.status_code == 200 and "results" in data:
            for p in data["results"]:
                place_id = p["place_id"]

                if place_id not in seen_places:
                    seen_places.add(place_id)
                    places.append({
                        "city": city_name,
                        "place_id": place_id,
                        "name": p.get("name", "Unknown"),
                        "rating": p.get("rating", "N/A"),
                        "address": p.get("vicinity", "Unknown"),
                        "latitude": p["geometry"]["location"]["lat"],
                        "longitude": p["geometry"]["location"]["lng"],
                        "types": ", ".join(p.get("types", []))
                    })
        
        # Wait a bit to avoid exceeding API request limits
        time.sleep(2)
    
    return places

# 🔍 Collect event venue data for all cities
all_places = []
for city in CITIES_FRANCE:
    all_places.extend(get_places(city["name"], city["lat"], city["lng"], city["radius"]))

# 📝 Save data to CSV
df_places = pd.DataFrame(all_places)
df_places.to_csv("france_event_locations.csv", index=False)

print(f"✅ Collected {len(df_places)} event venues across 10 cities in France! Saved as 'france_event_locations.csv'")


📍 Fetching event venues in Paris (stadium)...
📍 Fetching event venues in Paris (casino)...
📍 Fetching event venues in Paris (night_club)...
📍 Fetching event venues in Paris (art_gallery)...
📍 Fetching event venues in Paris (museum)...
📍 Fetching event venues in Paris (tourist_attraction)...
📍 Fetching event venues in Paris (amusement_park)...
📍 Fetching event venues in Paris (university)...
📍 Fetching event venues in Paris (movie_theater)...
📍 Fetching event venues in Nice (stadium)...
📍 Fetching event venues in Nice (casino)...
📍 Fetching event venues in Nice (night_club)...
📍 Fetching event venues in Nice (art_gallery)...
📍 Fetching event venues in Nice (museum)...
📍 Fetching event venues in Nice (tourist_attraction)...
📍 Fetching event venues in Nice (amusement_park)...
📍 Fetching event venues in Nice (university)...
📍 Fetching event venues in Nice (movie_theater)...
📍 Fetching event venues in Lyon (stadium)...
📍 Fetching event venues in Lyon (casino)...
📍 Fetching event venues in L

### Hotels

In [49]:
import requests 
import pandas as pd
import time

OVERPASS_URL = "https://overpass-api.de/api/interpreter"

CITIES_FRANCE = [
    {"name": "Paris", "lat": 48.8566, "lng": 2.3522, "radius": 20000},  
    {"name": "Nice", "lat": 43.7102, "lng": 7.2620, "radius": 18000},  
    {"name": "Lyon", "lat": 45.7640, "lng": 4.8357, "radius": 15000},  
    {"name": "Marseille", "lat": 43.2965, "lng": 5.3698, "radius": 15000},  
    {"name": "Toulouse", "lat": 43.6047, "lng": 1.4442, "radius": 12000},  
    {"name": "Bordeaux", "lat": 44.8378, "lng": -0.5792, "radius": 12000},  
    {"name": "Lille", "lat": 50.6292, "lng": 3.0573, "radius": 10000},  
    {"name": "Strasbourg", "lat": 48.5734, "lng": 7.7521, "radius": 12000},  
    {"name": "Montpellier", "lat": 43.6108, "lng": 3.8767, "radius": 10000},  
    {"name": "Cannes", "lat": 43.5528, "lng": 7.0174, "radius": 15000}  
]

def get_osm_hotels(lat, lon):
    query = f"""
    [out:json];
    (
      node["tourism"="hotel"](around:10000,{lat},{lon});
      node["tourism"="hostel"](around:10000,{lat},{lon});
      node["tourism"="guest_house"](around:10000,{lat},{lon});
    );
    out center;
    """
    
    response = requests.get(OVERPASS_URL, params={"data": query})
    return response.json().get("elements", []) if response.status_code == 200 else []

all_hotels = []
for city in CITIES_FRANCE:
    print(f"📍 Fetching hotels and hostels in {city['name']}...")
    hotels = get_osm_hotels(city["lat"], city["lng"])

    for hotel in hotels:
        tags = hotel.get("tags", {})

        all_hotels.append({
            "city": city["name"],
            "name": tags.get("name", "Unknown"),
            "type": tags.get("tourism", "Unknown"),
            "latitude": hotel.get("lat"),
            "longitude": hotel.get("lon"),
            "stars": tags.get("stars", "Unknown"),
            "operator": tags.get("operator", "Unknown")
        })
    time.sleep(2)

df_hotels = pd.DataFrame(all_hotels)
df_hotels.to_csv("france_hotels_hostels.csv", index=False)

print(f"✅ Collected {len(df_hotels)} hotels and hostels across 10 cities in France!")


📍 Fetching hotels and hostels in Paris...
📍 Fetching hotels and hostels in Nice...
📍 Fetching hotels and hostels in Lyon...
📍 Fetching hotels and hostels in Marseille...
📍 Fetching hotels and hostels in Toulouse...
📍 Fetching hotels and hostels in Bordeaux...
📍 Fetching hotels and hostels in Lille...
📍 Fetching hotels and hostels in Strasbourg...
📍 Fetching hotels and hostels in Montpellier...
📍 Fetching hotels and hostels in Cannes...
✅ Collected 2530 hotels and hostels across 10 cities in France!


### Restaurants

In [25]:
import requests
import pandas as pd
import time

OVERPASS_URL = "https://overpass-api.de/api/interpreter"

# 📍 Top 10 Most Visited Cities in France
CITIES_FRANCE = [
    {"name": "Paris", "lat": 48.8566, "lon": 2.3522, "radius": 10000},
    {"name": "Nice", "lat": 43.7102, "lon": 7.2620, "radius": 8000},
    {"name": "Lyon", "lat": 45.7640, "lon": 4.8357, "radius": 8000},
    {"name": "Marseille", "lat": 43.2965, "lon": 5.3698, "radius": 8000},
    {"name": "Toulouse", "lat": 43.6047, "lon": 1.4442, "radius": 8000},
    {"name": "Bordeaux", "lat": 44.8378, "lon": -0.5792, "radius": 7000},
    {"name": "Lille", "lat": 50.6292, "lon": 3.0573, "radius": 7000},
    {"name": "Strasbourg", "lat": 48.5734, "lon": 7.7521, "radius": 7000},
    {"name": "Montpellier", "lat": 43.6119, "lon": 3.8772, "radius": 7000},
    {"name": "Cannes", "lat": 43.5528, "lon": 7.0174, "radius": 6000}
]

def get_osm_restaurants(city):
    """Fetches restaurants, cafés, and bars in a specific French city with retries."""
    query = f"""
    [out:json];
    (
      node["amenity"="restaurant"](around:{city["radius"]},{city["lat"]},{city["lon"]});
      node["amenity"="cafe"](around:{city["radius"]},{city["lat"]},{city["lon"]});
      node["amenity"="bar"](around:{city["radius"]},{city["lat"]},{city["lon"]});
    );
    out center;
    """

    retries = 3
    while retries > 0:
        response = requests.get(OVERPASS_URL, params={"data": query})
        
        if response.status_code == 200:
            return response.json().get("elements", [])
        else:
            print(f"⚠️ Error fetching restaurants in {city['name']} ({response.status_code}), retrying... ({3 - retries + 1}/3)")
            retries -= 1
            time.sleep(3)  # Aguardar antes de tentar novamente
    
    print(f"❌ Failed to fetch data for {city['name']} after multiple retries.")
    return []

# 🔍 Collect data for all cities
all_places = []
for city in CITIES_FRANCE:
    print(f"📍 Fetching gastronomic locations in {city['name']}...")
    places = get_osm_restaurants(city)

    for place in places:
        tags = place.get("tags", {})

        all_places.append({
            "city": city["name"],
            "name": tags.get("name", "Unknown"),
            "type": tags.get("amenity", "Unknown"),
            "latitude": place.get("lat"),
            "longitude": place.get("lon"),
            "street": tags.get("addr:street", "Unknown"),
            "postcode": tags.get("addr:postcode", "Unknown"),
            "cuisine": tags.get("cuisine", "Not specified")  # Tipo de comida
        })

    time.sleep(2)  # Evitar sobrecarga na API

# 📝 Save data to CSV
df = pd.DataFrame(all_places)
df.to_csv("france_all_restaurants_bars_cafes.csv", index=False)

print(f"✅ Collected {len(df)} gastronomic locations across 10 cities in France! Saved as 'france_all_restaurants_bars_cafes.csv'")


📍 Fetching gastronomic locations in Paris...
📍 Fetching gastronomic locations in Nice...
📍 Fetching gastronomic locations in Lyon...
⚠️ Error fetching restaurants in Lyon (504), retrying... (1/3)
📍 Fetching gastronomic locations in Marseille...
📍 Fetching gastronomic locations in Toulouse...
📍 Fetching gastronomic locations in Bordeaux...
📍 Fetching gastronomic locations in Lille...
📍 Fetching gastronomic locations in Strasbourg...
📍 Fetching gastronomic locations in Montpellier...
📍 Fetching gastronomic locations in Cannes...
✅ Collected 27730 gastronomic locations across 10 cities in France! Saved as 'france_all_restaurants_bars_cafes.csv'


### Public Transport Stops

In [36]:
import requests
import pandas as pd

OVERPASS_URL = "https://overpass-api.de/api/interpreter"

CITIES_FRANCE = [
    {"name": "Paris", "lat": 48.8566, "lon": 2.3522, "radius": 10000},
    {"name": "Nice", "lat": 43.7102, "lon": 7.2620, "radius": 8000},
    {"name": "Lyon", "lat": 45.7640, "lon": 4.8357, "radius": 8000},
    {"name": "Marseille", "lat": 43.2965, "lon": 5.3698, "radius": 8000},
    {"name": "Toulouse", "lat": 43.6047, "lon": 1.4442, "radius": 8000},
    {"name": "Bordeaux", "lat": 44.8378, "lon": -0.5792, "radius": 7000},
    {"name": "Lille", "lat": 50.6292, "lon": 3.0573, "radius": 7000},
    {"name": "Strasbourg", "lat": 48.5734, "lon": 7.7521, "radius": 7000},
    {"name": "Montpellier", "lat": 43.6119, "lon": 3.8772, "radius": 7000},
    {"name": "Cannes", "lat": 43.5528, "lon": 7.0174, "radius": 6000}
]

def get_osm_transit_stops(city):
    query = f"""
    [out:json];
    (
      node["highway"="bus_stop"](around:{city["radius"]},{city["lat"]},{city["lon"]});
      node["railway"="station"](around:{city["radius"]},{city["lat"]},{city["lon"]});
      node["public_transport"="platform"](around:{city["radius"]},{city["lat"]},{city["lon"]});
    );
    out center;
    """
    
    response = requests.get(OVERPASS_URL, params={"data": query})
    return response.json().get("elements", []) if response.status_code == 200 else []

all_stops = []
for city in CITIES_FRANCE:
    print(f"📍 Fetching public transport stops in {city['name']}...")
    stops = get_osm_transit_stops(city)

    for stop in stops:
        tags = stop.get("tags", {})

        all_stops.append({
            "city": city["name"],
            "name": tags.get("name", "Unknown"),
            "type": tags.get("highway", tags.get("railway", tags.get("public_transport", "Unknown"))),
            "latitude": stop.get("lat"),
            "longitude": stop.get("lon"),
            "operator": tags.get("operator", "Unknown")
        })

df_transit = pd.DataFrame(all_stops)
df_transit.to_csv("france_transit_stops.csv", index=False)

print(f"✅ Collected {len(df_transit)} public transport stops across 10 cities in France!")


📍 Fetching public transport stops in Paris...
📍 Fetching public transport stops in Nice...
📍 Fetching public transport stops in Lyon...
📍 Fetching public transport stops in Marseille...
📍 Fetching public transport stops in Toulouse...
📍 Fetching public transport stops in Bordeaux...
📍 Fetching public transport stops in Lille...
📍 Fetching public transport stops in Strasbourg...
📍 Fetching public transport stops in Montpellier...
📍 Fetching public transport stops in Cannes...
✅ Collected 18141 public transport stops across 10 cities in France!


### Nature Spots

In [60]:
import requests
import pandas as pd
import time

OVERPASS_URL = "https://overpass-api.de/api/interpreter"

CITIES_FRANCE = [
    {"name": "Paris", "lat": 48.8566, "lng": 2.3522, "radius": 20000},  
    {"name": "Nice", "lat": 43.7102, "lng": 7.2620, "radius": 18000},  
    {"name": "Lyon", "lat": 45.7640, "lng": 4.8357, "radius": 15000},  
    {"name": "Marseille", "lat": 43.2965, "lng": 5.3698, "radius": 15000},  
    {"name": "Toulouse", "lat": 43.6047, "lng": 1.4442, "radius": 12000},  
    {"name": "Bordeaux", "lat": 44.8378, "lng": -0.5792, "radius": 12000},  
    {"name": "Lille", "lat": 50.6292, "lng": 3.0573, "radius": 10000},  
    {"name": "Strasbourg", "lat": 48.5734, "lng": 7.7521, "radius": 12000},  
    {"name": "Montpellier", "lat": 43.6108, "lng": 3.8767, "radius": 10000},  
    {"name": "Cannes", "lat": 43.5528, "lng": 7.0174, "radius": 15000}  
]

def get_osm_nature_spots(lat, lon):
    query = f"""
    [out:json];
    (
      node["leisure"="nature_reserve"](around:10000,{lat},{lon});
      node["leisure"="park"](around:10000,{lat},{lon});
      node["route"="hiking"](around:10000,{lat},{lon});
      node["piste:type"="downhill"](around:10000,{lat},{lon});
      node["natural"="peak"](around:10000,{lat},{lon});
    );
    out center;
    """
    
    response = requests.get(OVERPASS_URL, params={"data": query})
    return response.json().get("elements", []) if response.status_code == 200 else []

all_nature_spots = []
for city in CITIES_FRANCE:
    print(f"📍 Fetching nature spots in {city['name']}...")
    nature_spots = get_osm_nature_spots(city["lat"], city["lng"])

    for spot in nature_spots:
        tags = spot.get("tags", {})

        all_nature_spots.append({
            "city": city["name"],
            "name": tags.get("name", "Unknown"),
            "type": tags.get("leisure", tags.get("route", tags.get("natural", "Unknown"))),
            "latitude": spot.get("lat"),
            "longitude": spot.get("lon"),
            "elevation": tags.get("ele", "Unknown")
        })
    time.sleep(2)

df_nature = pd.DataFrame(all_nature_spots)
df_nature.to_csv("france_nature_spots.csv", index=False)

print(f"✅ Collected {len(df_nature)} nature spots across 10 cities in France!")

📍 Fetching nature spots in Paris...
📍 Fetching nature spots in Nice...
📍 Fetching nature spots in Lyon...
📍 Fetching nature spots in Marseille...
📍 Fetching nature spots in Toulouse...
📍 Fetching nature spots in Bordeaux...
📍 Fetching nature spots in Lille...
📍 Fetching nature spots in Strasbourg...
📍 Fetching nature spots in Montpellier...
📍 Fetching nature spots in Cannes...
✅ Collected 167 nature spots across 10 cities in France!


### Weather

In [71]:
import os
import requests
import pandas as pd
import time

# Enter your OpenWeather API key
OPENWEATHER_KEY = input("Enter your OpenWeather API key: ")
WEATHER_URL = "https://api.openweathermap.org/data/2.5/weather"

CITIES_FRANCE = [
    {"name": "Paris", "lat": 48.8566, "lng": 2.3522},  
    {"name": "Nice", "lat": 43.7102, "lng": 7.2620},  
    {"name": "Lyon", "lat": 45.7640, "lng": 4.8357},  
    {"name": "Marseille", "lat": 43.2965, "lng": 5.3698},  
    {"name": "Toulouse", "lat": 43.6047, "lng": 1.4442},  
    {"name": "Bordeaux", "lat": 44.8378, "lng": -0.5792},  
    {"name": "Lille", "lat": 50.6292, "lng": 3.0573},  
    {"name": "Strasbourg", "lat": 48.5734, "lng": 7.7521},  
    {"name": "Montpellier", "lat": 43.6108, "lng": 3.8767},  
    {"name": "Cannes", "lat": 43.5528, "lng": 7.0174}  
]

# Function to fetch weather data based on latitude and longitude
def get_weather(lat, lon):
    params = {
        "lat": lat,
        "lon": lon,
        "appid": OPENWEATHER_KEY,
        "units": "metric"  # Celsius
    }
    response = requests.get(WEATHER_URL, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"❌ Error fetching weather for {lat}, {lon}: {response.status_code}")
        return None

# Collect weather data for French cities
all_weather_data = []

for city in CITIES_FRANCE:
    print(f"Fetching weather data for {city['name']}...")
    weather_data = get_weather(city["lat"], city["lng"])
    
    if weather_data:
        weather_info = {
            "city": city["name"],
            "latitude": city["lat"],
            "longitude": city["lng"],
            "temperature": weather_data["main"]["temp"],
            "weather": weather_data["weather"][0]["description"],
            "humidity": weather_data["main"]["humidity"],
            "wind_speed": weather_data["wind"]["speed"]
        }
        all_weather_data.append(weather_info)
    time.sleep(1)  # Avoid exceeding API limits

# Create DataFrame and save weather data
df_weather = pd.DataFrame(all_weather_data)
df_weather.to_csv("france_weather_data.csv", index=False)

print(f"✅ Weather data for {len(df_weather)} French cities collected and saved!")


Fetching weather data for Paris...
Fetching weather data for Nice...
Fetching weather data for Lyon...
Fetching weather data for Marseille...
Fetching weather data for Toulouse...
Fetching weather data for Bordeaux...
Fetching weather data for Lille...
Fetching weather data for Strasbourg...
Fetching weather data for Montpellier...
Fetching weather data for Cannes...
✅ Weather data for 10 French cities collected and saved!


## Turkey

### Tourist Attractions

In [33]:
import requests
import pandas as pd
import time

# 🔑 Insert your Google API Key here
API_KEY = input("Enter your Google API key: ")
PLACES_URL = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# 📍 Top 10 Most Visited Cities in Turkey with Coordinates & Search Radius (in meters)
CITIES_TURKEY = [
    {"name": "Istanbul", "lat": 41.0082, "lng": 28.9784, "radius": 20000},  
    {"name": "Antalya", "lat": 36.8969, "lng": 30.7133, "radius": 18000},  
    {"name": "Cappadocia", "lat": 38.6400, "lng": 34.8300, "radius": 15000},  
    {"name": "Ankara", "lat": 39.9208, "lng": 32.8541, "radius": 15000},  
    {"name": "Izmir", "lat": 38.4192, "lng": 27.1287, "radius": 12000},  
    {"name": "Bodrum", "lat": 37.0344, "lng": 27.4305, "radius": 12000},  
    {"name": "Pamukkale", "lat": 37.9190, "lng": 29.1187, "radius": 10000},  
    {"name": "Trabzon", "lat": 41.0015, "lng": 39.7178, "radius": 12000},  
    {"name": "Fethiye", "lat": 36.6210, "lng": 29.1163, "radius": 10000},  
    {"name": "Ephesus", "lat": 37.9395, "lng": 27.3410, "radius": 15000}  
]

# Function to fetch tourist attractions for a given city
def get_places(city_name, lat, lng, radius, type_place="tourist_attraction"):
    params = {
        "location": f"{lat},{lng}",
        "radius": radius,
        "type": type_place,
        "key": API_KEY
    }
    
    places = []
    next_page_token = None

    print(f"📍 Fetching attractions in {city_name}...")
    
    while True:
        if next_page_token:
            params["pagetoken"] = next_page_token
        
        response = requests.get(PLACES_URL, params=params)
        data = response.json()
        
        if response.status_code != 200 or "results" not in data:
            print(f"⚠️ Error fetching data for {city_name}: {data.get('status', 'Unknown error')}")
            break
        
        places.extend(data["results"])
        
        next_page_token = data.get("next_page_token")
        if not next_page_token:
            break
        
        time.sleep(2)  # Google requires a small delay before using next_page_token

    return places

# Collect data for all cities
all_places = []
seen_places = set()

for city in CITIES_TURKEY:
    places = get_places(city["name"], city["lat"], city["lng"], city["radius"])
    
    for p in places:
        place_id = p["place_id"]
        
        if place_id not in seen_places:
            seen_places.add(place_id)
            all_places.append({
                "city": city["name"],
                "place_id": place_id,
                "name": p.get("name", "Unknown"),
                "rating": p.get("rating", "N/A"),
                "address": p.get("vicinity", "Unknown"),
                "latitude": p["geometry"]["location"]["lat"],
                "longitude": p["geometry"]["location"]["lng"],
                "types": ", ".join(p.get("types", []))
            })

    time.sleep(5)  # Avoid excessive API requests

# Save data to CSV
df_places = pd.DataFrame(all_places)
df_places.to_csv("turkey_tourist_attractions.csv", index=False)

print(f"✅ Collected {len(df_places)} tourist attractions across 10 cities in Turkey!")


📍 Fetching attractions in Istanbul...
📍 Fetching attractions in Antalya...
📍 Fetching attractions in Cappadocia...
📍 Fetching attractions in Ankara...
📍 Fetching attractions in Izmir...
📍 Fetching attractions in Bodrum...
📍 Fetching attractions in Pamukkale...
📍 Fetching attractions in Trabzon...
📍 Fetching attractions in Fethiye...
📍 Fetching attractions in Ephesus...
✅ Collected 568 tourist attractions across 10 cities in Turkey!


### Events

In [26]:
import requests
import pandas as pd
import time

# 🔑 Solicita a chave da API do usuário
API_KEY = input("Enter your Google API key: ")
PLACES_URL = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# 📍 Top 10 Most Visited Cities in Turkey with Coordinates & Search Radius (in meters)
CITIES_TURKEY = [
    {"name": "Istanbul", "lat": 41.0082, "lng": 28.9784, "radius": 20000},
    {"name": "Antalya", "lat": 36.8969, "lng": 30.7133, "radius": 15000},
    {"name": "Cappadocia", "lat": 38.6456, "lng": 34.8318, "radius": 12000},
    {"name": "Ankara", "lat": 39.9208, "lng": 32.8541, "radius": 15000},
    {"name": "Izmir", "lat": 38.4192, "lng": 27.1287, "radius": 12000},
    {"name": "Bodrum", "lat": 37.0344, "lng": 27.4305, "radius": 10000},
    {"name": "Pamukkale", "lat": 37.9189, "lng": 29.1266, "radius": 8000},
    {"name": "Trabzon", "lat": 41.0053, "lng": 39.7267, "radius": 10000},
    {"name": "Fethiye", "lat": 36.6200, "lng": 29.1171, "radius": 10000},
    {"name": "Ephesus", "lat": 37.9499, "lng": 27.3680, "radius": 8000}
]

# 🎭 Define categories of places where events may occur
EVENT_TYPES = [
    "stadium", "casino", "night_club", "art_gallery", "museum",
    "tourist_attraction", "amusement_park", "university", "movie_theater"
]

# Function to fetch event venues for a city
def get_places(city_name, lat, lng, radius):
    places = []
    seen_places = set()

    for place_type in EVENT_TYPES:
        params = {
            "location": f"{lat},{lng}",
            "radius": radius,
            "type": place_type,
            "key": API_KEY
        }
        
        print(f"📍 Fetching event venues in {city_name} ({place_type})...")
        
        response = requests.get(PLACES_URL, params=params)
        data = response.json()

        if response.status_code == 200 and "results" in data:
            for p in data["results"]:
                place_id = p["place_id"]

                if place_id not in seen_places:
                    seen_places.add(place_id)
                    places.append({
                        "city": city_name,
                        "place_id": place_id,
                        "name": p.get("name", "Unknown"),
                        "rating": p.get("rating", "N/A"),
                        "address": p.get("vicinity", "Unknown"),
                        "latitude": p["geometry"]["location"]["lat"],
                        "longitude": p["geometry"]["location"]["lng"],
                        "types": ", ".join(p.get("types", []))
                    })
        
        # Wait a bit to avoid exceeding API request limits
        time.sleep(2)
    
    return places

# 🔍 Collect event venue data for all cities
all_places = []
for city in CITIES_TURKEY:
    all_places.extend(get_places(city["name"], city["lat"], city["lng"], city["radius"]))

# 📝 Save data to CSV
df_places = pd.DataFrame(all_places)
df_places.to_csv("turkey_event_locations.csv", index=False)

print(f"✅ Collected {len(df_places)} event venues across 10 cities in Turkey! Saved as 'turkey_event_locations.csv'")


📍 Fetching event venues in Istanbul (stadium)...
📍 Fetching event venues in Istanbul (casino)...
📍 Fetching event venues in Istanbul (night_club)...
📍 Fetching event venues in Istanbul (art_gallery)...
📍 Fetching event venues in Istanbul (museum)...
📍 Fetching event venues in Istanbul (tourist_attraction)...
📍 Fetching event venues in Istanbul (amusement_park)...
📍 Fetching event venues in Istanbul (university)...
📍 Fetching event venues in Istanbul (movie_theater)...
📍 Fetching event venues in Antalya (stadium)...
📍 Fetching event venues in Antalya (casino)...
📍 Fetching event venues in Antalya (night_club)...
📍 Fetching event venues in Antalya (art_gallery)...
📍 Fetching event venues in Antalya (museum)...
📍 Fetching event venues in Antalya (tourist_attraction)...
📍 Fetching event venues in Antalya (amusement_park)...
📍 Fetching event venues in Antalya (university)...
📍 Fetching event venues in Antalya (movie_theater)...
📍 Fetching event venues in Cappadocia (stadium)...
📍 Fetching e

### Hotels

In [50]:
import requests 
import pandas as pd
import time

OVERPASS_URL = "https://overpass-api.de/api/interpreter"

CITIES_TURKEY = [
    {"name": "Istanbul", "lat": 41.0082, "lng": 28.9784, "radius": 20000},  
    {"name": "Antalya", "lat": 36.8969, "lng": 30.7133, "radius": 18000},  
    {"name": "Cappadocia", "lat": 38.6400, "lng": 34.8300, "radius": 15000},  
    {"name": "Ankara", "lat": 39.9208, "lng": 32.8541, "radius": 15000},  
    {"name": "Izmir", "lat": 38.4192, "lng": 27.1287, "radius": 12000},  
    {"name": "Bodrum", "lat": 37.0344, "lng": 27.4305, "radius": 12000},  
    {"name": "Pamukkale", "lat": 37.9190, "lng": 29.1187, "radius": 10000},  
    {"name": "Trabzon", "lat": 41.0015, "lng": 39.7178, "radius": 12000},  
    {"name": "Fethiye", "lat": 36.6210, "lng": 29.1163, "radius": 10000},  
    {"name": "Ephesus", "lat": 37.9395, "lng": 27.3410, "radius": 15000}  
]

def get_osm_hotels(lat, lon):
    query = f"""
    [out:json];
    (
      node["tourism"="hotel"](around:10000,{lat},{lon});
      node["tourism"="hostel"](around:10000,{lat},{lon});
      node["tourism"="guest_house"](around:10000,{lat},{lon});
    );
    out center;
    """
    
    response = requests.get(OVERPASS_URL, params={"data": query})
    return response.json().get("elements", []) if response.status_code == 200 else []

all_hotels = []
for city in CITIES_TURKEY:
    print(f"📍 Fetching hotels and hostels in {city['name']}...")
    hotels = get_osm_hotels(city["lat"], city["lng"])

    for hotel in hotels:
        tags = hotel.get("tags", {})

        all_hotels.append({
            "city": city["name"],
            "name": tags.get("name", "Unknown"),
            "type": tags.get("tourism", "Unknown"),
            "latitude": hotel.get("lat"),
            "longitude": hotel.get("lon"),
            "stars": tags.get("stars", "Unknown"),
            "operator": tags.get("operator", "Unknown")
        })
    time.sleep(2)

df_hotels = pd.DataFrame(all_hotels)
df_hotels.to_csv("turkey_hotels_hostels.csv", index=False)

print(f"✅ Collected {len(df_hotels)} hotels and hostels across 10 cities in Turkey!")


📍 Fetching hotels and hostels in Istanbul...
📍 Fetching hotels and hostels in Antalya...
📍 Fetching hotels and hostels in Cappadocia...
📍 Fetching hotels and hostels in Ankara...
📍 Fetching hotels and hostels in Izmir...
📍 Fetching hotels and hostels in Bodrum...
📍 Fetching hotels and hostels in Pamukkale...
📍 Fetching hotels and hostels in Trabzon...
📍 Fetching hotels and hostels in Fethiye...
📍 Fetching hotels and hostels in Ephesus...
✅ Collected 2479 hotels and hostels across 10 cities in Turkey!


### Restaurants

In [27]:
import requests
import pandas as pd
import time

OVERPASS_URL = "https://overpass-api.de/api/interpreter"

# 📍 Top 10 Most Visited Cities in Turkey
CITIES_TURKEY = [
    {"name": "Istanbul", "lat": 41.0082, "lon": 28.9784, "radius": 10000},
    {"name": "Antalya", "lat": 36.8969, "lon": 30.7133, "radius": 8000},
    {"name": "Cappadocia", "lat": 38.6450, "lon": 34.8270, "radius": 8000},
    {"name": "Ankara", "lat": 39.9208, "lon": 32.8541, "radius": 8000},
    {"name": "Izmir", "lat": 38.4192, "lon": 27.1287, "radius": 8000},
    {"name": "Bodrum", "lat": 37.0344, "lon": 27.4305, "radius": 7000},
    {"name": "Pamukkale", "lat": 37.9206, "lon": 29.1187, "radius": 7000},
    {"name": "Trabzon", "lat": 41.0053, "lon": 39.7264, "radius": 7000},
    {"name": "Fethiye", "lat": 36.6219, "lon": 29.1160, "radius": 7000},
    {"name": "Ephesus", "lat": 37.9499, "lon": 27.3636, "radius": 6000}
]

def get_osm_restaurants(city):
    """Fetches restaurants, cafés, and bars in a specific Turkish city with retries."""
    query = f"""
    [out:json];
    (
      node["amenity"="restaurant"](around:{city["radius"]},{city["lat"]},{city["lon"]});
      node["amenity"="cafe"](around:{city["radius"]},{city["lat"]},{city["lon"]});
      node["amenity"="bar"](around:{city["radius"]},{city["lat"]},{city["lon"]});
    );
    out center;
    """

    retries = 3
    while retries > 0:
        response = requests.get(OVERPASS_URL, params={"data": query})
        
        if response.status_code == 200:
            return response.json().get("elements", [])
        else:
            print(f"⚠️ Error fetching restaurants in {city['name']} ({response.status_code}), retrying... ({3 - retries + 1}/3)")
            retries -= 1
            time.sleep(3)  # Aguardar antes de tentar novamente
    
    print(f"❌ Failed to fetch data for {city['name']} after multiple retries.")
    return []

# 🔍 Collect data for all cities
all_places = []
for city in CITIES_TURKEY:
    print(f"📍 Fetching gastronomic locations in {city['name']}...")
    places = get_osm_restaurants(city)

    for place in places:
        tags = place.get("tags", {})

        all_places.append({
            "city": city["name"],
            "name": tags.get("name", "Unknown"),
            "type": tags.get("amenity", "Unknown"),
            "latitude": place.get("lat"),
            "longitude": place.get("lon"),
            "street": tags.get("addr:street", "Unknown"),
            "postcode": tags.get("addr:postcode", "Unknown"),
            "cuisine": tags.get("cuisine", "Not specified")  # Tipo de comida
        })

    time.sleep(2)  # Evitar sobrecarga na API

# 📝 Save data to CSV
df = pd.DataFrame(all_places)
df.to_csv("turkey_all_restaurants_bars_cafes.csv", index=False)

print(f"✅ Collected {len(df)} gastronomic locations across 10 cities in Turkey! Saved as 'turkey_all_restaurants_bars_cafes.csv'")


📍 Fetching gastronomic locations in Istanbul...
📍 Fetching gastronomic locations in Antalya...
📍 Fetching gastronomic locations in Cappadocia...
📍 Fetching gastronomic locations in Ankara...
📍 Fetching gastronomic locations in Izmir...
📍 Fetching gastronomic locations in Bodrum...
📍 Fetching gastronomic locations in Pamukkale...
📍 Fetching gastronomic locations in Trabzon...
📍 Fetching gastronomic locations in Fethiye...
📍 Fetching gastronomic locations in Ephesus...
✅ Collected 8299 gastronomic locations across 10 cities in Turkey! Saved as 'turkey_all_restaurants_bars_cafes.csv'


### Public Transport Stops

In [38]:
import requests
import pandas as pd

OVERPASS_URL = "https://overpass-api.de/api/interpreter"

CITIES_TURKEY = [
    {"name": "Istanbul", "lat": 41.0082, "lon": 28.9784, "radius": 10000},
    {"name": "Antalya", "lat": 36.8969, "lon": 30.7133, "radius": 8000},
    {"name": "Cappadocia", "lat": 38.6450, "lon": 34.8270, "radius": 8000},
    {"name": "Ankara", "lat": 39.9208, "lon": 32.8541, "radius": 8000},
    {"name": "Izmir", "lat": 38.4192, "lon": 27.1287, "radius": 8000},
    {"name": "Bodrum", "lat": 37.0344, "lon": 27.4305, "radius": 7000},
    {"name": "Pamukkale", "lat": 37.9206, "lon": 29.1187, "radius": 7000},
    {"name": "Trabzon", "lat": 41.0053, "lon": 39.7264, "radius": 7000},
    {"name": "Fethiye", "lat": 36.6219, "lon": 29.1160, "radius": 7000},
    {"name": "Ephesus", "lat": 37.9499, "lon": 27.3636, "radius": 6000}
]

def get_osm_transit_stops(city):
    query = f"""
    [out:json];
    (
      node["highway"="bus_stop"](around:{city["radius"]},{city["lat"]},{city["lon"]});
      node["railway"="station"](around:{city["radius"]},{city["lat"]},{city["lon"]});
      node["public_transport"="platform"](around:{city["radius"]},{city["lat"]},{city["lon"]});
    );
    out center;
    """
    
    response = requests.get(OVERPASS_URL, params={"data": query})
    return response.json().get("elements", []) if response.status_code == 200 else []

all_stops = []
for city in CITIES_TURKEY:
    print(f"📍 Fetching public transport stops in {city['name']}...")
    stops = get_osm_transit_stops(city)

    for stop in stops:
        tags = stop.get("tags", {})

        all_stops.append({
            "city": city["name"],
            "name": tags.get("name", "Unknown"),
            "type": tags.get("highway", tags.get("railway", tags.get("public_transport", "Unknown"))),
            "latitude": stop.get("lat"),
            "longitude": stop.get("lon"),
            "operator": tags.get("operator", "Unknown")
        })

df_transit = pd.DataFrame(all_stops)
df_transit.to_csv("turkey_transit_stops.csv", index=False)

print(f"✅ Collected {len(df_transit)} public transport stops across 10 cities in Turkey!")


📍 Fetching public transport stops in Istanbul...
📍 Fetching public transport stops in Antalya...
📍 Fetching public transport stops in Cappadocia...
📍 Fetching public transport stops in Ankara...
📍 Fetching public transport stops in Izmir...
📍 Fetching public transport stops in Bodrum...
📍 Fetching public transport stops in Pamukkale...
📍 Fetching public transport stops in Trabzon...
📍 Fetching public transport stops in Fethiye...
📍 Fetching public transport stops in Ephesus...
✅ Collected 5527 public transport stops across 10 cities in Turkey!


### Nature Spots

In [59]:
import requests
import pandas as pd
import time

OVERPASS_URL = "https://overpass-api.de/api/interpreter"

CITIES_TURKEY = [
    {"name": "Istanbul", "lat": 41.0082, "lng": 28.9784, "radius": 20000},  
    {"name": "Antalya", "lat": 36.8969, "lng": 30.7133, "radius": 18000},  
    {"name": "Cappadocia", "lat": 38.6400, "lng": 34.8300, "radius": 15000},  
    {"name": "Ankara", "lat": 39.9208, "lng": 32.8541, "radius": 15000},  
    {"name": "Izmir", "lat": 38.4192, "lng": 27.1287, "radius": 12000},  
    {"name": "Bodrum", "lat": 37.0344, "lng": 27.4305, "radius": 12000},  
    {"name": "Pamukkale", "lat": 37.9190, "lng": 29.1187, "radius": 10000},  
    {"name": "Trabzon", "lat": 41.0015, "lng": 39.7178, "radius": 12000},  
    {"name": "Fethiye", "lat": 36.6210, "lng": 29.1163, "radius": 10000},  
    {"name": "Ephesus", "lat": 37.9395, "lng": 27.3410, "radius": 15000}  
]

def get_osm_nature_spots(lat, lon):
    query = f"""
    [out:json];
    (
      node["leisure"="nature_reserve"](around:10000,{lat},{lon});
      node["leisure"="park"](around:10000,{lat},{lon});
      node["route"="hiking"](around:10000,{lat},{lon});
      node["piste:type"="downhill"](around:10000,{lat},{lon});
      node["natural"="peak"](around:10000,{lat},{lon});
    );
    out center;
    """
    
    response = requests.get(OVERPASS_URL, params={"data": query})
    return response.json().get("elements", []) if response.status_code == 200 else []

all_nature_spots = []
for city in CITIES_TURKEY:
    print(f"📍 Fetching nature spots in {city['name']}...")
    nature_spots = get_osm_nature_spots(city["lat"], city["lng"])

    for spot in nature_spots:
        tags = spot.get("tags", {})

        all_nature_spots.append({
            "city": city["name"],
            "name": tags.get("name", "Unknown"),
            "type": tags.get("leisure", tags.get("route", tags.get("natural", "Unknown"))),
            "latitude": spot.get("lat"),
            "longitude": spot.get("lon"),
            "elevation": tags.get("ele", "Unknown")
        })
    time.sleep(2)

df_nature = pd.DataFrame(all_nature_spots)
df_nature.to_csv("turkey_nature_spots.csv", index=False)

print(f"✅ Collected {len(df_nature)} nature spots across 10 cities in Turkey!")


📍 Fetching nature spots in Istanbul...
📍 Fetching nature spots in Antalya...
📍 Fetching nature spots in Cappadocia...
📍 Fetching nature spots in Ankara...
📍 Fetching nature spots in Izmir...
📍 Fetching nature spots in Bodrum...
📍 Fetching nature spots in Pamukkale...
📍 Fetching nature spots in Trabzon...
📍 Fetching nature spots in Fethiye...
📍 Fetching nature spots in Ephesus...
✅ Collected 272 nature spots across 10 cities in Turkey!


### Weather

In [72]:
import os
import requests
import pandas as pd
import time

# Enter your OpenWeather API key
OPENWEATHER_KEY = input("Enter your OpenWeather API key: ")
WEATHER_URL = "https://api.openweathermap.org/data/2.5/weather"

CITIES_TURKEY = [
    {"name": "Istanbul", "lat": 41.0082, "lng": 28.9784},  
    {"name": "Antalya", "lat": 36.8969, "lng": 30.7133},  
    {"name": "Cappadocia", "lat": 38.6400, "lng": 34.8300},  
    {"name": "Ankara", "lat": 39.9208, "lng": 32.8541},  
    {"name": "Izmir", "lat": 38.4192, "lng": 27.1287},  
    {"name": "Bodrum", "lat": 37.0344, "lng": 27.4305},  
    {"name": "Pamukkale", "lat": 37.9190, "lng": 29.1187},  
    {"name": "Trabzon", "lat": 41.0015, "lng": 39.7178},  
    {"name": "Fethiye", "lat": 36.6210, "lng": 29.1163},  
    {"name": "Ephesus", "lat": 37.9395, "lng": 27.3410}  
]

# Function to fetch weather data based on latitude and longitude
def get_weather(lat, lon):
    params = {
        "lat": lat,
        "lon": lon,
        "appid": OPENWEATHER_KEY,
        "units": "metric"  # Celsius
    }
    response = requests.get(WEATHER_URL, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"❌ Error fetching weather for {lat}, {lon}: {response.status_code}")
        return None

# Collect weather data for Turkish cities
all_weather_data = []

for city in CITIES_TURKEY:
    print(f"Fetching weather data for {city['name']}...")
    weather_data = get_weather(city["lat"], city["lng"])
    
    if weather_data:
        weather_info = {
            "city": city["name"],
            "latitude": city["lat"],
            "longitude": city["lng"],
            "temperature": weather_data["main"]["temp"],
            "weather": weather_data["weather"][0]["description"],
            "humidity": weather_data["main"]["humidity"],
            "wind_speed": weather_data["wind"]["speed"]
        }
        all_weather_data.append(weather_info)
    time.sleep(1)  # Avoid exceeding API limits

# Create DataFrame and save weather data
df_weather = pd.DataFrame(all_weather_data)
df_weather.to_csv("turkey_weather_data.csv", index=False)

print(f"✅ Weather data for {len(df_weather)} Turkish cities collected and saved!")


Fetching weather data for Istanbul...
Fetching weather data for Antalya...
Fetching weather data for Cappadocia...
Fetching weather data for Ankara...
Fetching weather data for Izmir...
Fetching weather data for Bodrum...
Fetching weather data for Pamukkale...
Fetching weather data for Trabzon...
Fetching weather data for Fethiye...
Fetching weather data for Ephesus...
✅ Weather data for 10 Turkish cities collected and saved!
